![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Spark NLP for Healthcare Workshop - 3 hr

This is the 3 hr workshop version of the entire SparkNLP healthcare training notebooks : https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/tutorials/Certification_Trainings/Healthcare

In [0]:
import functools 
import numpy as np
import pandas as pd
from scipy import spatial

import pyspark.sql.types as T
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *


pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 4.2.4
sparknlp_jsl.version : 4.2.4
Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

# 1.Clinical Named Entity Recognition (NER)

The deep neural network architecture for NER model in
Spark NLP is BiLSTM-CNN-Char framework. a slightly modified version of the architecture proposed by Jason PC Chiu and Eric Nichols ([Named Entity Recognition with Bidirectional LSTM-CNNs
](https://arxiv.org/abs/1511.08308)). It is a neural network architecture that
automatically detects word and character-level features using a
hybrid bidirectional LSTM and CNN architecture, eliminating
the need for most feature engineering steps.

  In the original framework, the CNN extracts a fixed length
feature vector from character-level features. For each word,
these vectors are concatenated and fed to the BLSTM network
and then to the output layers. They employed a stacked
bi-directional recurrent neural network with long short-term
memory units to transform word features into named entity
tag scores. The extracted features of each word are fed into a
forward LSTM network and a backward LSTM network. The
output of each network at each time step is decoded by a linear
layer and a log-softmax layer into log-probabilities for each tag
category. These two vectors are then simply added together to
produce the final output. In the architecture of the proposed framework in the original paper, 50-dimensional pretrained word
embeddings is used for word features, 25-dimension character
embeddings is used for char features, and capitalization features
(allCaps, upperInitial, lowercase, mixedCaps, noinfo) are used
for case features.

## Blogposts and videos:

- [How to Setup Spark NLP for HEALTHCARE on UBUNTU - Video](https://www.youtube.com/watch?v=yKnF-_oz0GE)

- [Named Entity Recognition (NER) with BERT in Spark NLP](https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77)

- [State of the art Clinical Named Entity Recognition in Spark NLP - Youtube](https://www.youtube.com/watch?v=YM-e4eOiQ34)

- [Named Entity Recognition for Healthcare with SparkNLP NerDL and NerCRF](https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571)

- [Named Entity Recognition for Clinical Text](https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180)

For more details about NER with SparkNLP, please visit [SparkNLP Workshop Repo](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.Clinical_Named_Entity_Recognition_Model.ipynb)

## Clinical NER Pipeline (with pretrained models)

SparkNLP has more than a hundred pretrained clinical NER pipelines which are trained with various domain-specific datasets and have various architectures. <br/>

**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

**CLINICAL NER MODELS**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_clinical_en.html)  | 2| [jsl_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_greedy_clinical_en.html)  | 3| [jsl_ner_wip_modifier_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_ner_wip_modifier_clinical_en.html)  | 4| [jsl_rd_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_rd_ner_wip_greedy_clinical_en.html)  |
| 5| [ner_abbreviation_clinical](https://nlp.johnsnowlabs.com/2021/12/30/ner_abbreviation_clinical_en.html)  | 6| [ner_ade_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinical_en.html)  | 7| [ner_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinicalbert_en.html)  | 8| [ner_ade_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_healthcare_en.html)  |
| 9| [ner_anatomy](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_en.html)  | 10| [ner_anatomy_coarse](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_en.html)  | 11| [ner_anatomy_coarse_en](https://nlp.johnsnowlabs.com/2020/11/04/ner_anatomy_coarse_en.html)  | 12| [ner_anatomy_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_anatomy_en.html)  |
| 13| [ner_aspect_based_sentiment](https://nlp.johnsnowlabs.com/2021/03/31/ner_aspect_based_sentiment_en.html)  | 14| [ner_bacterial_species](https://nlp.johnsnowlabs.com/2021/04/01/ner_bacterial_species_en.html)  | 15| [ner_biomarker](https://nlp.johnsnowlabs.com/2021/11/26/ner_biomarker_en.html)  | 16| [ner_biomedical_bc2gm](https://nlp.johnsnowlabs.com/2022/05/11/ner_biomedical_bc2gm_en_2_4.html)  |
| 17| [ner_bionlp](https://nlp.johnsnowlabs.com/2021/03/31/ner_bionlp_en.html)  | 18| [ner_bionlp_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_bionlp_en.html)  | 19| [ner_cancer_genetics](https://nlp.johnsnowlabs.com/2021/03/31/ner_cancer_genetics_en.html)  | 20| [ner_cellular](https://nlp.johnsnowlabs.com/2021/03/31/ner_cellular_en.html)  |
| 21| [ner_cellular_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_cellular_en.html)  | 22| [ner_chemd_clinical](https://nlp.johnsnowlabs.com/2021/11/04/ner_chemd_clinical_en.html)  | 23| [ner_chemicals](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemicals_en.html)  | 24| [ner_chemprot_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_chemprot_clinical_en.html)  |
| 25| [ner_chexpert](https://nlp.johnsnowlabs.com/2021/09/30/ner_chexpert_en.html)  | 26| [ner_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_en.html)  | 27| [ner_clinical_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_clinical_en.html)  | 28| [ner_clinical_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_large_en.html)  |
| 29| [ner_clinical_large_en](https://nlp.johnsnowlabs.com/2020/05/23/ner_clinical_large_en.html)  | 30| [ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/22/ner_clinical_trials_abstracts_en_3_0.html)  | 31| [ner_covid_trials](https://nlp.johnsnowlabs.com/2022/10/19/ner_covid_trials_en.html)  | 32| [ner_crf](https://nlp.johnsnowlabs.com/2020/01/28/ner_crf_en.html)  |
| 33| [ner_deid_augmented](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_augmented_en.html)  | 34| [ner_deid_enriched](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_enriched_en.html)  | 35| [ner_deid_generic_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_generic_glove_en.html)  | 36| [ner_deid_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_large_en.html)  |
| 37| [ner_deid_sd](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_en.html)  | 38| [ner_deid_sd_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_large_en.html)  | 39| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2021/09/03/ner_deid_subentity_augmented_en.html)  | 40| [ner_deid_subentity_augmented_i2b2](https://nlp.johnsnowlabs.com/2021/11/29/ner_deid_subentity_augmented_i2b2_en.html)  |
| 41| [ner_deid_subentity_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_subentity_glove_en.html)  | 42| [ner_deid_synthetic](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_synthetic_en.html)  | 43| [ner_deidentify_dl](https://nlp.johnsnowlabs.com/2021/03/31/ner_deidentify_dl_en.html)  | 44| [ner_diseases](https://nlp.johnsnowlabs.com/2021/03/31/ner_diseases_en.html)  |
| 45| [ner_diseases_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_diseases_en.html)  | 46| [ner_diseases_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_large_en.html)  | 47| [ner_drugprot_clinical](https://nlp.johnsnowlabs.com/2021/12/20/ner_drugprot_clinical_en.html)  | 48| [ner_drugs](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_en.html)  |
| 49| [ner_drugs_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_drugs_en.html)  | 50| [ner_drugs_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_greedy_en.html)  | 51| [ner_drugs_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_large_en.html)  | 52| [ner_drugs_large_en](https://nlp.johnsnowlabs.com/2021/01/29/ner_drugs_large_en.html)  |
| 53| [ner_events_admission_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_admission_clinical_en.html)  | 54| [ner_events_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_clinical_en.html)  | 55| [ner_events_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_healthcare_en.html)  | 56| [ner_financial_contract](https://nlp.johnsnowlabs.com/2021/04/01/ner_financial_contract_en.html)  |
| 57| [ner_genetic_variants](https://nlp.johnsnowlabs.com/2021/06/25/ner_genetic_variants_en.html)  | 58| [ner_healthcare](https://nlp.johnsnowlabs.com/2021/04/21/ner_healthcare_en.html)  | 59| [ner_healthcare_en](https://nlp.johnsnowlabs.com/2020/03/26/ner_healthcare_en.html)  | 60| [ner_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_gene_clinical_en.html)  |
| 61| [ner_human_phenotype_go_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_go_clinical_en.html)  | 62| [ner_jsl](https://nlp.johnsnowlabs.com/2022/10/19/ner_jsl_en.html)  | 63| [ner_jsl_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_en.html)  | 64| [ner_jsl_enriched](https://nlp.johnsnowlabs.com/2021/10/22/ner_jsl_enriched_en.html)  |
| 65| [ner_jsl_enriched_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_enriched_en.html)  | 66| [ner_jsl_greedy](https://nlp.johnsnowlabs.com/2021/06/24/ner_jsl_greedy_en.html)  | 67| [ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_slim_en.html)  | 68| [ner_living_species](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_en_3_0.html)  |
| 69| [ner_measurements_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_measurements_clinical_en.html)  | 70| [ner_medmentions_coarse](https://nlp.johnsnowlabs.com/2021/04/01/ner_medmentions_coarse_en.html)  | 71| [ner_nature_nero_clinical](https://nlp.johnsnowlabs.com/2022/02/08/ner_nature_nero_clinical_en.html)  | 72| [ner_nihss](https://nlp.johnsnowlabs.com/2021/11/15/ner_nihss_en.html)  |
| 73| [ner_oncology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_en.html)  | 74| [ner_oncology_anatomy_general](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_general_en.html)  | 75| [ner_oncology_anatomy_granular](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_granular_en.html)  | 76| [ner_oncology_biomarker](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_biomarker_en.html)  |
| 77| [ner_oncology_demographics](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_demographics_en.html)  | 78| [ner_oncology_diagnosis](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_diagnosis_en.html)  | 79| [ner_oncology_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_posology_en.html)  | 80| [ner_oncology_response_to_treatment](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_response_to_treatment_en.html)  |
| 81| [ner_oncology_test](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_test_en.html)  | 82| [ner_oncology_therapy](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_therapy_en.html)  | 83| [ner_oncology_tnm](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_tnm_en.html)  | 84| [ner_oncology_unspecific_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_unspecific_posology_en.html)  |
| 85| [ner_pathogen](https://nlp.johnsnowlabs.com/2022/06/28/ner_pathogen_en_3_0.html)  | 86| [ner_posology](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_en.html)  | 87| [ner_posology_en](https://nlp.johnsnowlabs.com/2020/04/15/ner_posology_en.html)  | 88| [ner_posology_experimental](https://nlp.johnsnowlabs.com/2021/09/01/ner_posology_experimental_en.html)  |
| 89| [ner_posology_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_greedy_en.html)  | 90| [ner_posology_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_healthcare_en.html)  | 91| [ner_posology_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_large_en.html)  | 92| [ner_posology_small](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_small_en.html)  |
| 93| [ner_radiology](https://nlp.johnsnowlabs.com/2021/03/31/ner_radiology_en.html)  | 94| [ner_radiology_wip_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_radiology_wip_clinical_en.html)  | 95| [ner_risk_factors](https://nlp.johnsnowlabs.com/2021/03/31/ner_risk_factors_en.html)  | 96| [ner_risk_factors_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_risk_factors_en.html)  |
| 97| [ner_sdoh_mentions](https://nlp.johnsnowlabs.com/2022/12/18/ner_sdoh_mentions_en.html)  | 98| [ner_sdoh_mentions_test](https://nlp.johnsnowlabs.com/2022/12/18/ner_sdoh_mentions_test_en.html)  | 99| [ner_supplement_clinical](https://nlp.johnsnowlabs.com/2022/02/01/ner_supplement_clinical_en.html)  | 100| [nerdl_tumour_demo](https://nlp.johnsnowlabs.com/2021/04/01/nerdl_tumour_demo_en.html)  |




- **BioBert NER Models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_ner_wip_greedy_biobert_en.html)  | 2| [jsl_rd_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_rd_ner_wip_greedy_biobert_en.html)  | 3| [ner_ade_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_biobert_en.html)  | 4| [ner_anatomy_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_anatomy_biobert_en.html)  |
| 5| [ner_anatomy_coarse_biobert](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_biobert_en.html)  | 6| [ner_bionlp_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_bionlp_biobert_en.html)  | 7| [ner_cellular_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_cellular_biobert_en.html)  | 8| [ner_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemprot_biobert_en.html)  |
| 9| [ner_clinical_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_clinical_biobert_en.html)  | 10| [ner_deid_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_biobert_en.html)  | 11| [ner_deid_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_enriched_biobert_en.html)  | 12| [ner_diseases_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_biobert_en.html)  |
| 13| [ner_events_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_biobert_en.html)  | 14| [ner_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_gene_biobert_en.html)  | 15| [ner_human_phenotype_go_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_go_biobert_en.html)  | 16| [ner_jsl_biobert](https://nlp.johnsnowlabs.com/2021/09/05/ner_jsl_biobert_en.html)  |
| 17| [ner_jsl_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_jsl_enriched_biobert_en.html)  | 18| [ner_jsl_greedy_biobert](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_greedy_biobert_en.html)  | 19| [ner_living_species_biobert](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_biobert_en_3_0.html)  | 20| [ner_posology_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_biobert_en.html)  |
| 21| [ner_posology_large_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_large_biobert_en.html)  | 22| [ner_risk_factors_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_risk_factors_biobert_en.html)  | 23| []()| 24| []()|




- **BertForTokenClassification Clinical NER models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [bert_token_classifier_ade_tweet_binary](https://nlp.johnsnowlabs.com/2022/07/29/bert_token_classifier_ade_tweet_binary_en_3_0.html)  | 2| [bert_token_classifier_drug_development_trials](https://nlp.johnsnowlabs.com/2022/03/22/bert_token_classifier_drug_development_trials_en_3_0.html)  | 3| [bert_token_classifier_ner_ade](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_ade_en.html)  | 4| [bert_token_classifier_ner_ade_binary](https://nlp.johnsnowlabs.com/2022/07/27/bert_token_classifier_ner_ade_binary_en_3_0.html)  |
| 5| [bert_token_classifier_ner_anatem](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_anatem_en_3_0.html)  | 6| [bert_token_classifier_ner_anatomy](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_anatomy_en.html)  | 7| [bert_token_classifier_ner_bacteria](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_bacteria_en.html)  | 8| [bert_token_classifier_ner_bc2gm_gene](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc2gm_gene_en_3_0.html)  |
| 9| [bert_token_classifier_ner_bc4chemd_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc4chemd_chemicals_en_3_0.html)  | 10| [bert_token_classifier_ner_bc5cdr_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_chemicals_en_3_0.html)  | 11| [bert_token_classifier_ner_bc5cdr_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_disease_en_3_0.html)  | 12| [bert_token_classifier_ner_bionlp](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_bionlp_en.html)  |
| 13| [bert_token_classifier_ner_cellular](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_cellular_en.html)  | 14| [bert_token_classifier_ner_chemicals](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemicals_en.html)  | 15| [bert_token_classifier_ner_chemprot](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemprot_en.html)  | 16| [bert_token_classifier_ner_clinical](https://nlp.johnsnowlabs.com/2022/01/06/bert_token_classifier_ner_clinical_en.html)  |
| 17| [bert_token_classifier_ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/29/bert_token_classifier_ner_clinical_trials_abstracts_en_3_0.html)  | 18| [bert_token_classifier_ner_deid](https://nlp.johnsnowlabs.com/2021/09/13/bert_token_classifier_ner_deid_en.html)  | 19| [bert_token_classifier_ner_drugs](https://nlp.johnsnowlabs.com/2021/09/20/bert_token_classifier_ner_drugs_en.html)  | 20| [bert_token_classifier_ner_jnlpba_cellular](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_jnlpba_cellular_en_3_0.html)  |
| 21| [bert_token_classifier_ner_jsl](https://nlp.johnsnowlabs.com/2021/09/16/bert_token_classifier_ner_jsl_en.html)  | 22| [bert_token_classifier_ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/09/24/bert_token_classifier_ner_jsl_slim_en.html)  | 23| [bert_token_classifier_ner_linnaeus_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_linnaeus_species_en_3_0.html)  | 24| [bert_token_classifier_ner_living_species](https://nlp.johnsnowlabs.com/2022/06/26/bert_token_classifier_ner_living_species_en_3_0.html)  |
| 25| [bert_token_classifier_ner_ncbi_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_ncbi_disease_en_3_0.html)  | 26| [bert_token_classifier_ner_pathogen](https://nlp.johnsnowlabs.com/2022/07/28/bert_token_classifier_ner_pathogen_en_3_0.html)  | 27| [bert_token_classifier_ner_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_species_en_3_0.html)  | 28| [bert_token_classifier_ner_supplement](https://nlp.johnsnowlabs.com/2022/02/09/bert_token_classifier_ner_supplement_en.html)  |






**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

Let's start creating an example NER Pipeline. <br/>
In this pipeline, we will use `ner_jsl` pretrained model. By using this pretrained model, we can extract about 80 clinical entity labels by changing just only the ner model name in our pipeline.

**Entities**

| | | | | |
|-|-|-|-|-|
|Injury_or_Poisoning|Direction|Test|Admission_Discharge|Death_Entity|
|Relationship_Status|Duration|Respiration|Hyperlipidemia|Birth_Entity|
|Age|Labour_Delivery|Family_History_Header|BMI|Temperature|
|Alcohol|Kidney_Disease|Oncological|Medical_History_Header|Cerebrovascular_Disease|
|Oxygen_Therapy|O2_Saturation|Psychological_Condition|Heart_Disease|Employment|
|Obesity|Disease_Syndrome_Disorder|Pregnancy|ImagingFindings|Procedure|
|Medical_Device|Race_Ethnicity|Section_Header|Symptom|Treatment|
|Substance|Route|Drug_Ingredient|Blood_Pressure|Diet|
|External_body_part_or_region|LDL|VS_Finding|Allergen|EKG_Findings|
|Imaging_Technique|Triglycerides|RelativeTime|Gender|Pulse|
|Social_History_Header|Substance_Quantity|Diabetes|Modifier|Internal_organ_or_component|
|Clinical_Dept|Form|Drug_BrandName|Strength|Fetus_NewBorn|
|RelativeDate|Height|Test_Result|Sexually_Active_or_Sexual_Orientation|Frequency|
|Time|Weight|Vaccine|Vital_Signs_Header|Communicable_Disease|
|Dosage|Overweight|Hypertension|HDL|Total_Cholesterol|
|Smoking|Date||||

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
jsl_ner = MedicalNerModel.pretrained("ner_jsl","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("jsl_ner")\
    # .setLabelCasing("upper") #decide if we want to return the tags in upper or lower case 

jsl_ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("jsl_ner_chunk")

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        jsl_ner,
        jsl_ner_converter])


model = nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

We created our NER pipeline, let's check the stages.

In [0]:
model.stages

Out[3]: [DocumentAssembler_14fe686ebbb4,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_3ad62e5b2bbf,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_c89cbceb1028,
 NerConverter_b6cc0d5d693f]

Now, let's transform our pipeline with sample data and see how our NER model works.

In [0]:
# Downloading sample datasets (oncology-related).
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/oncology_notes/mt_oncology_0.txt

dbutils.fs.cp("file:/databricks/driver/mt_oncology_0.txt", "dbfs:/") 

Out[4]: True

In [0]:
df = spark.read.format("text").load("dbfs:/mt_oncology_0.txt")
df= df.withColumnRenamed("value", "text")
df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Sample Type / Medical Specialty: |
Hematology - Oncology |
Sample Name: |
Discharge Summary - Mesothelioma - 1 |
Description: |
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis. |
(Medical Transcription Sample Report) |
PRINCIPAL DIAGNOSIS: |
Mesothelioma. |
SECONDARY DIAGNOSES: |
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis. |
PROCEDURES |
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy. |
2. On August 20, 2007, thoracentesis. |
3. On August 31, 2007, Port-A-Cath placement. |
HISTORY AND PHYSICAL: |
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.|
PAST MEDICAL HISTORY |
1. Pericardectomy. |
2. Pericarditis. |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
df.count()

Out[6]: 54

In [0]:
#transform the pipeline with sample data
result = model.transform(df)

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| jsl_ner| jsl_ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Sample Type / Med...|[{document, 0, 31...|[{document, 0, 31...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|
Hematology - Onco...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 9, He...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, He...|
 Sample Name:|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|
Discharge Summary...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 8, Di...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 8, Di...|
 Description:|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 10, D...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, D...|
Mesothelioma, ple...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 11, M...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, M...|
(Medical Transcri...|[{document, 0, 36...|[{document, 0, 36...|[{token, 0, 0, (,...|[{word_embeddings...|[{named_entity, 0...| []|
PRINCIPAL DIAGNOSIS:|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 8, PR...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, P...|
 Mesothelioma.|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 11, M...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, M...|
SECONDARY DIAGNOSES:|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 8, SE...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, S...|
Pleural effusion,...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 6, Pl...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 15, P...|
 PROCEDURES|[{document, 0, 9,...|[{document, 0, 9,...|[{token, 0, 9, PR...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, PR...|
1. On August 24, ...|[{document, 0, 97...|[{document, 0, 97...|[{token, 0, 0, 1,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
2. On August 20, ...|[{document, 0, 36...|[{document, 0, 36...|[{token, 0, 0, 2,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
3. On August 31, ...|[{document, 0, 44...|[{document, 0, 44...|[{token, 0, 0, 3,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
HISTORY AND PHYSI...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 6, HI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 20, H...|
The patient is a ...|[{document, 0, 38...|[{document, 0, 96...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 17, 27, ...|
PAST MEDICAL HISTORY|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 3, PA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, P...|
 1. Pericardectomy.|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 0, 1,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 16, P...|
 2. Pericarditis.|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 0, 2,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 14, P...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

Let's see the labels of each token

In [0]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result, result.jsl_ner.result)).alias("cols"))\
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+-------------+---------------------------+
 token| ner_label|
+-------------+---------------------------+
 Sample| O|
 Type| O|
 /| O|
 Medical| O|
 Specialty| O|
 :| O|
 Hematology| B-Clinical_Dept|
 -| O|
 Oncology| O|
 Sample| O|
 Name| O|
 :| O|
 Discharge| B-Admission_Discharge|
 Summary| O|
 -| O|
 Mesothelioma| B-Oncological|
 -| O|
 1| O|
 Description| B-Section_Header|
 :| I-Section_Header|
 Mesothelioma| B-Oncological|
 ,| O|
 pleural|B-Disease_Syndrome_Disorder|
 effusion|I-Disease_Syndrome_Disorder|
 ,| O|
 atrial| B-Heart_Disease|
 fibrillation| I-Heart_Disease|
 ,| O|
 anemia|B-Disease_Syndrome_Disorder|
 ,| O|
 ascites|B-Disease_Syndrome_Disorder|
 ,| O|
 esophageal|B-Disease_Syndrome_Disorder|
 reflux|I-Disease_Syndrome_Disorder|
 ,| O|
 and| O|
 history| O|
 of| O|
 deep|B-Disease_Syndrome_Disorder|
 venous|I-Disease_Syndrome_Disorder|
 thrombosis|I-Disease_Syndrome_Disorder|
 .| O|
 (| O|
 Medical| O|
Transcription| O|
 Sample| O|
 Report| O|
 )| O|
 PRINCIPAL| B-Section_Header|
 DIAGNOSIS| I-Section_Header|
+-------------+---------------------------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label")\
         .groupBy('ner_label').count()\
         .orderBy('count', ascending=False)\
         .show(truncate=False)

+-----------------------------+-----+
ner_label |count|
+-----------------------------+-----+
O |579 |
B-Gender |30 |
B-Symptom |28 |
B-Disease_Syndrome_Disorder |21 |
B-Procedure |20 |
I-Section_Header |19 |
I-Disease_Syndrome_Disorder |16 |
I-Procedure |16 |
I-Symptom |14 |
B-Test |12 |
B-Direction |11 |
B-Section_Header |11 |
I-Date |10 |
B-Date |10 |
B-Drug_BrandName |9 |
I-Test |8 |
B-Internal_organ_or_component|7 |
B-Oncological |7 |
I-Strength |7 |
B-Strength |7 |
+-----------------------------+-----+
only showing top 20 rows

As you see above, we have labels start with **"O", "B"** and **"I"**. <br/>
O means that the corresponding token is not labeled as any named entity. "B" means that the corresponding token refers to the beginning of the ner chunk while "I" means the continuation of the ner chunk.  <br/>

So, in our example, we have `deep venous thrombosis` `Disease_Syndrome_Disorder` chunk. In this case, `deep` labeled as `B-Disease_Syndrome_Disorder` since it is the beginnig of the chunk, `venous` and `thrombosis` labeled as `I-Disease_Syndrome_Disorder` as they are the continuation of the `Disease_Syndrome_Disorder` chunk.

Now, let's check out the results as a **chunk-based** instead of token-based.

In [0]:
result.select(F.explode(F.arrays_zip(result.jsl_ner_chunk.result, result.jsl_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------------+-------------------------+
chunk |ner_label |
+----------------------+-------------------------+
Hematology |Clinical_Dept |
Discharge |Admission_Discharge |
Mesothelioma |Oncological |
Description: |Section_Header |
Mesothelioma |Oncological |
pleural effusion |Disease_Syndrome_Disorder|
atrial fibrillation |Heart_Disease |
anemia |Disease_Syndrome_Disorder|
ascites |Disease_Syndrome_Disorder|
esophageal reflux |Disease_Syndrome_Disorder|
deep venous thrombosis|Disease_Syndrome_Disorder|
PRINCIPAL DIAGNOSIS: |Section_Header |
Mesothelioma |Oncological |
SECONDARY DIAGNOSES: |Section_Header |
Pleural effusion |Disease_Syndrome_Disorder|
atrial fibrillation |Heart_Disease |
anemia |Disease_Syndrome_Disorder|
ascites |Disease_Syndrome_Disorder|
esophageal reflux |Disease_Syndrome_Disorder|
deep venous thrombosis|Disease_Syndrome_Disorder|
+----------------------+-------------------------+
only showing top 20 rows

As you see, our ner model successfully extracted useful named entities. <br/>

### with LightPipeline

We created a pipeline with `ner_jsl`. This time, let's use **LightPipeline** instead of `transform()`.

**LightPipeline** is a Spark NLP specific Pipeline class equivalent to Spark ML Pipeline. The difference is that its execution does not hold to Spark principles, instead, it computes everything locally (but in parallel) in order to achieve faster inference when dealing with small amounts of data. This means, we don't have to Spark Dataframe, but a string or an array of strings instead, to be annotated. To create Light Pipelines, you need to input an already trained (fit) Spark ML Pipeline.

 It’s `transform()` stage is converted into `annotate()` or `fullAnnotate()` instead. <br/>

Let's convert our existing ner pipeline to LightPipeline and see the results with an example text.

In [0]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.annotate(text)

light_result

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )
Out[12]: {'document': ['\nA 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )'],
 'jsl_ner_chunk': ['28-year-old',
 'female',
 'gestational diabetes mellitus',
 'eight years prior',
 'type two diabetes mellitus',
 'T2DM'],
 'jsl_ner': ['O',
 'B-Age',
 'B-Gender',
 'O',
 'O',
 'O',
 'O',
 'B-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'O',
 'B-RelativeDate',
 'I-RelativeDate',
 'I-RelativeDate',
 'O',
 'O',
 'O',
 'O',
 'B-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'O',
 'B-Diabetes',
 'O'],
 'token': ['A',
 '28-year-old',
 'female',
 'with',
 'a',
 'history',
 'of',
 'gestational',
 'diabetes',
 'mellitus',
 'diagnosed',
 'eight',
 'years',
 'prior',
 'to',
 'presentation',
 'and',
 'subsequent',
 'type',
 'two',
 'diabetes',
 'mellitus',
 '(',
 'T2DM',
 ')'],
 'embeddings': ['A',
 '28-year-old',
 'female',
 'with',
 'a',
 'history',
 'of',
 'gestational',
 'diabetes',
 'mellitus',
 'diagnosed',
 'eight',
 'years',
 'prior',
 'to',
 'presentation',
 'and',
 'subsequent',
 'type',
 'two',
 'diabetes',
 'mellitus',
 '(',
 'T2DM',
 ')'],
 'sentence': ['A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )']}

In [0]:
light_result.keys()

Out[13]: dict_keys(['document', 'jsl_ner_chunk', 'jsl_ner', 'token', 'embeddings', 'sentence'])

In [0]:
list(zip(light_result['token'], light_result['jsl_ner']))

Out[14]: [('A', 'O'),
 ('28-year-old', 'B-Age'),
 ('female', 'B-Gender'),
 ('with', 'O'),
 ('a', 'O'),
 ('history', 'O'),
 ('of', 'O'),
 ('gestational', 'B-Diabetes'),
 ('diabetes', 'I-Diabetes'),
 ('mellitus', 'I-Diabetes'),
 ('diagnosed', 'O'),
 ('eight', 'B-RelativeDate'),
 ('years', 'I-RelativeDate'),
 ('prior', 'I-RelativeDate'),
 ('to', 'O'),
 ('presentation', 'O'),
 ('and', 'O'),
 ('subsequent', 'O'),
 ('type', 'B-Diabetes'),
 ('two', 'I-Diabetes'),
 ('diabetes', 'I-Diabetes'),
 ('mellitus', 'I-Diabetes'),
 ('(', 'O'),
 ('T2DM', 'B-Diabetes'),
 (')', 'O')]

In [0]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl ,  creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 . 
'''

light_result_full = light_model.fullAnnotate(text)

In [0]:
light_result_full

Out[16]: [{'document': [Annotation(document, 0, 901, 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting . 
 She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
 Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 . 
 , {})],
 'jsl_ner_chunk': [Annotation(chunk, 3, 13, 28-year-old, {'entity': 'Age', 'sentence': '0', 'chunk': '0', 'confidence': '0.9991'}),
 Annotation(chunk, 15, 20, female, {'entity': 'Gender', 'sentence': '0', 'chunk': '1', 'confidence': '0.9999'}),
 Annotation(chunk, 40, 68, gestational diabetes mellitus, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '2', 'confidence': '0.9663334'}),
 Annotation(chunk, 80, 96, eight years prior, {'entity': 'RelativeDate', 'sentence': '0', 'chunk': '3', 'confidence': '0.6821'}),
 Annotation(chunk, 129, 154, type two diabetes mellitus, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '4', 'confidence': '0.94152504'}),
 Annotation(chunk, 158, 161, T2DM, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '5', 'confidence': '0.9233'}),
 Annotation(chunk, 187, 210, HTG-induced pancreatitis, {'entity': 'Disease_Syndrome_Disorder', 'sentence': '0', 'chunk': '6', 'confidence': '0.51625'}),
 Annotation(chunk, 212, 228, three years prior, {'entity': 'RelativeDate', 'sentence': '0', 'chunk': '7', 'confidence': '0.8452'}),
 Annotation(chunk, 271, 275, acute, {'entity': 'Modifier', 'sentence': '0', 'chunk': '8', 'confidence': '0.9484'}),
 Annotation(chunk, 277, 285, hepatitis, {'entity': 'Disease_Syndrome_Disorder', 'sentence': '0', 'chunk': '9', 'confidence': '0.5994'}),
 Annotation(chunk, 306, 313, one-week, {'entity': 'Duration', 'sentence': '0', 'chunk': '10', 'confidence': '0.9544'}),
 Annotation(chunk, 326, 333, polyuria, {'entity': 'Symptom', 'sentence': '0', 'chunk': '11', 'confidence': '0.986'}),
 Annotation(chunk, 337, 349, poor appetite, {'entity': 'Symptom', 'sentence': '0', 'chunk': '12', 'confidence': '0.9307'}),
 Annotation(chunk, 357, 364, vomiting, {'entity': 'Symptom', 'sentence': '0', 'chunk': '13', 'confidence': '0.9918'}),
 Annotation(chunk, 369, 371, She, {'entity': 'Gender', 'sentence': '1', 'chunk': '14', 'confidence': '1.0'}),
 Annotation(chunk, 380, 388, metformin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '15', 'confidence': '0.9898'}),
 Annotation(chunk, 392, 400, glipizide, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '16', 'confidence': '0.9993'}),
 Annotation(chunk, 408, 420, dapagliflozin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '17', 'confidence': '0.9971'}),
 Annotation(chunk, 426, 429, T2DM, {'entity': 'Diabetes', 'sentence': '1', 'chunk': '18', 'confidence': '0.8716'}),
 Annotation(chunk, 435, 446, atorvastatin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '19', 'confidence': '0.9968'}),
 Annotation(chunk, 452, 462, gemfibrozil, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '20', 'confidence': '0.9933'}),
 Annotation(chunk, 468, 470, HTG, {'entity': 'Hyperlipidemia', 'sentence': '1', 'chunk': '21', 'confidence': '0.938'}),
 Annotation(chunk, 474, 476, She, {'entity': 'Gender', 'sentence': '2', 'chunk': '22', 'confidence': '1.0'}),
 Annotation(chunk, 490, 502, dapagliflozin, {'entity': 'Drug_Ingredient', 'sentence': '2', 'chunk': '23', 'confidence': '0.9832'}),
 Annot

We used `fullAnnotate()` to see more detailed result

In [0]:
chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result_full[0]['jsl_ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

Out[17]:

,chunks,begin,end,sentence_id,entities
0,28-year-old,3,13,0,Age
1,female,15,20,0,Gender
2,gestational diabetes mellitus,40,68,0,Diabetes
3,eight years prior,80,96,0,RelativeDate
4,type two diabetes mellitus,129,154,0,Diabetes
5,T2DM,158,161,0,Diabetes
6,HTG-induced pancreatitis,187,210,0,Disease_Syndrome_Disorder
7,three years prior,212,228,0,RelativeDate
8,acute,271,275,0,Modifier
9,hepatitis,277,285,0,Disease_Syndrome_Disorder


### NER Visualizer

The **NerVisualizer** highlights the named entities that are identified by Spark NLP and also displays their labels as decorations on top of the analyzed text. The colors assigned to the predicted labels can be configured to fit the particular needs of the application.

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis= visualiser.display(light_result_full[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

# Change color of an entity label

#visualiser = NerVisualizer()
#visualiser.set_label_colors({'Age':'#008080', 'Gender':'#800080', 'Diabetes':'#808080'})
#ner_vis= visualiser.display(light_result_full[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)
#displayHTML(ner_vis)

# Set label filter

# visualiser.display(light_result[0], label_col='jsl_ner_chunk', document_col='document',
                   #labels=['Symptom','Gender'])

A 28-year-old Age female Gender with a history of gestational diabetes mellitus Diabetes diagnosed eight years prior RelativeDate to presentation and subsequent type two diabetes mellitus Diabetes ( T2DM Diabetes ), one prior episode of HTG-induced pancreatitis Disease_Syndrome_Disorder three years prior RelativeDate to presentation , and associated with an acute Modifier hepatitis Disease_Syndrome_Disorder , presented with a one-week Duration history of polyuria Symptom , poor appetite Symptom , and vomiting Symptom . She Gender was on metformin Drug_Ingredient , glipizide Drug_Ingredient , and dapagliflozin Drug_Ingredient for T2DM Diabetes and atorvastatin Drug_Ingredient and gemfibrozil Drug_Ingredient for HTG Hyperlipidemia . She Gender had been on dapagliflozin Drug_Ingredient for six months Duration at the time of presentation . Physical examination on presentation was significant for dry oral mucosa Symptom ; significantly , her Gender abdominal External_body_part_or_region examination was benign with no tenderness Symptom , guarding Symptom , or rigidity Symptom . Pertinent laboratory findings on admission Admission_Discharge were : serum glucose Test 111 mg/dl Test_Result , creatinine Test 0.4 mg/dL Test_Result , triglycerides Triglycerides 508 mg/dL Test_Result , total cholesterol 122 mg/dL Total_Cholesterol , and venous pH Test 7.27 Test_Result .

## Clinical NER Chunk Merger

Sparknlp has the ability to merge the chunks that come from different annotators(NER, ContextualParser or any other annotator producing chunks) through the `ChunkMergeApproach()` annotator.  The decision on which chunk to select is made according to the chunk indices in the source document. (chunks with longer lengths and highest information will be kept from each source)


Let's create a sample pipeline consisting `ner_jsl`, `ner_deid_generic` and `ner_posology` models, and merge the outputs of these models.

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model to detect PHI entities
deid_ner = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")\
    .setWhiteList(["AGE", "DATE", "LOCATION"]) # only AGE, DATE and LOCATION entities

# NER model to detect clinical entities (PROBLEM, TEST, TREATMENT)
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")\
    .setBlackList(["TREATMENT"]) # blocks TREATMENT entities 

# NER model to detect medication entities (Durg, Dosage, etc.))
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")


# merge ner_chunks by prioritizing the overlapping indices (chunks with longer lengths and highest information will be kept from each ner model)
chunk_merger = ChunkMergeApproach()\
    .setInputCols('deid_ner_chunk', "clinical_ner_chunk", "posology_ner_chunk")\
    .setOutputCol('merged_ner_chunk')


nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    deid_ner,
    deid_ner_converter,
    clinical_ner,
    clinical_ner_converter,
    posology_ner,
    posology_ner_converter,
    chunk_merger])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][ / ][ — ][OK!]
ner_clinical download started this may take some time.
[ | ][ / ][OK!]
ner_posology download started this may take some time.
[ | ][ / ][OK!]

In [0]:
text = """HISTORY AND PHYSICAL: The patient is from Vietnam, 41-year-old female with a nonproductive cough that started on Sep 2006. 
          She has had right-sided chest pain radiating to her back with fever starting on November 5, 2006. 
          She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
          MEDICATIONS: Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and INR was 2.3. Amiodarone 100 mg p.o. daily.
       """

spark_df = spark.createDataFrame([[text]]).toDF("text")

merged_data = model.transform(spark_df)

%time merged_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| clinical_ner| clinical_ner_chunk| posology_ner| posology_ner_chunk| merged_ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
HISTORY AND PHYSI...|[{document, 0, 58...|[{document, 0, 12...|[{token, 0, 6, HI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 42, 48, ...|[{named_entity, 0...|[{chunk, 75, 95, ...|[{named_entity, 0...|[{chunk, 466, 473...|[{chunk, 42, 48, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

CPU times: user 57.5 ms, sys: 10.2 ms, total: 67.7 ms
Wall time: 10.6 s

In [0]:
lp_model = LightPipeline(model)

In [0]:
%time lp_result = lp_model.annotate(text)

CPU times: user 57 ms, sys: 23.9 ms, total: 80.9 ms
Wall time: 9.8 s

Let's see the results

In [0]:
result_df = merged_data.select(F.explode('merged_ner_chunk').alias("cols")) \
                       .select(F.expr("cols.begin").alias("begin"),
                               F.expr("cols.end").alias("end"),
                               F.expr("cols.result").alias("chunk"),
                               F.expr("cols.metadata.entity").alias("entity"))

result_df.show(50, truncate=100)

+-----+---+----------------------------+---------+
begin|end| chunk| entity|
+-----+---+----------------------------+---------+
 42| 48| Vietnam| LOCATION|
 51| 61| 41-year-old| AGE|
 75| 95| a nonproductive cough| PROBLEM|
 113|120| Sep 2006| DATE|
 146|167| right-sided chest pain| PROBLEM|
 196|200| fever| PROBLEM|
 214|229| November 5, 2006| DATE|
 264|275| pericarditis| PROBLEM|
 299|306| May 2006| DATE|
 322|326| cough| PROBLEM|
 333|354| right-sided chest pain| PROBLEM|
 392|402| Chest x-ray| TEST|
 413|440|right-sided pleural effusion| PROBLEM|
 466|473| Coumadin| DRUG|
 475|478| 1 mg| STRENGTH|
 480|484| daily|FREQUENCY|
 492|494| INR| TEST|
 503|509| Tuesday| DATE|
 512|526| August 14, 2007| DATE|
 533|535| INR| TEST|
 546|555| Amiodarone| DRUG|
 557|562| 100 mg| STRENGTH|
 564|566| p.o| ROUTE|
 569|573| daily|FREQUENCY|
+-----+---+----------------------------+---------+

## Clinical NER Training

In this section, we will train our own NER model. <br/>
Firstly, let's download NCBI Disease dataset for training.

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[3]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')
conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[5]: 3266

Checking the number of labels in the dataset

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
'''
As you can see, there are too many `O` labels in the dataset. 
To make it more balanced, we can drop the sentences have only O labels.
(`c>1` means we drop all the sentences that have no valuable labels other than `O`)
'''

'''
conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)
'''

Out[28]: '\nconll_data = conll_data.withColumn(\'unique\', F.array_distinct("label.result")) .withColumn(\'c\', F.size(\'unique\')) .filter(F.col(\'c\')>1)\n\nconll_data.select(F.explode(F.arrays_zip(\'token.result\',\'label.result\')).alias("cols")) .select(F.expr("cols[\'0\']").alias("token"),\n F.expr("cols[\'1\']").alias("ground_truth")) .groupBy(\'ground_truth\') .count() .orderBy(\'count\', ascending=False) .show(100,truncate=False)\n'

Now, let's download the embeddings. We will transform the embeddings with our test data in order to use the `setTestDataset()` parameter of `MedicalNerApproach()` during the training. This parameter is used to calculate statistics on test dataset during training.

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

Let's read the test dataset and get its embeddings, then save it as a parquet format to use it in the training.

In [0]:
test_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltest.txt')

'dbfs:/NER_NCBIconlltest.parquet'test_data = clinical_embeddings.transform(test_data)
test_data.write.parquet('dbfs:/NER_NCBIconlltest.parquet')

### NERDL Graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the `MedicalNerApproach` annotator.

In [0]:
#!pip install -q tensorflow==2.7.0
#!pip install -q tensorflow-addons

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

Firstly, we will create graph and log folder.

In [0]:
%fs mkdirs file:/dbfs/ner/ner_logs

res0: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/ner/medical_ner_graphs

res1: Boolean = true

In [0]:
graph_folder_path = "/dbfs/ner/medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

We created ner log and graph files. 
Now, we will create graph and fit the model.

In [0]:
# TensorFlow graph file (`.pb` extension) can be produced for NER training externally. 
# If this method is used, graph folder should be added to MedicalNerApproach training 
# model as `.setGraphFolder(graph_folder_path)` .

'''
tf_graph.print_model_params("ner_dl")
tf_graph.build("ner_dl", 
               build_params={"embeddings_dim": 200, 
                             "nchars": 85, 
                             "ntags": 12, 
                             "is_medical": 1},
               model_location="/dbfs/ner/medical_ner_graphs", 
               model_filename="auto")
'''

Out[15]: '\ntf_graph.print_model_params("ner_dl")\ntf_graph.build("ner_dl", \n build_params={"embeddings_dim": 200, \n "nchars": 85, \n "ntags": 12, \n "is_medical": 1},\n model_location="/dbfs/ner/medical_ner_graphs", \n model_filename="auto")\n'

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

We will use `setEarlyStoppingCriterion` and `setEarlyStoppingPatience` parameters to stop training if it doesn't improve any more. Also, these parameters will provide us to save time and resources.

- `setEarlyStoppingCriterion(float)`: This parameter is used to set the minimal improvement of the test metric to terminate training. The metric monitored is the same which is used to save the best best model (macro F1 if using test or validation set, loss otherwise). If it is F1, then training is stopped if `F1_current - F1_previous < earlyStoppingCriterion`. If monitoring loss, then training is stopped if `loss_previous - loss_current < earlyStoppingCriterion`. If earlyStoppingCriterion is `0`, then no early stopping is applied. Default is `0`.

- `setEarlyStoppingPatience(int)`: This is the number of epoch without improvement which will be tolerated. Default is `0`, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch (taking earlyStoppingCriterion into account).

Now we can train our model through `MedicalNerApproach()`

In [0]:
nerTagger = MedicalNerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(3)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('dbfs:/ner/ner_logs')\
    .setGraphFolder('dbfs:/ner/medical_ner_graphs')\
    .setTestDataset("/NER_NCBIconlltest.parquet")\
    .setUseBestModel(True)\
    .setEarlyStoppingCriterion(0.04)\
    .setEarlyStoppingPatience(3)\
    .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

In [0]:
%%time
ner_model = ner_pipeline.fit(conll_data)

# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook in public folder to create a graph
# licensed users can also use 17.Graph_builder_for_DL_models.ipynb to create tf graphs easily.

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: /dbfs/ner/medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 20}
/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:111: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
 warnings.warn('`layer.add_variable` is deprecated and '
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_3_200_20_85.pb
CPU times: user 14 s, sys: 818 ms, total: 14.8 s
Wall time: 2min 43s

`getTrainingClassDistribution()` parameter returns the distribution of labels used when training the NER model.

In [0]:
ner_model.stages[-1].getTrainingClassDistribution()

Out[21]: {'O': 61243, 'B-Disease': 2543, 'I-Disease': 2899}

Let's check the results saved in the log file.

In [0]:
import os 
log_file= os.listdir("/dbfs/ner/ner_logs")[0]

with open (f"/dbfs/ner/ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: medical-ner-dl/blstm_38_768_128_200.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 32 - labels: 22 - chars: 99 - training examples: 1327


Epoch 1/2 started, lr: 0.003, dataset size: 1327


Epoch 1/2 - 20.82s - loss: 1199.3247 - avg training loss: 27.25738 - batches: 44
Quality on validation dataset (20.0%), validation examples = 265
time to finish evaluation: 4.99s
Total validation loss: 104.5392	Avg validation loss: 8.7116
label tp fp fn prec rec f1
I-AMOUNT 232 4 9 0.9830508 0.9626556 0.9727463
B-AMOUNT 297 28 0 0.91384614 1.0 0.9549839
B-DATE 238 28 124 0.8947368 0.65745854 0.7579618
I-DATE 317 61 22 0.83862436 0.93510324 0.8842399
I-EXPENSE 2 5 61 0.2857143 0.031746034 0.05714286
B-PROFIT_INCREASE 3 2 33 0.6 0.083333336 0.14634147
B-EXPENSE 0 2 33 0.0 0.0 0.0
I-PROFIT_DECLINE 0 0 9 0.0 0.0 0.0
I-PROFIT 0 0 60 0.0 0.0 0.0
B-CURRENCY 295 2 2 0.993266 0.993266 0.993266
I-PROFIT_INCREASE 8 9 42 0.47058824 0.16 0.23880596
B-PROFIT 0 0 28 0.0 0.0 0.0
B-PERCENTAGE 73 4 5 0.9480519 0.9358974 0.9419355
I-FISCAL_YEAR 366 54 12 0.87142855 0.96825397 0.9172932
B-PROFIT_DECLINE 0 0 7 0.0 0.0 0.0
B-EXPENSE_INCREASE 14 26 47 0.35 0.22950819 0.27722773
B-EXPENSE_DECREASE 0 0 41 0.0 0.0 0.0
B-FISCAL_YEAR 122 10 6 0.92424244 0.953125 0.9384615
I-EXPENSE_DECREASE 2 4 80 0.33333334 0.024390243 0.045454543
I-EXPENSE_INCREASE 34 56 90 0.37777779 0.27419356 0.31775704
tp: 2003 fp: 295 fn: 711 labels: 20
Macro-average prec: 0.48923302, rec: 0.41044655, f1: 0.44639006
Micro-average prec: 0.8716275, rec: 0.73802507, f1: 0.7992818


Epoch 2/2 started, lr: 0.0029850747, dataset size: 1327


Epoch 2/2 - 18.62s - loss: 374.4107 - avg training loss: 8.509335 - batches: 44
Quality on validation dataset (20.0%), validation examples = 265
time to finish evaluation: 4.12s
Total validation loss: 86.1391	Avg validation loss: 7.1783
label tp fp fn prec rec f1
I-AMOUNT 232 1 9 0.99570817 0.9626556 0.97890294
B-AMOUNT 296 9 1 0.9704918 0.996633 0.9833887
B-DATE 292 17 70 0.94498384 0.80662984 0.8703428
I-DATE 315 17 24 0.9487952 0.9292035 0.93889713
I-EXPENSE 5 6 58 0.45454547 0.07936508 0.13513513
B-PROFIT_INCREASE 14 14 22 0.5 0.3888889 0.4375
B-EXPENSE 1 4 32 0.2 0.030303031 0.05263158
I-PROFIT_DECLINE 0 0 9 0.0 0.0 0.0
I-PROFIT 1 0 59 1.0 0.016666668 0.032786887
B-CURRENCY 295 1 2 0.9966216 0.993266 0.994941
I-PROFIT_INCREASE 31 36 19 0.46268657 0.62 0.52991456
B-PROFIT 0 0 28 0.0 0.0 0.0
B-PERCENTAGE 78 0 0 1.0 1.0 1.0
I-FISCAL_YEAR 371 13 7 0.9661458 0.9814815 0.9737533
B-PROFIT_DECLINE 0 0 7 0.0 0.0 0.0
B-EXPENSE_INCREASE 27 34 34 0.44262296 0.44262296 0.44262296
B-EXPENSE_DECREASE 9 10 32 0.47368422 0.2195122 0.29999998
B-FISCAL_YEAR 122 5 6 0.96062994 0.953125 0.95686275
I-EXPENSE_DECREASE 37 41 45 0.47435898 0.4512195 0.4625
I-EXPENSE_INCREASE 39 40 85 0.49367088 0.31451613 0.38423643
tp: 2165 fp: 248 fn: 549 labels: 20
Macro-average prec: 0.6142472, rec: 0.5093044, f1: 0.5568748
Micro-average prec: 0.89722335, rec: 0.79771554, f1: 0.84454846

As you see above, our earlyStopping feature worked, trainining was terminated before 3th epoch.

### Evaluating the Model

Firstly, we will get the predictions from test data

In [0]:
pred_df = ner_model.stages[-1].transform(test_data)

In [0]:
pred_df.columns

Out[28]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

Now, let's see the metrics by using **eval** package of sparknlp_jsl

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|317.0|179.0|387.0|704.0| 0.6391|0.4503|0.5283|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
 macro|
+------------------+
0.5283333333333333|
+------------------+

None
+------------------+
 micro|
+------------------+
0.5283333333333333|
+------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+------+---------+------+---+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+-----+----+-----+------+---------+------+---+
Disease|847.0|76.0|650.0|1497.0| 0.9177|0.5658|0.7|
+-------+-----+----+-----+------+---------+------+---+

+-----+
macro|
+-----+
 0.7|
+-----+

None
+-----+
micro|
+-----+
 0.7|
+-----+

None

`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate( preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

### Saving the model and using it in different pipeline

We can save our custom ner model and use it in other pieplines with the `load()` method of `MedicalNerModel`

In [0]:
#saving the model
ner_model.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/custom_NER')

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

#custom ner model
loaded_ner_model = MedicalNerModel.load("dbfs:/databricks/driver/models/custom_NER")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

converter = NerConverterInternal()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)


We just created a pipeline with our ner model, let's apply it over a sample data and see the results with LightPipeline quickly

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "The patient just received a lung cancer diagnosis"

result = light_model.fullAnnotate(text)[0]

[(i.result, i.metadata['entity']) for i in result['ner_span']]

Out[34]: [('lung cancer', 'Disease')]

As you see, we succesfully trained an NER model with our own data, saved it and got predictions!

**FINE-TUNING NER MODELS:**

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once. In Spark NLP we have two different NER profiling pipelines;

- `ner_profiling_clinical` : Returns results for clinical NER models trained with `embeddings_clinical`.
- `ner_profiling_biobert` : Returns results for clinical NER models trained with `biobert_pubmed_base_cased`.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).

<center><b>NER Profiling Clinical Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical | jsl_ner_wip_greedy_clinical | jsl_ner_wip_modifier_clinical | jsl_rd_ner_wip_greedy_clinical |
| ner_abbreviation_clinical | ner_ade_binary | ner_ade_clinical | ner_anatomy |
| ner_anatomy_coarse | ner_bacterial_species | ner_biomarker | ner_biomedical_bc2gm |
| ner_bionlp | ner_cancer_genetics | ner_cellular | ner_chemd_clinical |
| ner_chemicals | ner_chemprot_clinical | ner_chexpert | ner_clinical |
| ner_clinical_large | ner_clinical_trials_abstracts | ner_covid_trials | ner_deid_augmented |
| ner_deid_enriched | ner_deid_generic_augmented | ner_deid_large | ner_deid_sd |
| ner_deid_sd_large | ner_deid_subentity_augmented | ner_deid_subentity_augmented_i2b2 | ner_deid_synthetic |
| ner_deidentify_dl | ner_diseases | ner_diseases_large | ner_drugprot_clinical |
| ner_drugs | ner_drugs_greedy | ner_drugs_large | ner_events_admission_clinical |
| ner_events_clinical | ner_genetic_variants | ner_human_phenotype_gene_clinical | ner_human_phenotype_go_clinical |
| ner_jsl | ner_jsl_enriched | ner_jsl_greedy | ner_jsl_slim |
| ner_living_species | ner_measurements_clinical | ner_medmentions_coarse | ner_nature_nero_clinical |
| ner_nihss | ner_pathogen | ner_posology | ner_posology_experimental |
| ner_posology_greedy | ner_posology_large | ner_posology_small | ner_radiology |
| ner_radiology_wip_clinical | ner_risk_factors | ner_supplement_clinical | nerdl_tumour_demo |

<b>NER Profiling BioBert Model List</b>

| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |


</center>

We can easily apply `ner_profiling_clinical` as follows

In [0]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.5 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
#sample text for ner profiling
text = '''28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

We will `fullAnnotate()` with our sample text and see the results

In [0]:
clinical_result = clinical_profiling_pipeline.fullAnnotate(text)[0]
clinical_result.keys()

Out[74]: dict_keys(['ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_radiology_wip_clinical', 'ner_deidentify_dl', 'ner_jsl_slim', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_deid_synthetic', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_cancer_genetics_chunks', 'ner_biomedical_bc2gm_chunks', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_large_chunks', 'ner_drugs_large', 'ner_living_species_chunks', 'ner_clinical_large', 'ner_chemicals', 'ner_radiology_chunks', 'ner_bacterial_species', 'ner_deid_augmented_chunks', 'ner_bionlp', 'ner_drugprot_clinical', 'ner_anatomy_chunks', 'ner_deid_subentity_augmented_i2b2_chunks', 'ner_deid_large', 'jsl_ner_wip_modifier_clinical', 'ner_chemprot_clinical_chunks', 'nerdl_tumour_demo', 'ner_posology_experimental_chunks', 'ner_cancer_genetics', 'ner_biomarker', 'ner_drugprot_clinical_chunks', 'ner_posology_small', 'ner_drugs_chunks', 'ner_deid_sd_chunks', 'ner_chemd_clinical', 'ner_human_phenotype_gene_clinical', 'ner_ade_clinical', 'ner_human_phenotype_go_clinical', 'ner_risk_factors', 'ner_clinical', 'ner_posology_large_chunks', 'ner_cellular', 'ner_nature_nero_clinical_chunks', 'ner_deid_large_chunks', 'ner_clinical_trials_abstracts_chunks', 'ner_deid_generic_augmented_chunks', 'ner_posology_chunks', 'ner_deidentify_dl_chunks', 'ner_supplement_clinical_chunks', 'ner_deid_enriched_chunks', 'ner_chemd_clinical_chunks', 'ner_deid_sd_large', 'jsl_ner_wip_greedy_clinical', 'ner_bacterial_species_chunks', 'ner_living_species', 'ner_abbreviation_clinical', 'ner_diseases', 'jsl_rd_ner_wip_greedy_clinical', 'ner_drugs', 'ner_drugs_large_chunks', 'ner_biomarker_chunks', 'ner_clinical_trials_abstracts', 'ner_clinical_large_chunks', 'ner_anatomy_coarse', 'ner_nature_nero_clinical', 'token', 'ner_medmentions_coarse', 'ner_deid_sd', 'jsl_rd_ner_wip_greedy_clinical_chunks', 'ner_deid_generic_augmented', 'ner_pathogen_chunks', 'ner_medmentions_coarse_chunks', 'ner_anatomy', 'ner_deid_enriched', 'ner_deid_subentity_augmented', 'ner_radiology_wip_clinical_chunks', 'ner_clinical_chunks', 'ner_measurements_clinical', 'ner_ade_binary', 'ner_nihss', 'ner_chexpert_chunks', 'ner_chemicals_chunks', 'ner_pathogen', 'ner_jsl_enriched', 'ner_deid_synthetic_chunks', 'ner_events_clinical_chunks', 'ner_posology_experimental', 'jsl_ner_wip_clinical', 'ner_biomedical_bc2gm', 'ner_posology_small_chunks', 'ner_nihss_chunks', 'ner_anatomy_coarse_chunks', 'ner_human_phenotype_go_clinical_chunks', 'ner_jsl', 'ner_events_clinical', 'ner_supplement_clinical', 'ner_jsl_slim_chunks', 'ner_jsl_chunks', 'ner_jsl_greedy_chunks', 'ner_genetic_variants', 'ner_radiology', 'ner_posology', 'sentence', 'ner_covid_trials', 'ner_ade_binary_chunks', 'ner_events_admission_clinical_chunks'])

Let's write a function to get the results properly and more clear

In [0]:
def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:
    
    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)
    
  return df

In [0]:
get_token_results(clinical_result)

Out[76]:

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_radiology_wip_clinical,ner_deidentify_dl,ner_jsl_slim,ner_deid_synthetic,ner_drugs_greedy,ner_events_admission_clinical,ner_posology_greedy,ner_jsl_greedy,ner_diseases_large,ner_chemprot_clinical,ner_posology_large,ner_chexpert,ner_drugs_large,ner_clinical_large,ner_chemicals,ner_bacterial_species,ner_bionlp,ner_drugprot_clinical,ner_deid_large,jsl_ner_wip_modifier_clinical,nerdl_tumour_demo,ner_cancer_genetics,ner_biomarker,ner_posology_small,ner_chemd_clinical,ner_human_phenotype_gene_clinical,ner_ade_clinical,ner_human_phenotype_go_clinical,ner_risk_factors,ner_clinical,ner_cellular,ner_deid_sd_large,jsl_ner_wip_greedy_clinical,ner_living_species,ner_abbreviation_clinical,ner_diseases,jsl_rd_ner_wip_greedy_clinical,ner_drugs,ner_clinical_trials_abstracts,ner_anatomy_coarse,ner_nature_nero_clinical,ner_medmentions_coarse,ner_deid_sd,ner_deid_generic_augmented,ner_anatomy,ner_deid_enriched,ner_deid_subentity_augmented,ner_measurements_clinical,ner_ade_binary,ner_nihss,ner_pathogen,ner_jsl_enriched,ner_posology_experimental,jsl_ner_wip_clinical,ner_biomedical_bc2gm,ner_jsl,ner_events_clinical,ner_supplement_clinical,ner_genetic_variants,ner_radiology,ner_posology,ner_covid_trials
0,0,0,10,28-year-old,O,O,O,O,B-Age,O,O,O,O,B-Age,O,O,O,O,O,O,O,O,O,O,O,B-Age,O,O,B-Age,O,O,O,O,O,O,O,O,B-AGE,B-Age,O,O,O,B-Age,O,O,O,B-Person,O,O,B-AGE,O,O,B-AGE,O,O,O,O,B-Age,O,B-Age,O,B-Age,O,O,O,O,O,B-Gender
1,0,12,17,female,O,O,O,O,B-Demographics,O,O,O,O,B-Gender,O,O,O,O,O,O,O,O,O,O,O,B-Gender,O,O,B-Gender,O,O,O,O,O,O,O,O,O,B-Gender,B-HUMAN,O,O,B-Gender,O,O,O,I-Person,B-Organism_Attribute,O,O,O,O,O,O,O,O,O,B-Gender,O,B-Gender,O,B-Gender,O,O,O,O,O,B-Gender
2,0,19,22,with,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
3,0,24,24,a,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,26,32,history,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-Medicalfinding,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,407,414,appetite,O,O,I-Symptom,O,I-Symptom,O,O,I-PROBLEM,O,I-Symptom,O,O,O,O,O,I-PROBLEM,O,O,O,O,O,I-Symptom,O,O,O,O,O,O,O,O,O,I-PROBLEM,O,O,I-Symptom,O,O,I-Disease,I-Symptom,O,O,O,I-Medicalfinding,I-Sign_or_Symptom,O,O,O,O,O,O,I-ADE,O,I-MedicalCondition,I-Symptom,O,I-Symptom,O,I-Symptom,I-PROBLEM,O,O,I-Symptom,O,O
68,0,416,416,",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
69,0,418,420,and,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
70,0,422,429,vomiting,O,O,B-Symptom,O,B-Symptom,O,O,B-PROBLEM,O,B-Symptom,B-Disease,O,O,O,O,B-PROBLEM,O,O,O,O,O,B-Symptom,O,O,O,O,O,O,B-ADE,B-HP,O,B-PROBLEM,O,O,B-Symptom,O,O,B-Disease,B-Symptom,O,O,O,B-Medicalfinding,B-Sign_or_Symptom,O,O,O,O,O,O,B-ADE,O,B-MedicalCondition,B-Symptom,O,B-Symptom,O,B-Symptom,B-PROBLEM,B-CONDITION,O,B-Symptom,O,O


## NER Model Finder Pretrained Pipeline
`ner_model_finder`  pretrained pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

In [0]:
from sparknlp.pretrained import PretrainedPipeline
finder_pipeline = PretrainedPipeline("ner_model_finder", "en", "clinical/models")

ner_model_finder download started this may take some time.
Approx size to download 148.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
result = finder_pipeline.fullAnnotate("carcinoma")[0]
result.keys()

Out[78]: dict_keys(['model_names'])

From the metadata in the `model_names` column, we'll get to the top models to the given 'oncology' entity and oncology related categories.

In [0]:
df= pd.DataFrame(zip(result["model_names"][0].metadata["all_k_resolutions"].split(":::"), 
                     result["model_names"][0].metadata["all_k_results"].split(":::")), 
                 columns=["category", "top_models"])

df.head(10)

Out[79]:

,category,top_models
0,cancer,"['ner_jsl', 'ner_bionlp', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ne..."
1,disease,"['ner_covid_trials', 'ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'ner_medmentions_coarse', 'jsl..."
2,metastasis,"['ner_jsl_enriched', 'ner_biomarker']"
3,heart disease,"['ner_covid_trials', 'ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical..."
4,tumour,"['ner_jsl_enriched', 'nerdl_tumour_demo']"
5,pathogen,['ner_pathogen']
6,biomedical unit,['ner_clinical_trials_abstracts']
7,adverse drug event,"['ner_ade_healthcare', 'ner_ade_clinical']"
8,genetic organism,['ner_human_phenotype_go_clinical']
9,limbataxia,['ner_nihss']


**NER Model Playground:**

You can check [Spark NLP Clinical NER Model Playground Notebook](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/Spark_NLP_Clinical_NER_Playground_Streamlit_app.ipynb#scrollTo=yFLcEPoAPB48) for a streamlit demo with all NER models in Spark NLP.

# 2.ContextualParser (Rule Based NER)

Spark NLP's `ContextualParser` is a licensed annotator that allows users to extract entities from a document based on pattern matching. It provides more functionality than its open-source counterpart `EntityRuler` by allowing users to customize specific characteristics for pattern matching. You're able to find entities using regex rules for full and partial matches, a dictionary with normalizing options and context parameters to take into account things such as token distances.

There are 3 components necessary to understand when using the `ContextualParser` annotator:

- **`ContextualParser` annotator's parameters**
Here are all the parameters available to use with the ContextualParserApproach
```
contextualParser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity") \
    .setCaseSensitive(True) \
    .setJsonPath("context_config.json") \
    .setPrefixAndSuffixMatch(True) \
    .setDictionary("dictionary.tsv", options=.    
                    {"orientation":"vertical"})
```

- **JSON configuration file** ContextualParser extracts entity from a document based on user defined rules. Rule matching is based on a RegexMatcher defined in a JSON file. It is set through the parameter `setJsonPath()`. In this JSON file, regex is defined that you want to match along with the information that will output on metadata field. 

- **Dictionary** You can specify a path to a dictionary in tsv or csv format using the `setDictionary()` parameter. Using a dictionary is a useful when you have a list of exact words that you want the parser to pick up when processing some text.

For more details about the components(parameters, JSON and dictionary), please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.2.Contextual_Parser_Rule_Based_NER.ipynb)

Now, we will create a dictionary and json file to build a sample `ContextualParser` pipeline to detect "gender" and "age" from given text. And merge their results with NER model which we will get only `DATE` and `LOCATION` labels.

In [0]:
# Here's some sample text
sample_text = """A 28 year old female with a history of gestational diabetes mellitus diagnosed 8 years ago. 
                 3 years ago, he reported an episode of HTG-induced pancreatitis . 
                 5 months old boy with repeated concussions."""

In [0]:
# Create a dictionary to detect gender
gender = '''male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,they,neutral,it'''

with open('/dbfs/gender.csv', 'w') as f:
    f.write(gender)

# Check what dictionary looks like
!cat /dbfs/gender.csv

male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,they,neutral,it

In [0]:
# Create JSON file for gender
gender = {
  "entity": "Gender",
  "ruleScope": "sentence", 
  "completeMatchRegex": "true",
  "matchScope":"token"
}

import json
with open('/dbfs/gender.json', 'w') as f:
    json.dump(gender, f)

In [0]:
# Create JSON file for age
age = {
  "entity": "Age",
  "ruleScope": "sentence",
  "matchScope":"token",
  "regex":"\\d{1,3}",
  "prefix":["age of", "age"],
  "suffix": ["-years-old", "years-old", "-year-old",
             "-months-old", "-month-old", "-months-old",
             "-day-old", "-days-old", "month old",
             "days old", "year old", "years old", 
             "years", "year", "months", "old"],
  "contextLength": 25,
  "contextException": ["ago"],
  "exceptionDistance": 12
}

with open('/dbfs/age.json', 'w') as f:
    json.dump(age, f)

In [0]:
# Build pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

gender_contextual_parser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("gender_chunk") \
    .setJsonPath("/dbfs/gender.json") \
    .setCaseSensitive(False) \
    .setDictionary('file:/dbfs/gender.csv', read_as=ReadAs.TEXT, options={"delimiter":","}) \
    .setPrefixAndSuffixMatch(False)      

age_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("chunk_age") \
        .setJsonPath("/dbfs/age.json") \
        .setCaseSensitive(False) \
        .setPrefixAndSuffixMatch(False)\
        .setShortestContextMatch(True)\
        .setOptionalContextRules(False)   

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

deid_ner = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")\
    .setWhiteList(["DATE", "LOCATION"])

# merge CP and NER model chunks
chunk_merger = ChunkMergeApproach() \
    .setInputCols(["gender_chunk", "chunk_age", "deid_ner_chunk"]) \
    .setOutputCol("ner_chunk")

parserPipeline = Pipeline(stages=[
        document_assembler, 
        sentence_detector,
        tokenizer,
        gender_contextual_parser,
        age_contextual_parser,
        word_embeddings,
        deid_ner,
        deid_ner_converter,
        chunk_merger
        ])

# Create a lightpipeline model
parserModel = parserPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = LightPipeline(parserModel)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][OK!]

**In the metadata, `ner_source` is for showing the source of the entities.**

In [0]:
# Annotate the sample text
annotations = light_model.fullAnnotate(sample_text)[0]

# Check outputs
annotations.get('ner_chunk')

Out[112]: [Annotation(chunk, 2, 3, 28, {'tokenIndex': '1', 'entity': 'Age', 'field': 'Age', 'chunk': '0', 'normalized': '', 'sentence': '0', 'confidenceValue': '0.74'}),
 Annotation(chunk, 14, 19, female, {'tokenIndex': '4', 'entity': 'Gender', 'field': 'Gender', 'chunk': '1', 'normalized': 'female', 'sentence': '0', 'confidenceValue': '0.50'}),
 Annotation(chunk, 123, 124, he, {'tokenIndex': '4', 'entity': 'Gender', 'field': 'Gender', 'chunk': '2', 'normalized': 'male', 'sentence': '1', 'confidenceValue': '0.50'}),
 Annotation(chunk, 194, 194, 5, {'tokenIndex': '0', 'entity': 'Age', 'field': 'Age', 'chunk': '3', 'normalized': '', 'sentence': '2', 'confidenceValue': '0.74'}),
 Annotation(chunk, 207, 209, boy, {'tokenIndex': '3', 'entity': 'Gender', 'field': 'Gender', 'chunk': '4', 'normalized': 'male', 'sentence': '2', 'confidenceValue': '0.50'})]

In [0]:
visualiser = NerVisualizer()

ner_vis = visualiser.display(annotations, label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

A 28 Age year old female Gender with a history of gestational diabetes mellitus diagnosed 8 years ago. 3 years ago, he Gender reported an episode of HTG-induced pancreatitis . 5 Age months old boy Gender with repeated concussions.

# 3.Clinical Assertion Status

The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|

### Pretrained Assertion Status Models

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | [assertion_dl](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_en.html)            |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  2 | [assertion_dl_biobert](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_biobert_en.html)    |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  3 | [assertion_dl_healthcare](https://nlp.johnsnowlabs.com/2020/09/23/assertion_dl_healthcare_en.html) |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  4 | [assertion_dl_large](https://nlp.johnsnowlabs.com/2020/05/21/assertion_dl_large_en.html)      |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  5 | [assertion_dl_radiology](https://nlp.johnsnowlabs.com/2021/03/18/assertion_dl_radiology_en.html)   |Confirmed, Suspected, Negative|
|  6 | [assertion_jsl](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_en.html)           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, Hypotetical|
|  7 | [assertion_jsl_large](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_large_en.html)     |present, absent, possible, planned, someoneelse, past, hypothetical|
|  8 |  [assertion_ml](https://nlp.johnsnowlabs.com/2020/01/30/assertion_ml_en.html) |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|
|  9 | [assertion_dl_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/17/assertion_dl_scope_L10R10_en_3_0.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 10 | [assertion_dl_biobert_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/24/assertion_dl_biobert_scope_L10R10_en_2_4.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 11 | [assertion_jsl_augmented](https://nlp.johnsnowlabs.com/2022/09/15/assertion_jsl_augmented_en.html)| Present, Absent, Possible, Planned, Past, Family, Hypotetical, SomeoneElse|

Let's start to create a Pipeline in order to detect assertion status with `assertion_dl` pretrained model. <br/>
This model detects following assertion status; 
`absent`, `present`, `conditional`, `associated_with_someone_else`, `hypothetical`, `possible`.

And we will use **`AssertionFilterer`** for filtering out the NERs by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like `present` or `conditional`; and do not want `absent` conditions get out of your pipeline.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")


# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "clinical_ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    # .setScopeWindow([9,15]) # default [9,15]
    
# filter out the named entities by the list of acceptable assertion statuses. present here.
assertion_filterer = AssertionFilterer()\
    .setInputCols("sentence","clinical_ner_chunk","assertion")\
    .setOutputCol("assertion_filtered")\
    .setWhiteList(["present"])

assertionPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    clinical_ner_converter,
    clinical_assertion,
    assertion_filterer
    ])


assertion_model = assertionPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
assertion_dl download started this may take some time.
[ | ][OK!]

Creating example dataframe with sample text and check the results

In [0]:
text = """
Patient with severe fever and sore throat also suffers from pain.
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also become short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

data = spark.createDataFrame([[text]]).toDF("text")

# fit/transform and show
results = assertion_model.transform(data)
results.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| clinical_ner| clinical_ner_chunk| assertion| assertion_filtered|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Patient with sev...|[{document, 0, 29...|[{document, 1, 65...|[{token, 1, 7, Pa...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 14, 25, ...|[{assertion, 14, ...|[{chunk, 14, 25, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
results.select(F.explode(F.arrays_zip(results.clinical_ner_chunk.result, results.clinical_ner_chunk.metadata, results.assertion.result)).alias("cols"))\
       .select(F.expr("cols['0']").alias("ner_chunk"),
               F.expr("cols['1']['entity']").alias("entities"),
               F.expr("cols['2']").alias("assertion")).show(truncate=False)

+---------------+---------+----------------------------+
ner_chunk |entities |assertion |
+---------------+---------+----------------------------+
severe fever |PROBLEM |present |
sore throat |PROBLEM |present |
pain |PROBLEM |present |
stomach pain |PROBLEM |absent |
an epidural |TREATMENT|present |
PCA |TREATMENT|present |
pain control |PROBLEM |present |
short of breath|PROBLEM |conditional |
CT |TEST |present |
lung tumor |PROBLEM |present |
Alzheimer |PROBLEM |associated_with_someone_else|
+---------------+---------+----------------------------+

#### Assertion Filterer Results

In [0]:
assertion_filterer.getWhiteList()

Out[117]: ['present']

Here are our filtered (only present assertions) chunks;

In [0]:
results.select("assertion_filtered.result").show(truncate=False)

+---------------------------------------------------------------------------------+
result |
+---------------------------------------------------------------------------------+
[severe fever, sore throat, pain, an epidural, PCA, pain control, CT, lung tumor]|
+---------------------------------------------------------------------------------+

As you see above, there is no **"stomach pain"**, **"short of breath"** and **"Alzheimer"** in filtered results

### Assertion Visualizer

Also, we can visualize the assertion results by using `AssertionVisualizer` of Sparknlp-Display library.

Now, we will create a LightPipeline with our existing model and visualize the results.

In [0]:
from sparknlp_display import AssertionVisualizer

light_model = LightPipeline(assertion_model)
light_result = light_model.fullAnnotate(text)[0]

vis = AssertionVisualizer()
asser_vis= vis.display(light_result, 'clinical_ner_chunk', 'assertion', return_html=True)
displayHTML(asser_vis)

Patient with severe fever PROBLEM present and sore throat PROBLEM present also suffers from pain PROBLEM present . He shows no stomach pain PROBLEM absent and he maintained on an epidural TREATMENT present and PCA TREATMENT present for pain control PROBLEM present . He also become short of breath PROBLEM conditional with climbing a flight of stairs. After CT TEST present , lung tumor PROBLEM present located at the right lower lobe. Father with Alzheimer PROBLEM associated_with_someone_else .

### Train a Custom Assertion Model

In this section, we will train a custom assertion model. <br/>

Firstly, download a sample dataset for training.

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv -P /dbfs/

In [0]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/i2b2_assertion_sample_short.csv")
assertion_df.show(5, truncate=100)

+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
 text| target| label|start|end|
+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
 She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .| undesired fertility|present| 1| 2|
 3) STATUS POST FALL .| fall|present| 3| 3|
 On physical exam , she is a healthy-appearing female with a normal voice and no exophthalmos .| exophthalmos| absent| 15| 15|
Abdomen distended , mild tenderness on the left , bowel sounds present , nohepatosplenomegaly .|nohepatosplenomegaly| absent| 13| 13|
+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
only showing top 5 rows

Splitting data into train and test set

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 721
Test Dataset Count: 170

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

+-------+-----+
label |count|
+-------+-----+
present|546 |
absent |175 |
+-------+-----+

We will transform our test data with a pipeline consisting of same steps with the pipeline which contains `AssertionDLApproach`.
By doing this, we enable that the test data will have the same columns with the training data in `AssertionDLApproach`. <br/>
The goal of this implementation is enabling the usage of `setTestDataset()` parameter in `AssertionDLApproach`.

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings])

assertion_test_data = clinical_assertion_pipeline.fit(test_data).transform(test_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
assertion_test_data.columns

Out[7]: ['text',
 'target',
 'label',
 'start',
 'end',
 'document',
 'chunk',
 'token',
 'embeddings']

We will save the test data in parquet format to use in `AssertionDLApproach()`.

In [0]:
assertion_test_data.write.parquet('dbfs:/i2b2_assertion_sample_test_data.parquet')

#### Graph Setup

In [0]:
# !pip install -q tensorflow==2.7.0
# !pip install -q tensorflow-addons

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [0]:
%fs mkdirs file:/dbfs/tf_graphs

res0: Boolean = true

In [0]:
assertion_graph_builder = TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder('file:/dbfs/tf_graphs')\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(250)\
    .setHiddenUnitsNumber(25)

In [0]:
%fs mkdirs file:/dbfs/assertion_logs

res1: Boolean = true

In [0]:
 # %fs mkdirs file:/dbfs/assertion_tf_graphs
 # %fs mkdirs file:/dbfs/assertion_logs

# if you want you can use existing graph

# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /dbfs/assertion_tf_graphs

In [0]:
# Create custom graph


# tf_graph.print_model_params("assertion_dl")

# feat_size = 200
# n_classes = 2

# tf_graph.build("assertion_dl",
#                build_params={"n_classes": n_classes},
#                model_location= "/dbfs/assertion_tf_graphs", 
#                model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

Now, let's define the `AssertionDLApproach()` to train a model

In [0]:
dbutils.fs.ls('/dbfs/ner')

Out[16]: [FileInfo(path='dbfs:/dbfs/ner/assertion_graphs', name='assertion_graphs', size=1332219),
 FileInfo(path='dbfs:/dbfs/ner/assertion_tf_graph', name='assertion_tf_graph', size=1332219),
 FileInfo(path='dbfs:/dbfs/ner/medical_ner_graphs', name='medical_ner_graphs', size=1754469),
 FileInfo(path='dbfs:/dbfs/ner/ner_logs_best/', name='ner_logs_best/', size=0),
 FileInfo(path='dbfs:/dbfs/ner/re_graphs', name='re_graphs', size=135162)]

In [0]:
scope_window = [10,10]

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(20)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('dbfs:/assertion_logs')\
    .setGraphFolder('dbfs:/tf_graphs')\
    .setGraphFile("file:/dbfs/tf_graphs/assertion_graph.pb")\
    .setTestDataset(path="/i2b2_assertion_sample_test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)


'''
If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
by a pipeline consisting of document, chunk, embeddings stages.
'''

Out[17]: '\nIf .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed\nby a pipeline consisting of document, chunk, embeddings stages.\n'

In [0]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertion_graph_builder,
    assertionStatus])

In [0]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: file:/dbfs/tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 2, 'feat_size': 200, 'max_seq_len': 250, 'n_hidden': 25}
/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:111: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Device mapping: no known devices.
/databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:715: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
 warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/rnn.py:229: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/rnn.py:441: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
 warnings.warn('`layer.add_variable` is deprecated and '
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:766: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Device mapping: no known devices.
assertion_dl graph exported to file:/dbfs/tf_graphs/assertion_graph.pb

In [0]:
assertion_model.stages

Out[20]: [DocumentAssembler_aa67827e6e2d,
 Doc2Chunk_a8a8715a2659,
 REGEX_TOKENIZER_2b19930e88cf,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 TFGraphBuilderModel_9f29ef83f6ee,
 ASSERTION_DL_7b37b18aeac3]

Now, let's get prediction from the test data

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+-------+---------+
 label| result|
+-------+---------+
present|[present]|
 absent|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
+-------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
preds_df

Out[24]:

,label,result
0,present,present
1,absent,present
2,present,present
3,present,present
4,present,present
...,...,...
165,present,present
166,absent,absent
167,absent,absent
168,absent,present


#### Evaluating the Model

We are going to use sklearn to evalute the results on test dataset

In [0]:
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['label']))

precision recall f1-score support

 absent 0.57 0.91 0.70 33
 present 0.97 0.83 0.90 137

 accuracy 0.85 170
 macro avg 0.77 0.87 0.80 170
weighted avg 0.90 0.85 0.86 170

And we can save the trained model

In [0]:
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/assertion_model')

# 4.Clinical Deidentification

Protected Health Information: 
- individual’s past, present, or future physical or mental health or condition
- provision of health care to the individual
- past, present, or future payment for the health care 

Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

<center><b>Deidentification NER Models for English</b></center>

|index|model|lang|index|model|lang|index|model|lang|
|-----:|:-----|----|-----:|:-----|----|-----:|:-----|----|
| 1| [deidentify_dl](https://nlp.johnsnowlabs.com/2021/01/28/deidentify_dl_en.html)  |en| 7| [ner_deid_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_enriched_biobert_en.html)  |en| 13| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2021/09/03/ner_deid_subentity_augmented_en.html)  |en|
| 2| [deidentify_large](https://nlp.johnsnowlabs.com/2020/08/04/deidentify_large_en.html)  |en| 8| [ner_deid_generic_augmented](https://nlp.johnsnowlabs.com/2021/06/30/ner_deid_generic_augmented_en.html)  |en| 14| [ner_deid_subentity_augmented_i2b2](https://nlp.johnsnowlabs.com/2021/11/29/ner_deid_subentity_augmented_i2b2_en.html)  |en|
| 3| [deidentify_rb](https://nlp.johnsnowlabs.com/2019/06/04/deidentify_rb_en.html)  |en| 9| [ner_deid_generic_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_generic_glove_en.html)  |en| 15| [ner_deid_subentity_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_subentity_glove_en.html)  |en|
| 4| [ner_deid_augmented](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_augmented_en.html)  |en| 10| [ner_deid_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_large_en.html)  |en| 16| [ner_deid_synthetic](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_synthetic_en.html)  |en|
| 5| [ner_deid_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_biobert_en.html)  |en| 11| [ner_deid_sd](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_en.html)  |en| 17| [ner_deidentify_dl](https://nlp.johnsnowlabs.com/2021/03/31/ner_deidentify_dl_en.html)  |en|
| 6| [ner_deid_enriched](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_enriched_en.html)  |en| 12| [ner_deid_sd_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_large_en.html)  |en| 18| []()|

<center><b>Deidentification NER Models for Other Languages</b></center>

|index|model|lang|index|model|lang|
|-----:|:-----|----|-----:|:-----|----|
| 1| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/01/06/ner_deid_generic_de.html)  |de| 11| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/02/11/ner_deid_generic_fr.html)  |fr|
| 2| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/01/06/ner_deid_subentity_de.html)  |de| 12| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/02/14/ner_deid_subentity_fr.html)  |fr|
| 3| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/01/18/ner_deid_generic_es.html)  |es| 13| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/03/25/ner_deid_generic_it_3_0.html)  |it|
| 4| [ner_deid_generic_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_generic_augmented_es.html)  |es| 14| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/03/25/ner_deid_subentity_it_2_4.html)  |it|
| 5| [ner_deid_generic_roberta](https://nlp.johnsnowlabs.com/2022/01/17/ner_deid_generic_roberta_es.html)  |es| 15| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/04/13/ner_deid_generic_pt_3_0.html)  |pt|
| 6| [ner_deid_generic_roberta_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_generic_roberta_augmented_es.html)  |es| 16| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/04/13/ner_deid_subentity_pt_3_0.html)  |pt|
| 7| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/01/18/ner_deid_subentity_es.html)  |es| 17| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/06/27/ner_deid_subentity_ro_3_0.html)  |ro|
| 8| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_subentity_augmented_es.html)  |es| 18| [ner_deid_subentity_bert](https://nlp.johnsnowlabs.com/2022/06/27/ner_deid_subentity_bert_ro_3_0.html)  |ro|
| 9| [ner_deid_subentity_roberta](https://nlp.johnsnowlabs.com/2022/01/17/ner_deid_subentity_roberta_es.html)  |es| 19| [ner_deid_generic](https://nlp.johnsnowlabs.com/models)  |ro|
| 10| [ner_deid_subentity_roberta_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_subentity_roberta_augmented_es.html)  |es| 20| [ner_deid_generic_bert](https://nlp.johnsnowlabs.com/models)  |ro|

You can find German, Spanish, French, Italian, Portuguese and Romanian deidentification models and pretrained pipeline examples in this notebook:   [Clinical Multi Language Deidentification Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.1.Clinical_Multi_Language_Deidentification.ipynb)

Firstly, we will load NER pipeline to identify protected entities. Pretrained NER models extracts:

- Name
- Profession
- Age
- Date
- Contact(Telephone numbers, FAX numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Employment information)
- Id (Social Security numbers, Medical record numbers, Internet protocol addresses)

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
deid_ner = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")


deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][OK!]

In [0]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .
'''

In [0]:
result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))

Let's check extracted sensitive entities by NER piepline

In [0]:
result.select(F.explode(F.arrays_zip(result.deid_ner_chunk.result, result.deid_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+---------+
chunk |ner_label|
+-----------------------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora |NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
Cocke County Baptist Hospital|LOCATION |
0295 Keats Street |LOCATION |
55-555-5555 |CONTACT |
+-----------------------------+---------+

Sometimes we need to leave some entities in the text, for example, if we want to analyze the frequency of the disease by the hospital. In this case, we may use the parameter **`setBlackList()`** to modify `ner_chunk` output. This parameter having using a list of entity types to deidentify as an input. So, if we want to exclude the location, we need to specify this tag in the list:

In [0]:
filtered_ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "deid_ner"])\
    .setOutputCol("deid_ner_chunk") \
    .setBlackList(['LOCATION'])

deidPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    deid_ner,
    filtered_ner_converter])


model_with_black_list = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

result_with_black_list = model_with_black_list.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
print("All Labels :")
result.select(F.explode(F.arrays_zip(result.deid_ner_chunk.result, result.deid_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

print("BlackListed Labels: ")
result_with_black_list.select(F.explode(F.arrays_zip(result_with_black_list.deid_ner_chunk.result, 
                                                     result_with_black_list.deid_ner_chunk.metadata)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

All Labels :
+-----------------------------+---------+
chunk |ner_label|
+-----------------------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora |NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
Cocke County Baptist Hospital|LOCATION |
0295 Keats Street |LOCATION |
55-555-5555 |CONTACT |
+-----------------------------+---------+

BlackListed Labels: 
+---------------+---------+
chunk |ner_label|
+---------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora|NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
55-555-5555 |CONTACT |
+---------------+---------+

### Masking

Now, we will replace this enitites with tags, this process is called **"masking"**. We will specifiy `setMode()` parameter as `setMode("mask")` in `DeIdentification()` in order to apply masking

In [0]:
deid_ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk") 

deidentification = DeIdentification() \
      .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deidentification])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deidentified| aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Record date : 20...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 19, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Let's check the results

In [0]:
pd.set_option('display.max_colwidth', None)

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[204]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <CONTACT> ."


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>
Let's try each of these and compare the results.

In [0]:
#deid model with "entity_labels"
deid_entity_labels= DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
policy_result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deid_entity_label| aux| deid_same_length| deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Record date : 20...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 17, ...|[{document, 0, 44...|[{document, 0, 32...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
policy_result.select(F.explode(F.arrays_zip(policy_result.sentence.result, 
                                            policy_result.deid_entity_label.result, 
                                            policy_result.deid_same_length.result, 
                                            policy_result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

Out[207]:

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D .","Record date : [********] , [********] , M.D .","Record date : **** , **** , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> .",", Name : [*************] , MR [******] Date : [******] .",", Name : **** , MR **** Date : **** ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> .","PCP : [******] , ** years-old , Record date : [********] .","PCP : **** , **** years-old , Record date : **** ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <CONTACT> .","[***************************] , [***************] , Phone [*********] .","**** , **** , Phone **** ."


### Reidentification

We can use `ReIdentification` annotator to go back to the original sentence. Let's see how it works

In [0]:
reIdentification = ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [0]:
reid_result = reIdentification.transform(result)
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deidentified| aux| original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Record date : 20...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 19, ...|[{document, 1, 45...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
print(text)

reid_result.select('original.result').show(truncate=False)

Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[Record date : 2093-01-13 , David Hale , M.D ., , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 ., PCP : Oliveira , 25 years-old , Record date : 2079-11-09 ., Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

### Enriching with Regex and Override NER

We can enrich our existing Deidentification model with both custom and embedded regex.

Firstly, let's build our deid pipeline again, but this time by filtering "`CONTACT`" entities from ner model. 

Then, we will set `setReturnEntityMappings(True)`. In this way, we are going to see that we still be able to deidentify "`CONTACT`" as `PHONE` even though ner model doesn't detect "`CONTACT`" entity since the regex embedded in deid model will be taken account.

In [0]:
filtered_ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")\
      .setBlackList(["CONTACT"])

deidentification = DeIdentification() \
      .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setRegexOverride(True)

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      filtered_ner_converter,
      deidentification])

model_default_rgx = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
result = model_default_rgx.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

Out[212]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <PHONE> ."


As you see above, we were able to deidentify `CONTACT` entity as `PHONE` which comes from the embedded regex even though ner model doesn't detect them.

Now, let's identify a custom regex and see how it works.

We will write a regex to deidentify "number" and save it as .txt file. Then, we will specify this file in `Deidentification()` through `setRegexPatternsDictionary()` parameter.

In [0]:
import os
import json
import string

In [0]:
rgx = '''NUMBER ([0-9]{2})
NUMBER (\d{7})''' 

with open("/dbfs/custom_regex.txt", "w") as f:
  f.write(rgx)


In [0]:
deid_ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

deidentification_rgx = DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deidentified")\
    .setMode("mask") \
    .setRegexPatternsDictionary("file:/dbfs/custom_regex.txt")\
    .setRegexOverride(True)

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deidentification_rgx])

model_rgx = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
text ='''Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street.'''
result = model_rgx.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

Out[216]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson , Ora MR .",", Name : <NAME> MR ."
2,"# 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","# <NUMBER> Date : <DATE> PCP : <NAME> , <NUMBER> years-old , Record date : <DATE> ."
3,Cocke County Baptist Hospital .,<LOCATION> .
4,0295 Keats Street.,<LOCATION>.


As you see above, we deidentified the "number" through our custom regex even though ner model doesn't detect the "number" entity.

### Obfuscation

In the obfuscation mode **DeIdentificationModel** will replace sensetive entities with random values of the same type. 
We can specify `.setMode()` parameter as `.setMode("obfuscate")` to use this feature. 

Let's create a piepline and see how it works

In [0]:
deid_ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

obfuscation = DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      obfuscation])

obfuscation_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR #7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

Out[218]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-02-02 , Cloyde Budd , M.D ."
1,", Name : Hendrickson , Ora MR #7194334 Date : 01/13/93 .",", Name : Alphonza Camera L961584 Date : 10-20-1983 ."
2,"Patient : Oliveira, 25 years-old , Record date : 2079-11-09 .","Patient : Carol Stabs, 78 years-old , Record date : 2079-11-22 ."
3,Cocke County Baptist Hospital .,3001 w dr. martin luther king jr blvd .
4,0295 Keats Street,1501 hartford st


As you see, we succesfully obfuscated the entities.

In addition, we can define a reference file for obfuscation. In this way, deid model uses the fake entities that we defined in this file.

If we want to use reference file, we should give the path of txt file that includes our custom entities to the `setObfuscateRefFile()` parameter. Then, we should set `setObfuscateRefSource("file")` parameter.

However, we can set `setObfuscateRefSource` parameter as **faker** or **both** as well. 

The faker module allows  the user to use a set of fake entities that are in the memory of the spark-nlp-internal. You can set up this module for using the following property `setObfuscateRefSource('faker')`.

If we select the `setObfuscateRefSource('both')` then we choose randomly the entities using the faker and the fake entities from the obfuscateRefFile.

Now, let's define a reference file and use it in our deidentification model.

In [0]:
obs_lines = """Marvin MARSHALL#NAME
Hubert GROGAN#NAME
ALTHEA COLBURN#NAME
Kalil AMIN#NAME
Inci FOUNTAIN#NAME
Ekaterina Rosa#NAME
Rudiger Chao#NAME
COLLETTE KOHLER#NAME
Mufi HIGGS#NAME"""


with open ('/dbfs/obfuscation.txt', 'w') as f:
  f.write(obs_lines)

In [0]:
deid_ner_converter = NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

obfuscation = DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('/dbfs/obfuscation.txt')\
    .setObfuscateRefSource("file") #both or faker

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      obfuscation])


obfuscation_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[221]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-03-03 , Marvin MARSHALL , M.D ."
1,", Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 .",", Name : Marvin MARSHALL MR # <ID> Date : 03-14-1997 ."
2,"Patient : Oliveira, 25 years-old , Record date : 2079-11-09 .","Patient : Ekaterina Rosa, <AGE> years-old , Record date : 2079-12-06 ."
3,Cocke County Baptist Hospital .,<LOCATION> .
4,0295 Keats Street,<LOCATION>


**`.setObfuscateDate(True)` parameter obfuscated the `DATE` entities and the `NAME`s were replaced with the custom list.**

### Structured Deidentification

We can use deidentification not only text but also structured data as well.

Let's see how it works

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/hipaa-table-001.txt
  
dbutils.fs.cp("file:/databricks/driver/hipaa-table-001.txt", "dbfs:/")

Out[222]: True

In [0]:
df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("/hipaa-table-001.txt")

df = df.withColumnRenamed("PATIENT","NAME")
df.show(truncate=False)

+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
NAME |DOB |AGE|ADDRESS |ZIPCODE|TEL |SBP|DBP|
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
Cecilia Chapman|04/02/1935|83 |711-2880 Nulla St. Mankato Mississippi |69200 |(257) 563-7401|101|42 |
Iris Watson |03/10/2009|9 |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska |20620 |(372) 587-2335|159|122|
Bryar Pitts |11/01/1921|98 |5543 Aliquet St. Fort Dodge GA |20783 |(717) 450-4729|149|52 |
Theodore Lowe |13/02/2002|16 |Ap #867-859 Sit Rd. Azusa New York |39531 |(793) 151-6230|134|115|
Calista Wise |20/08/1942|76 |7292 Dictum Av. San Antonio MI |47096 |(492) 709-6392|139|78 |
Kyla Olsen |12/05/1973|45 |Ap #651-8679 Sodales Av. Tamuning PA |10855 |(654) 393-5734|120|112|
Forrest Ray |11/01/1991|27 |191-103 Integer Rd. Corona New Mexico |8219 |(404) 960-3807|143|126|
Hiroko Potter |18/11/1937|81 |P.O. Box 887 2508 Dolor. Av. Muskegon KY |12482 |(314) 244-6306|147|75 |
Celeste Slater |12/05/1980|38 |606-3727 Ullamcorper. Street Roseville NH |11523 |(786) 713-8616|147|123|
Nyssa Vazquez |24/09/1956|62 |511-5762 At Rd. Chelsea MI |67708 |(947) 278-5929|129|50 |
Lawrence Moreno|26/12/1906|112|935-9940 Tortor. Street Santa Rosa MN |98804 |(684) 579-1879|133|102|
Ina Moran |26/10/1983|35 |P.O. Box 929 4189 Nunc Road Lebanon KY |69409 |(389) 737-2852|101|67 |
Aaron Hawkins |26/09/2009|9 |5587 Nunc. Avenue Erie Rhode Island |24975 |(660) 663-4518|87 |81 |
Hedy Greene |03/10/1920|98 |Ap #696-3279 Viverra. Avenue Latrobe DE |38100 |(608) 265-2215|128|123|
Melvin Porter |14/08/1911|107|P.O. Box 132 1599 Curabitur Rd. Bandera South Dakota|45149 |(959) 119-8364|83 |43 |
Keefe Sellers |16/05/1937|81 |347-7666 Iaculis St. Woodruff SC |49854 |(468) 353-2641|148|109|
Joan Romero |08/12/2004|14 |666-4366 Lacinia Avenue Idaho Falls Ohio |19253 |(248) 675-4007|75 |53 |
Davis Patrick |09/01/1956|63 |P.O. Box 147 2546 Sociosqu Rd. Bethlehem Utah |2913 |(939) 353-1107|142|62 |
Leilani Boyer |18/10/1934|84 |557-6308 Lacinia Road San Bernardino ND |9289 |(570) 873-7090|137|48 |
Colby Bernard |02/10/1905|113|Ap #285-7193 Ullamcorper Avenue Amesbury HI |93373 |(302) 259-2375|84 |41 |
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
only showing top 20 rows

In [0]:
from sparknlp_jsl.structured_deidentification import StructuredDeidentification

In [0]:
obfuscator = StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, obfuscateRefSource = "faker")
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.show(truncate=False)

+------------------+----------+----+----------------------------------------------------+-------+--------------+---+---+
NAME |DOB |AGE |ADDRESS |ZIPCODE|TEL |SBP|DBP|
+------------------+----------+----+----------------------------------------------------+-------+--------------+---+---+
[Oralia Grove] |04/02/1935|[47]|711-2880 Nulla St. Mankato Mississippi |69200 |(257) 563-7401|101|42 |
[Kenny Hamburger] |03/10/2009|[8] |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska |20620 |(372) 587-2335|159|122|
[Enrique Oliphant]|11/01/1921|[47]|5543 Aliquet St. Fort Dodge GA |20783 |(717) 450-4729|149|52 |
[Ninette Root] |13/02/2002|[70]|Ap #867-859 Sit Rd. Azusa New York |39531 |(793) 151-6230|134|115|
[Helaine Spikes] |20/08/1942|[11]|7292 Dictum Av. San Antonio MI |47096 |(492) 709-6392|139|78 |
[Sharan League] |12/05/1973|[38]|Ap #651-8679 Sodales Av. Tamuning PA |10855 |(654) 393-5734|120|112|
[Lida Pitter] |11/01/1991|[62]|191-103 Integer Rd. Corona New Mexico |8219 |(404) 960-3807|143|126|
[Delmon Leavens] |18/11/1937|[62]|P.O. Box 887 2508 Dolor. Av. Muskegon KY |12482 |(314) 244-6306|147|75 |
[Juana Ahr] |12/05/1980|[68]|606-3727 Ullamcorper. Street Roseville NH |11523 |(786) 713-8616|147|123|
[Avis Coca] |24/09/1956|[25]|511-5762 At Rd. Chelsea MI |67708 |(947) 278-5929|129|50 |
[Craig Partridge] |26/12/1906|[36]|935-9940 Tortor. Street Santa Rosa MN |98804 |(684) 579-1879|133|102|
[Rexine Bayley] |26/10/1983|[34]|P.O. Box 929 4189 Nunc Road Lebanon KY |69409 |(389) 737-2852|101|67 |
[Von Glassman] |26/09/2009|[8] |5587 Nunc. Avenue Erie Rhode Island |24975 |(660) 663-4518|87 |81 |
[Ardyth Lord] |03/10/1920|[47]|Ap #696-3279 Viverra. Avenue Latrobe DE |38100 |(608) 265-2215|128|123|
[Marykay Michaels]|14/08/1911|[84]|P.O. Box 132 1599 Curabitur Rd. Bandera South Dakota|45149 |(959) 119-8364|83 |43 |
[Randell Bourbon] |16/05/1937|[62]|347-7666 Iaculis St. Woodruff SC |49854 |(468) 353-2641|148|109|
[Luberta Ground] |08/12/2004|[79]|666-4366 Lacinia Avenue Idaho Falls Ohio |19253 |(248) 675-4007|75 |53 |
[Lola Hensen] |09/01/1956|[39]|P.O. Box 147 2546 Sociosqu Rd. Bethlehem Utah |2913 |(939) 353-1107|142|62 |
[Elana Hau] |18/10/1934|[5] |557-6308 Lacinia Road San Bernardino ND |9289 |(570) 873-7090|137|48 |
[Charlyne Marcus] |02/10/1905|[20]|Ap #285-7193 Ullamcorper Avenue Amesbury HI |93373 |(302) 259-2375|84 |41 |
+------------------+----------+----+----------------------------------------------------+-------+--------------+---+---+
only showing top 20 rows

We can define reference file for obfuscation as well like we did before. All we need to do is defining `obfuscateRefFile` and `onfuscateRefSource` parameters.

In [0]:
obfuscator_unique_ref_test = '''Will Perry#PATIENT
John Smith#PATIENT
Marvin MARSHALL#PATIENT
Hubert GROGAN#PATIENT
ALTHEA COLBURN#PATIENT
Kalil AMIN#PATIENT
Inci FOUNTAIN#PATIENT
Jackson WILLE#PATIENT
Jack SANTOS#PATIENT
Mahmood ALBURN#PATIENT
Marnie MELINGTON#PATIENT
Aysha GHAZI#PATIENT
Maryland CODER#PATIENT
Darene GEORGIOUS#PATIENT
Shelly WELLBECK#PATIENT
Min Kun JAE#PATIENT
Thomson THOMAS#PATIENT
Christian SUDDINBURG#PATIENT
Aberdeen#CITY
Louisburg St#STREET
France#LOC
Nick Riviera#DOCTOR
5552312#PHONE
St James Hospital#HOSPITAL
Calle del Libertador#ADDRESS
111#ID
Will#DOCTOR
20#AGE
30#AGE
40#AGE
50#AGE
60#AGE
'''

with open('/dbfs/obfuscator_unique_ref_test.txt', 'w') as f:
  f.write(obfuscator_unique_ref_test)

In [0]:
# obfuscateRefSource = "file"

obfuscator = StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, 
                                        obfuscateRefFile = "/dbfs/obfuscator_unique_ref_test.txt",
                                        obfuscateRefSource = "file",
                                        columnsSeed={"NAME": 23, "AGE": 23})
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.select("NAME","AGE").show(truncate=False)

+------------------+----+
NAME |AGE |
+------------------+----+
[Inci FOUNTAIN] |[60]|
[Jack SANTOS] |[30]|
[Darene GEORGIOUS]|[30]|
[Shelly WELLBECK] |[40]|
[Hubert GROGAN] |[40]|
[Kalil AMIN] |[40]|
[ALTHEA COLBURN] |[60]|
[Thomson THOMAS] |[60]|
[Jack SANTOS] |[60]|
[Will Perry] |[20]|
[Jackson WILLE] |[60]|
[Shelly WELLBECK] |[40]|
[Kalil AMIN] |[30]|
[Marnie MELINGTON]|[30]|
[Min Kun JAE] |[30]|
[Marvin MARSHALL] |[60]|
[Marvin MARSHALL] |[50]|
[Min Kun JAE] |[30]|
[Maryland CODER] |[20]|
[Marnie MELINGTON]|[20]|
+------------------+----+
only showing top 20 rows

### Pretrained Deidentification Pipeline

This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities.

Let's see how it works

In [0]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification", "en", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
text= """Name : Hendrickson, Ora, Record date: 2093-01-13, Age: 25, #719435. Dr. John Green, ID: 1231511863, IP 203.120.223.13. He is a 60-year-old male was admitted to the Day Hospital for cystectomy on 01/13/93. Patient's VIN : 1HGBH41JXMN109286, SSN #333-44-6666, Driver's license no:A334455B. Phone: 55-555-5555 , 0295 Keats Street, San Francisco"""

In [0]:
deid_res= deid_pipeline.annotate(text)

In [0]:
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"], 
                          deid_res["masked"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"], 
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

Out[5]:

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"Name : Hendrickson, Ora, Record date: 2093-01-13, Age: 25, #719435.","Name : <PATIENT>, Record date: <DATE>, Age: <AGE>, <MEDICALRECORD>.","Name : [**************], Record date: [********], Age: **, [*****].","Name : ****, Record date: ****, Age: ****, ****.","Name : Alexia Mcgill, Record date: 2093-02-13, Age: 35, F1808999."
1,"Dr. John Green, ID: 1231511863, IP 203.120.223.13.","Dr. <DOCTOR>, ID<IDNUM>, IP <IPADDR>.","Dr. [********], ID[**********], IP [************].","Dr. ****, ID****, IP ****.","Dr. Dr Dwyane Gaba, IDEA:3090906, IP 444.444.444.444."
2,He is a 60-year-old male was admitted to the Day Hospital for cystectomy on 01/13/93.,He is a <AGE> male was admitted to the <LOCATION> for cystectomy on <DATE>.,He is a [*********] male was admitted to the [**********] for cystectomy on [******].,He is a **** male was admitted to the **** for cystectomy on ****.,He is a 60-year-old male was admitted to the 401 Nw 42Nd Ave for cystectomy on 03-31-2003.
3,"Patient's VIN : 1HGBH41JXMN109286, SSN #333-44-6666, Driver's license no:A334455B.","Patient's VIN : <VIN>, SSN <SSN>, Driver's license <DLN>.","Patient's VIN : [***************], SSN [**********], Driver's license [*********].","Patient's VIN : ****, SSN ****, Driver's license ****.","Patient's VIN : 3CCCC22DDDD333888, SSN SSN-963-29-5264, Driver's license S99978095."
4,"Phone: 55-555-5555 , 0295 Keats Street, San Francisco","Phone: <PHONE> , <STREET>, <CITY>","Phone: [*********] , [***************], [***********]","Phone: **** , ****, ****","Phone: 519-215-2027 , 98 Spruce St, STAVANGER"


# 5.Clinical Relation Extraction

### Pretrained Relation Extraction Models

Extracts and classifies instances of relations between named entities. For this, relation pairs need to be defined with `setRelationPairs`, to specify between which entities the extraction should be done.

We have also trained as end-to-end bert models using **BioBERT** and ported in to the Spark NLP ecosystem (**ReDL**). They offer **SOTA performance** on most benchmark tasks and outperform our existing Relation Extraction Models.

*  **Relation Extraction Models**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [posology_re](https://nlp.johnsnowlabs.com/2020/09/01/posology_re.html)  | 16| [re_oncology_size_wip](https://nlp.johnsnowlabs.com/2022/09/26/re_oncology_size_wip_en.html)  | 31| [redl_date_clinical_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_date_clinical_biobert_en.html)  |
| 2| [re_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/16/re_ade_biobert_en.html)  | 17| [re_oncology_temporal_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_temporal_wip_en.html)  | 32| [redl_drug_drug_interaction_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_drug_drug_interaction_biobert_en.html)  |
| 3| [re_ade_clinical](https://nlp.johnsnowlabs.com/2021/07/12/re_ade_clinical_en.html)  | 18| [re_oncology_test_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_test_result_wip_en.html)  | 33| [redl_drugprot_biobert](https://nlp.johnsnowlabs.com/2022/01/05/redl_drugprot_biobert_en.html)  |
| 4| [re_ade_conversational](https://nlp.johnsnowlabs.com/2022/07/27/re_ade_conversational_en_3_0.html)  | 19| [re_oncology_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_wip_en.html)  | 34| [redl_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_human_phenotype_gene_biobert_en.html)  |
| 5| [re_bodypart_directions](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_directions_en.html)  | 20| [re_temporal_events_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_clinical_en.html)  | 35| [redl_nihss_biobert](https://nlp.johnsnowlabs.com/2021/11/16/redl_nihss_biobert_en.html)  |
| 6| [re_bodypart_problem](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_problem_en.html)  | 21| [re_temporal_events_enriched_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_enriched_clinical_en.html)  | 36| [redl_oncology_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biobert_wip_en.html)  |
| 7| [re_bodypart_proceduretest](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_proceduretest_en.html)  | 22| [re_test_problem_finding](https://nlp.johnsnowlabs.com/2021/04/19/re_test_problem_finding_en.html)  | 37| [redl_oncology_biomarker_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biomarker_result_biobert_wip_en.html)  |
| 8| [re_clinical](https://nlp.johnsnowlabs.com/2020/09/24/re_clinical_en.html)  | 23| [re_test_result_date](https://nlp.johnsnowlabs.com/2021/02/24/re_test_result_date_en.html)  | 38| [redl_oncology_granular_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_granular_biobert_wip_en.html)  |
| 9| [re_date_clinical](https://nlp.johnsnowlabs.com/2021/01/18/re_date_clinical_en.html)  | 24| [re_zeroshot_biobert](https://nlp.johnsnowlabs.com/2022/04/05/re_zeroshot_biobert_en_3_0.html)  | 39| [redl_oncology_location_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_location_biobert_wip_en.html)  |
| 10| [re_drug_drug_interaction_clinical](https://nlp.johnsnowlabs.com/2020/09/03/re_drug_drug_interaction_clinical_en.html)  | 25| [redl_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/12/redl_ade_biobert_en.html)  | 40| [redl_oncology_size_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/28/redl_oncology_size_biobert_wip_en.html)  |
| 11| [re_drugprot_clinical](https://nlp.johnsnowlabs.com/2022/01/05/re_drugprot_clinical_en.html)  | 26| [redl_bodypart_direction_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_direction_biobert_en.html)  | 41| [redl_oncology_temporal_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_temporal_biobert_wip_en.html)  |
| 12| [re_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/30/re_human_phenotype_gene_clinical_en.html)  | 27| [redl_bodypart_problem_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_problem_biobert_en.html)  | 42| [redl_oncology_test_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_test_result_biobert_wip_en.html)  |
| 13| [re_oncology_biomarker_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_biomarker_result_wip_en.html)  | 28| [redl_bodypart_procedure_test_biobert](https://nlp.johnsnowlabs.com/2021/09/10/redl_bodypart_procedure_test_biobert_en.html)  | 43| [redl_temporal_events_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_temporal_events_biobert_en.html)  |
| 14| [re_oncology_granular_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_granular_wip_en.html)  | 29| [redl_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_chemprot_biobert_en.html)  | |
| 15| [re_oncology_location_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_location_wip_en.html)  | 30| [redl_clinical_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_clinical_biobert_en.html)  | |

[In this link](https://nlp.johnsnowlabs.com/docs/en/best_practices_pretrained_models#relation-extraction-models-and-relation-pairs-table), available Relation Extraction models, its labels, optimal NER model, and meaningful relation pairs are illustrated.

#### Posology Relation Extraction

This is a demonstration of using SparkNLP for extracting posology relations. The following relatios are supported:

DRUG-DOSAGE
DRUG-FREQUENCY
DRUG-ADE (Adversed Drug Events)
DRUG-FORM
DRUG-ROUTE
DRUG-DURATION
DRUG-REASON
DRUG=STRENGTH

The model has been validated against the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

**Build pipeline using SparNLP pretrained models and the relation extration model optimized for posology**.
 
 The precision of the RE model is controlled by "setMaxSyntacticDistance(4)", which sets the maximum syntactic distance between named entities to 4. A larger value will improve recall at the expense at lower precision. A value of 4 leads to literally perfect precision (i.e. the model doesn't produce any false positives) and reasonably good recall.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "posology_ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
#   .setRelationPairs(["DRUG-FREQUENCY, FREQUENCY-DRUG", "DOSAGE-DRUG, DRUG-DOSAGE"])\
#   .setRelationPairsCaseSensitive(False)

relationPipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    reModel
])


relation_model = relationPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]

Now, let's create a light pipeline for annotating free text

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

lmodel = LightPipeline(relation_model)
results = lmodel.fullAnnotate(text)

In [0]:
from sparknlp_display import RelationExtractionVisualizer

'''
The `RelationExtractionVisualizer` can be used to visualize the relations predicted by Spark NLP. 
The two entities involved in a relation will be highlighted and their label will be displayed. 
Also a directed and labeled arc(line) will be used to connect the two entities.
'''
visualizer = RelationExtractionVisualizer()
vis= visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

/databricks/python/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
 relation_coordinates = np.array(relation_coordinates)

The patient was prescribed 1 unit DOSAGE of Advil DRUG for 5 days DURATION after meals. The patient was also given 1 unit DOSAGE of Metformin DRUG daily FREQUENCY . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . <polyline fill="none" points="333.0,155.0 332.11224489795916,153.7960798646822 331.2244897959183,152.59312871337622 330.3367346938776,151.39211553009378 329.44897959183675,150.19400929884657 328.5612244897959,148.99977900364644 327.67346938775506,147.81039362850512 326.78571428571433,146.62682215743445 325.8979591836735,145.45003357444605 325.01020408163265,144.28099686355176 324.1224489795918,143.12068100876337 323.234693877551,141.97005499409258 322.34693877551024,140.83008780355124 321.4591836734694,139.70174842115105 320.57142857142856,138.5860058309038 319.6836734693878,137.48382901682123 318.7959183673469,136.39618696291512 317.90816326530614,135.32404865319725 317.0204081632653,134.2683830716793 316.1326530612245,133.23015920237316 315.2448979591836,132.2103460292905 314.3571428571428,131.20991253644314 313.46938775510205,130.22982770784282 312.5816326530612,129.2710605275013 311.6938775510204,128.33457997943034 310.80612244897964,127.42135504764173 309.9183673469388,126.53235471614718 309.03061224489795,125.6685479689585 308.14285714285717,124.83090379008749 307.2551020408163,124.02039116354581 306.36734693877554,123.2379790733453 305.4795918367347,122.48463650349771 304.59183673469386,121.76133243801479 303.7040816326531,121.06903586090831 302.81632653061223,120.40871575619002 301.92857142857144,119.78134110787173 301.0408163265306,119.18788089996517 300.1530612244898,118.62930411648207 299.265306122449,118.10657974143427 298.3775510204082,117.62067675883348 297.48979591836735,117.17256415269148 296.60204081632656,116.76321090702004 295.7142857142857,116.39358600583091 294.8265306122449,116.06465843313586 293.9387755102041,115.77739717294665 293.05102040816325,115.53277120927505 292.16326530612247,115.33174952613282 291.2755102040816,115.17530110753174 290.38775510204084,115.06439493748353 289.5,115.0 289.5,115.0 288.61224489795916,114.9827622844223 287.7244897959184,115.01203580140927 286.83673469387753,115.08685156694916 285.94897959183675,115.20624059703015 285.0612244897959,115.36923390764053 284.17346938775506,115.57486251476851 283.28571428571433,115.82215743440234 282.39795918367355,116.11014968253025 281.51020408163276,116.43787027514047 280.6224489795918,116.80435022822121 279.73469387755097,117.20862055776078 278.8469387755101,117.64971227974738 277.95918367346934,118.12665641016923 277.07142857142856,118.63848396501459 276.1836734693878,119.18422596027166 275.2959183673469,119.7629134119287 274.40816326530614,120.37357733597398 273.5204081632653,121.01524874839565 272.63265306122446,121.68695866518203 271.7448979591836,122.3877381023213 270.85714285714283,123.11661807580174 269.96938775510205,123.87262960161156 269.0816326530612,124.654803695739 268.1938775510204,125.46217137417233 267.3061224489796,126.29376365289971 266.41836734693874,127.14861154790944 265.53061224489795,128.02574607518974 264.64285714285717,128.92419825072886 263.7551020408163,129.842999090515 262.86734693877554,130.7811796105364 261.97959183673476,131.73777082678137 261.09183673469386,132.71180375523804 260.2040816326531,133.7023094118947 259.31632653061223,134.7083188127396 258.42857142857144,135.72886297376095 257.54081632653066,136.76297291094698 256.6530612244898,137.80967964028594 255.76530612244898,138.86801417776604 254.87755102040816,139.9370075393756 253.98979591836735,141.01569074110276 253.10204081632654,142.1030947989358 252.21428571428572,143.19825072886297 251.32653061224488,144.30018954687245 250.4387755102041,145.40794226895255 249.55102040816325,146.52053991109145 248.66326530612247,147.63701348927742 247.7

In [0]:
results[0].keys()

Out[235]: dict_keys(['posology_ner', 'document', 'posology_ner_chunk', 'token', 'relations', 'embeddings', 'pos_tags', 'dependencies', 'sentence'])

In [0]:
results[0]['posology_ner_chunk']

Out[236]: [Annotation(chunk, 28, 33, 1 unit, {'chunk': '0', 'confidence': '0.71675', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '0'}),
 Annotation(chunk, 38, 42, Advil, {'chunk': '1', 'confidence': '0.9984', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 44, 53, for 5 days, {'chunk': '2', 'confidence': '0.7455', 'ner_source': 'posology_ner_chunk', 'entity': 'DURATION', 'sentence': '0'}),
 Annotation(chunk, 96, 101, 1 unit, {'chunk': '3', 'confidence': '0.72360003', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '1'}),
 Annotation(chunk, 106, 114, Metformin, {'chunk': '4', 'confidence': '0.9998', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '1'}),
 Annotation(chunk, 116, 120, daily, {'chunk': '5', 'confidence': '0.9997', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '1'}),
 Annotation(chunk, 190, 197, 40 units, {'chunk': '6', 'confidence': '0.84085', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '2'}),
 Annotation(chunk, 202, 217, insulin glargine, {'chunk': '7', 'confidence': '0.87525', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 219, 226, at night, {'chunk': '8', 'confidence': '0.76839995', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'}),
 Annotation(chunk, 231, 238, 12 units, {'chunk': '9', 'confidence': '0.88755', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '2'}),
 Annotation(chunk, 243, 256, insulin lispro, {'chunk': '10', 'confidence': '0.77805', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 258, 267, with meals, {'chunk': '11', 'confidence': '0.9213', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'}),
 Annotation(chunk, 275, 283, metformin, {'chunk': '12', 'confidence': '0.9997', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 285, 291, 1000 mg, {'chunk': '13', 'confidence': '0.84535', 'ner_source': 'posology_ner_chunk', 'entity': 'STRENGTH', 'sentence': '2'}),
 Annotation(chunk, 293, 307, two times a day, {'chunk': '14', 'confidence': '0.806225', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'})]

In [0]:
results[0]['relations']

Out[237]: [Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity2_begin': '38', 'entity1': 'DOSAGE', 'entity1_begin': '28', 'direction': 'both', 'entity1_end': '33', 'entity2': 'DRUG'}),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity2_begin': '44', 'entity1': 'DRUG', 'entity1_begin': '38', 'direction': 'both', 'entity1_end': '42', 'entity2': 'DURATION'}),
 Annotation(category, 96, 114, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '114', 'chunk1': '1 unit', 'entity2_begin': '106', 'entity1': 'DOSAGE', 'entity1_begin': '96', 'direction': 'both', 'entity1_end': '101', 'entity2': 'DRUG'}),
 Annotation(category, 106, 120, DRUG-FREQUENCY, {'chunk2': 'daily', 'confidence': '1.0', 'entity2_end': '120', 'chunk1': 'Metformin', 'entity2_begin': '116', 'entity1': 'DRUG', 'entity1_begin': '106', 'direction': 'both', 'entity1_end': '114', 'entity2': 'FREQUENCY'}),
 Annotation(category, 190, 217, DOSAGE-DRUG, {'chunk2': 'insulin glargine', 'confidence': '1.0', 'entity2_end': '217', 'chunk1': '40 units', 'entity2_begin': '202', 'entity1': 'DOSAGE', 'entity1_begin': '190', 'direction': 'both', 'entity1_end': '197', 'entity2': 'DRUG'}),
 Annotation(category, 202, 226, DRUG-FREQUENCY, {'chunk2': 'at night', 'confidence': '1.0', 'entity2_end': '226', 'chunk1': 'insulin glargine', 'entity2_begin': '219', 'entity1': 'DRUG', 'entity1_begin': '202', 'direction': 'both', 'entity1_end': '217', 'entity2': 'FREQUENCY'}),
 Annotation(category, 231, 256, DOSAGE-DRUG, {'chunk2': 'insulin lispro', 'confidence': '1.0', 'entity2_end': '256', 'chunk1': '12 units', 'entity2_begin': '243', 'entity1': 'DOSAGE', 'entity1_begin': '231', 'direction': 'both', 'entity1_end': '238', 'entity2': 'DRUG'}),
 Annotation(category, 243, 267, DRUG-FREQUENCY, {'chunk2': 'with meals', 'confidence': '1.0', 'entity2_end': '267', 'chunk1': 'insulin lispro', 'entity2_begin': '258', 'entity1': 'DRUG', 'entity1_begin': '243', 'direction': 'both', 'entity1_end': '256', 'entity2': 'FREQUENCY'}),
 Annotation(category, 275, 291, DRUG-STRENGTH, {'chunk2': '1000 mg', 'confidence': '1.0', 'entity2_end': '291', 'chunk1': 'metformin', 'entity2_begin': '285', 'entity1': 'DRUG', 'entity1_begin': '275', 'direction': 'both', 'entity1_end': '283', 'entity2': 'STRENGTH'}),
 Annotation(category, 275, 307, DRUG-FREQUENCY, {'chunk2': 'two times a day', 'confidence': '1.0', 'entity2_end': '307', 'chunk1': 'metformin', 'entity2_begin': '293', 'entity1': 'DRUG', 'entity1_begin': '275', 'direction': 'both', 'entity1_end': '283', 'entity2': 'FREQUENCY'})]

Showing extracted relations

In [0]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result,
        rel.metadata['entity1'], 
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Advil)
DRUG-DURATION(DRUG=Advil - DURATION=for 5 days)
DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Metformin)
DRUG-FREQUENCY(DRUG=Metformin - FREQUENCY=daily)
DOSAGE-DRUG(DOSAGE=40 units - DRUG=insulin glargine)
DRUG-FREQUENCY(DRUG=insulin glargine - FREQUENCY=at night)
DOSAGE-DRUG(DOSAGE=12 units - DRUG=insulin lispro)
DRUG-FREQUENCY(DRUG=insulin lispro - FREQUENCY=with meals)
DRUG-STRENGTH(DRUG=metformin - STRENGTH=1000 mg)
DRUG-FREQUENCY(DRUG=metformin - FREQUENCY=two times a day)

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df


rel_df = get_relations_df (results)
rel_df

Out[239]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DOSAGE-DRUG,DOSAGE,28,33,1 unit,DRUG,38,42,Advil,1.0
1,DRUG-DURATION,DRUG,38,42,Advil,DURATION,44,53,for 5 days,1.0
2,DOSAGE-DRUG,DOSAGE,96,101,1 unit,DRUG,106,114,Metformin,1.0
3,DRUG-FREQUENCY,DRUG,106,114,Metformin,FREQUENCY,116,120,daily,1.0
4,DOSAGE-DRUG,DOSAGE,190,197,40 units,DRUG,202,217,insulin glargine,1.0
5,DRUG-FREQUENCY,DRUG,202,217,insulin glargine,FREQUENCY,219,226,at night,1.0
6,DOSAGE-DRUG,DOSAGE,231,238,12 units,DRUG,243,256,insulin lispro,1.0
7,DRUG-FREQUENCY,DRUG,243,256,insulin lispro,FREQUENCY,258,267,with meals,1.0
8,DRUG-STRENGTH,DRUG,275,283,metformin,STRENGTH,285,291,1000 mg,1.0
9,DRUG-FREQUENCY,DRUG,275,283,metformin,FREQUENCY,293,307,two times a day,1.0


Let's try with another text

In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . 
She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . 
Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . 
Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . 
The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)
rel_df

Out[240]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DURATION-DRUG,DURATION,494,501,five-day,DRUG,513,523,amoxicillin,1.0
1,DRUG-DURATION,DRUG,684,696,dapagliflozin,DURATION,698,711,for six months,1.0
2,DRUG-ROUTE,DRUG,1947,1953,insulin,ROUTE,1955,1958,drip,1.0
3,DOSAGE-DRUG,DOSAGE,2263,2270,40 units,DRUG,2275,2290,insulin glargine,1.0
4,DRUG-FREQUENCY,DRUG,2275,2290,insulin glargine,FREQUENCY,2292,2299,at night,1.0
5,DOSAGE-DRUG,DOSAGE,2303,2310,12 units,DRUG,2315,2328,insulin lispro,1.0
6,DRUG-FREQUENCY,DRUG,2315,2328,insulin lispro,FREQUENCY,2330,2339,with meals,1.0
7,DRUG-STRENGTH,DRUG,2347,2355,metformin,STRENGTH,2357,2363,1000 mg,1.0
8,DRUG-FREQUENCY,DRUG,2347,2355,metformin,FREQUENCY,2365,2379,two times a day,1.0


#### ReDL - ADE

Now, let's create a sample pipeline with `redl_ade_biobert` model and see how it works

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

ade_ner = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ade_ner")  

ade_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ade_ner"]) \
    .setOutputCol("ade_ner_chunk")

ade_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ade_ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_re_model = RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentence"]) \
    .setOutputCol("relations")

ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ade_ner,
    ade_ner_converter,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

ade_model = ade_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][OK!]
redl_ade_biobert download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

Checking the results

In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_lmodel = LightPipeline(ade_model)
ade_results = ade_lmodel.fullAnnotate(text)

rel_df = get_relations_df (ade_results)
rel_df

Out[242]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,25,32,naproxen,ADE,137,148,tense bullae,0.9981279
1,1,DRUG,25,32,naproxen,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99783677
2,1,DRUG,87,95,oxaprozin,ADE,137,148,tense bullae,0.9978377
3,1,DRUG,87,95,oxaprozin,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99806887


### Merging Multiple RE Model Results

We can merge multiple RE model results by using `AnnotationMerger()`. <br/>
Now, we will build a pipeline consisting of `posology_re`, `re_ade_clinical` models and `AnnotationMerger()` to merge these RE models' results.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

pos_reModel = RelationExtractionModel.pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "posology_ner_chunk", "dependencies"])\
    .setOutputCol("pos_relations")\
    .setMaxSyntacticDistance(4)

ade_ner = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ade_ner")  

ade_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ade_ner"]) \
    .setOutputCol("ade_ner_chunk")

ade_reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ade_ner_chunk", "dependencies"])\
    .setOutputCol("ade_relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setCustomLabels({"1": "is_ade", "0": "not_ade"})


annotation_merger = AnnotationMerger()\
    .setInputCols("ade_relations", "pos_relations")\
    .setInputType("category")\
    .setOutputCol("all_relations")

merger_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    pos_reModel,
    ade_ner,
    ade_ner_converter,
    ade_reModel,
    annotation_merger
])

merger_model= merger_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][OK!]
re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[ | ][OK!]

In [0]:

ade_reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ade_ner_chunk", "dependencies"])\
    .setOutputCol("ade_relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setCustomLabels({"1": "is_ade", "0": "not_ade"})


annotation_merger = AnnotationMerger()\
    .setInputCols("ade_relations", "pos_relations")\
    .setInputType("category")\
    .setOutputCol("all_relations")

merger_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    pos_reModel,
    ade_ner,
    ade_ner_converter,
    ade_reModel,
    annotation_merger
])

merger_model= merger_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[ | ][OK!]

Transform and check the results

In [0]:
text = """
The patient was prescribed 1 unit of naproxen for 5 days after meals for chronic low back pain. The patient was also given 1 unit of oxaprozin daily for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.. 
"""
data = spark.createDataFrame([[text]]).toDF("text")

result = merger_model.transform(data)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| pos_tags| posology_ner| posology_ner_chunk| dependencies| pos_relations| ade_ner| ade_ner_chunk| ade_relations| all_relations|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

The patient was ...|[{document, 0, 26...|[{document, 1, 95...|[{token, 1, 3, Th...|[{word_embeddings...|[{pos, 1, 3, DD, ...|[{named_entity, 1...|[{chunk, 28, 33, ...|[{dependency, 1, ...|[{category, 28, 4...|[{named_entity, 1...|[{chunk, 38, 45, ...|[{category, 134, ...|[{category, 134, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.pos_relations.result, result.pos_relations.metadata, result.ade_relations.result, result.ade_relations.metadata, result.all_relations.result, result.all_relations.metadata)).alias("cols")) \
                  .select(
                          F.expr("cols['0']").alias("pos_relations"),\
                          F.expr("cols['1']['entity1']").alias("pos_relations_entity1"),\
                          F.expr("cols['1']['chunk1']" ).alias("pos_relations_chunk1" ),\
                          F.expr("cols['1']['entity2']").alias("pos_relations_entity2"),\
                          F.expr("cols['1']['chunk2']" ).alias("pos_relations_chunk2" ),\
                          F.expr("cols['2']").alias("ade_relations"),\
                          F.expr("cols['3']['entity1']").alias("ade_relations_entity1"),\
                          F.expr("cols['3']['chunk1']" ).alias("ade_relations_chunk1" ),\
                          F.expr("cols['3']['entity2']").alias("ade_relations_entity2"),\
                          F.expr("cols['3']['chunk2']" ).alias("ade_relations_chunk2" ),\
                          F.expr("cols['4']").alias("all_relations"),\
                          F.expr("cols['5']['entity1']").alias("all_relations_entity1"),\
                          F.expr("cols['5']['chunk1']" ).alias("all_relations_chunk1" ),\
                          F.expr("cols['5']['entity2']").alias("all_relations_entity2"),\
                          F.expr("cols['5']['chunk2']" ).alias("all_relations_chunk2" )
                          )

result_df.show()

+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
 pos_relations|pos_relations_entity1|pos_relations_chunk1|pos_relations_entity2|pos_relations_chunk2|ade_relations|ade_relations_entity1|ade_relations_chunk1|ade_relations_entity2|ade_relations_chunk2| all_relations|all_relations_entity1|all_relations_chunk1|all_relations_entity2|all_relations_chunk2|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
 DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| naproxen| is_ade| DRUG| oxaprozin| ADE| tense bullae| is_ade| DRUG| oxaprozin| ADE| tense bullae|
 DRUG-DURATION| DRUG| naproxen| DURATION| for 5 days| is_ade| DRUG| oxaprozin| ADE|cutaneous fragili...| is_ade| DRUG| oxaprozin| ADE|cutaneous fragili...|
 DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| oxaprozin| null| null| null| null| null| DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| naproxen|
DRUG-FREQUENCY| DRUG| oxaprozin| FREQUENCY| daily| null| null| null| null| null| DRUG-DURATION| DRUG| naproxen| DURATION| for 5 days|
 null| null| null| null| null| null| null| null| null| null| DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| oxaprozin|
 null| null| null| null| null| null| null| null| null| null|DRUG-FREQUENCY| DRUG| oxaprozin| FREQUENCY| daily|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+

In [0]:
lmodel = LightPipeline(merger_model)
merged_results = lmodel.fullAnnotate(text)

visualizer = RelationExtractionVisualizer()
vis= visualizer.display(merged_results[0], 'all_relations', show_relations=True, return_html=True) # default show_relations: True
displayHTML(vis)

/databricks/python/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
 relation_coordinates = np.array(relation_coordinates)

The patient was prescribed 1 unit DOSAGE of naproxen DRUG for 5 days DURATION after meals for chronic low back pain. The patient was also given 1 unit DOSAGE of oxaprozin DRUG daily FREQUENCY for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE .. <polyline fill="none" points="620.0,155.0 619.112244897959,153.7960798646822 618.2244897959182,152.59312871337622 617.3367346938776,151.39211553009378 616.4489795918367,150.19400929884657 615.5612244897959,148.99977900364644 614.6734693877551,147.81039362850512 613.7857142857144,146.62682215743445 612.8979591836735,145.45003357444605 612.0102040816327,144.28099686355176 611.1224489795918,143.12068100876337 610.234693877551,141.97005499409258 609.34693877551,140.83008780355124 608.4591836734694,139.70174842115105 607.5714285714286,138.5860058309038 606.6836734693879,137.48382901682123 605.795918367347,136.39618696291512 604.9081632653061,135.32404865319725 604.0204081632653,134.2683830716793 603.1326530612245,133.23015920237316 602.2448979591836,132.2103460292905 601.3571428571428,131.20991253644314 600.469387755102,130.22982770784282 599.5816326530612,129.2710605275013 598.6938775510205,128.33457997943034 597.8061224489795,127.42135504764173 596.9183673469388,126.53235471614718 596.030612244898,125.6685479689585 595.1428571428572,124.83090379008749 594.2551020408164,124.02039116354581 593.3673469387755,123.2379790733453 592.4795918367347,122.48463650349771 591.5918367346939,121.76133243801479 590.704081632653,121.06903586090831 589.8163265306123,120.40871575619002 588.9285714285714,119.78134110787173 588.0408163265306,119.18788089996517 587.1530612244899,118.62930411648207 586.265306122449,118.10657974143427 585.3775510204082,117.62067675883348 584.4897959183673,117.17256415269148 583.6020408163265,116.76321090702004 582.7142857142858,116.39358600583091 581.8265306122448,116.06465843313586 580.9387755102041,115.77739717294665 580.0510204081633,115.53277120927505 579.1632653061224,115.33174952613282 578.2755102040817,115.17530110753174 577.3877551020408,115.06439493748353 576.5,115.0 576.5,115.0 575.612244897959,114.9827622844223 574.7244897959183,115.01203580140927 573.8367346938776,115.08685156694916 572.9489795918367,115.20624059703015 572.0612244897959,115.36923390764053 571.173469387755,115.57486251476851 570.2857142857144,115.82215743440234 569.3979591836735,116.11014968253025 568.5102040816327,116.43787027514047 567.6224489795918,116.80435022822121 566.7346938775509,117.20862055776078 565.8469387755101,117.64971227974738 564.9591836734694,118.12665641016923 564.0714285714286,118.63848396501459 563.1836734693879,119.18422596027166 562.295918367347,119.7629134119287 561.4081632653061,120.37357733597398 560.5204081632653,121.01524874839565 559.6326530612245,121.68695866518203 558.7448979591835,122.3877381023213 557.8571428571428,123.11661807580174 556.969387755102,123.87262960161156 556.0816326530612,124.654803695739 555.1938775510205,125.46217137417233 554.3061224489796,126.29376365289971 553.4183673469388,127.14861154790944 552.530612244898,128.02574607518974 551.6428571428572,128.92419825072886 550.7551020408164,129.842999090515 549.8673469387755,130.7811796105364 548.9795918367347,131.73777082678137 548.0918367346939,132.71180375523804 547.204081632653,133.7023094118947 546.3163265306123,134.7083188127396 545.4285714285714,135.72886297376095 544.5408163265306,136.76297291094698 543.6530612244898,137.80967964028594 542.7653061224489,138.86801417776604 541.8775510204082,139.9370075393756 540.9897959183675,141.01569074110276 540.1020408163265,142.1030947989358 539.2142857142858,143.19825072886297 538.3265306122448,144.30018954687245 537.4387755102041,145.40794226895255 536.5510204081633,146.52053991109145 535.6632653061224,147.63701348927742 534.7755102040817,148.75639401949866 533.8877551020408,149.87771251774345 533.0,151.0" stroke="slategray" stroke-width="1" /> DRUG-FREQUENCY <polyline fill="none" points="4

## Zero-shot Clinical Relation Extraction Model

Zero-shot Relation Extraction to extract relations between clinical entities with no training dataset. zero-shot relation extraction model leverages `BertForSequenceClassificaiton` to return, based on a predefined set of relation candidates (including no-relation / O), which one has the higher probability to be linking two entities.

The dataset will be a csv which contains the following columns: `sentence`, `chunk1`, `firstCharEnt1`, `lastCharEnt1`, `label1`, `chunk2`, `firstCharEnt2`, `lastCharEnt2`, `label2`, `rel`.

For more detailed explanation of the usage of this model, please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.3.ZeroShot_Clinical_Relation_Extraction.ipynb)

Now, we will use the pretrained `re_zeroshot_biobert model`, this model available in Models Hub under the Relation Extraction category.

This is our Zero-shot RE pipeline;

In [0]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

zero_tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

zero_sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

words_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel() \
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos_tags")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en") \
    .setInputCols(["document", "pos_tags", "token"]) \
    .setOutputCol("dependencies")

re_ner_chunk_filter = RENerChunksFilter() \
    .setRelationPairs(["problem-test","problem-treatment"]) \
    .setMaxSyntacticDistance(4)\
    .setDocLevelRelations(False)\
    .setInputCols(["clinical_ner_chunk", "dependencies"]) \
    .setOutputCol("re_ner_chunks")

re_model = ZeroShotRelationExtractionModel \
    .pretrained("re_zeroshot_biobert", "en", "clinical/models")\
    .setRelationalCategories({
        "CURE": ["{TREATMENT} cures {PROBLEM}."],
        "IMPROVE": ["{TREATMENT} improves {PROBLEM}.", "{TREATMENT} cures {PROBLEM}."],
        "REVEAL": ["{TEST} reveals {PROBLEM}."]})\
    .setMultiLabel(False)\
    .setInputCols(["re_ner_chunks", "sentence"]) \
    .setOutputCol("relations")

zeroPipeline = sparknlp.base.Pipeline() \
    .setStages([documenter, 
                zero_tokenizer, 
                zero_sentencer, 
                words_embedder, 
                pos_tagger, 
                clinical_ner, 
                clinical_ner_converter,
                dependency_parser, 
                re_ner_chunk_filter, 
                re_model])
    
zero_model = zeroPipeline.fit(spark.createDataFrame([['']]).toDF('text'))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
re_zeroshot_biobert download started this may take some time.
[ | ][OK!]

Let's transform and see the results.

In [0]:
results = zero_model.transform(spark.createDataFrame([["Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer."]]).toDF("text"))

results\
    .selectExpr("explode(relations) as relation")\
    .show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
relation |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{category, 534, 613, REVEAL, {entity1_begin -> 48, relation -> REVEAL, hypothesis -> An MRI test reveals cancer., confidence -> 0.9760039, nli_prediction -> entail, entity1 -> TEST, syntactic_distance -> 4, chunk2 -> cancer, entity2_end -> 85, entity1_end -> 58, entity2_begin -> 80, entity2 -> PROBLEM, chunk1 -> An MRI test, sentence -> 1}, []} |
{category, 267, 357, IMPROVE, {entity1_begin -> 0, relation -> IMPROVE, hypothesis -> Paracetamol improves sickness., confidence -> 0.98819494, nli_prediction -> entail, entity1 -> TREATMENT, syntactic_distance -> 3, chunk2 -> sickness, entity2_end -> 45, entity1_end -> 10, entity2_begin -> 38, entity2 -> PROBLEM, chunk1 -> Paracetamol, sentence -> 0}, []}|
{category, 0, 90, IMPROVE, {entity1_begin -> 0, relation -> IMPROVE, hypothesis -> Paracetamol improves headache., confidence -> 0.9929625, nli_prediction -> entail, entity1 -> TREATMENT, syntactic_distance -> 2, chunk2 -> headache, entity2_end -> 33, entity1_end -> 10, entity2_begin -> 26, entity2 -> PROBLEM, chunk1 -> Paracetamol, sentence -> 0}, []} |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

## Train a Custom Relation Extraction Model

Now, we will train our custom RE model with sample data, let's see how it works

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

dbutils.fs.cp("file:/databricks/driver/i2b2_clinical_rel_dataset.csv", "dbfs:/")

Out[10]: True

In [0]:
data = spark.read.option("header","true").format("csv").load("/i2b2_clinical_rel_dataset.csv")
data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
 sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2| chunk1| chunk2| label1| label2| rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
VITAL SIGNS - Tem...| 49| 75| 64| 84| respiratory rate| saturation| test| test| O| test|
No lotions , crea...| 3| 34| 9| 42| lotions| incisions|treatment| problem|TrNAP| test|
Because of expect...| 11| 58| 54| 68|expected long ter...| a picc line|treatment|treatment| O| train|
She states this l...| 16| 82| 31| 92| light-headedness| diaphoresis| problem| problem| PIP| train|
Initial electroca...| 196| 145| 239| 176|an inferior and r...|1-mm st depressio...| problem| problem| PIP| test|
Abdominal ultraso...| 1| 54| 20| 71|abdominal ultrasound| gallbladder sludge| test| problem| TeRP| test|
At that time he w...| 99| 139| 133| 173|ir placement of a...|his abdominopelvi...|treatment| problem| TrAP| test|
He was transferre...| 143| 195| 154| 213| reintubation| worsening pneumonia|treatment| problem| TrAP| test|
5) Anemia - previ...| 12| 35| 23| 47| previous w/u| elev ferritin| test| problem| TeRP| test|
The patient &apos...| 161| 149| 179| 156| po pain medications| his pain|treatment| problem| TrAP| train|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
only showing top 10 rows

In [0]:
data.groupby('dataset').count().show()

+-------+-----+
dataset|count|
+-------+-----+
 train| 350|
 test| 650|
+-------+-----+

In [0]:
data.groupby('rel').count().show()

+-----+-----+
 rel|count|
+-----+-----+
 TrIP| 14|
 TrAP| 164|
 TeCP| 26|
 O| 414|
TrNAP| 14|
 TrCP| 28|
 PIP| 153|
 TrWP| 11|
 TeRP| 176|
+-----+-----+

In [0]:
from sparknlp_jsl.training import REDatasetHelper

# map entity columns to dataset columns
column_map = {
    "begin1": "firstCharEnt1",
    "end1": "lastCharEnt1",
    "begin2": "firstCharEnt2",
    "end2": "lastCharEnt2",
    "chunk1": "chunk1",
    "chunk2": "chunk2",
    "label1": "label1",
    "label2": "label2"
}

# apply preprocess function to dataframe
data = REDatasetHelper(data).create_annotation_column(
    column_map,
    ner_column_name="train_ner_chunks" # optional, default train_ner_chunks
)

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:
data.show(10)

+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
lastCharEnt1| train_ner_chunks| chunk2|firstCharEnt2|dataset| label2|lastCharEnt2| label1| chunk1| sentence| rel|firstCharEnt1|
+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
 64|[{chunk, 49, 64, ...| saturation| 75| test| test| 84| test| respiratory rate|VITAL SIGNS - Tem...| O| 49|
 9|[{chunk, 3, 9, lo...| incisions| 34| test| problem| 42|treatment| lotions|No lotions , crea...|TrNAP| 3|
 54|[{chunk, 11, 54, ...| a picc line| 58| train|treatment| 68|treatment|expected long ter...|Because of expect...| O| 11|
 31|[{chunk, 16, 31, ...| diaphoresis| 82| train| problem| 92| problem| light-headedness|She states this l...| PIP| 16|
 239|[{chunk, 196, 239...|1-mm st depressio...| 145| test| problem| 176| problem|an inferior and r...|Initial electroca...| PIP| 196|
 20|[{chunk, 1, 20, a...| gallbladder sludge| 54| test| problem| 71| test|abdominal ultrasound|Abdominal ultraso...| TeRP| 1|
 133|[{chunk, 99, 133,...|his abdominopelvi...| 139| test| problem| 173|treatment|ir placement of a...|At that time he w...| TrAP| 99|
 154|[{chunk, 143, 154...| worsening pneumonia| 195| test| problem| 213|treatment| reintubation|He was transferre...| TrAP| 143|
 23|[{chunk, 12, 23, ...| elev ferritin| 35| test| problem| 47| test| previous w/u|5) Anemia - previ...| TeRP| 12|
 179|[{chunk, 161, 179...| his pain| 149| train| problem| 156|treatment| po pain medications|The patient &apos...| TrAP| 161|
+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
only showing top 10 rows

We have prepared our train and test data in desired format for traininig.

## Create graph 

We will use `TFGraphBuilder` annotator which can be used to create graphs automatically in the model training pipeline. 

`TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the approach.

You can also create a custom graph by using `tf_graph` module in Spark NLP for Healthcare.

In [0]:
#!pip install -q tensorflow==2.7.0 tensorflow-addons

In [0]:
'''
# custom graph

from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("relation_extraction")

tf_graph.build("relation_extraction", 
             build_params={"input_dim": 10000, 
                          "output_dim": 10, 
                          'batch_norm':1, 
                          "hidden_layers": [300, 200], 
                          "hidden_act": "relu", 
                          'hidden_act_l2':1}, 
                          'model_location'=".", 
                          'model_filename'="re_in6000D_out10.pb")
'''

'''
# ready to use graph
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb
'''

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [0]:
graph_folder= "./tf_graphs"

re_graph_builder = TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

Our graph is ready to use, let's train a model with `RelationExtractionApproach()`

In [0]:
train_documenter = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

train_tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

train_embedder = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reApproach = RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile("/dbfs/re_graphs/rel_e.in1200.out20.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('dbfs:/re_logs')

finisher = Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = Pipeline(stages=[
    train_documenter, 
    train_tokenizer, 
    train_embedder, 
    pos_tagger, 
    dependency_parser, 
    re_graph_builder,
    reApproach, 
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]

In [0]:
%%time

rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 3.16 s, sys: 428 ms, total: 3.59 s
Wall time: 25.5 s

In [0]:
rel_model.stages

Out[17]: [DocumentAssembler_60a193306ec8,
 REGEX_TOKENIZER_2770dc7a08c7,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 TFGraphBuilderModel_85a8cb47fb7f,
 RelationExtractionModel_4bd1c2c12c9b,
 Finisher_fbf3add05173]

In [0]:
rel_model.stages[-2]

Out[18]: RelationExtractionModel_4bd1c2c12c9b

In [0]:
#save the model
rel_model.stages[-2].write().overwrite().save('dbfs:/databricks/driver/models/custom_RE_model')

### Evaluating The Performance of the Model

We will create a pipeline and use our trained RE model by calling with `.load` method. Then we will get predictions by transforming our test set on this pipeline.

In [0]:
customReModel = RelationExtractionModel.load("dbfs:/databricks/driver/models/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline = Pipeline(stages=[
    train_documenter, 
    train_tokenizer, 
    train_embedder, 
    pos_tagger, 
    dependency_parser, 
    customReModel])

test_results = test_pipeline.fit(test_data).transform(test_data)

In [0]:
test_results.show(5)

+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
lastCharEnt1| train_ner_chunks| chunk2|firstCharEnt2|dataset| label2|lastCharEnt2| label1| chunk1| sentence| rel|firstCharEnt1| sentences| tokens| embeddings| pos_tags| dependencies| relations_pred|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 64|[{chunk, 49, 64, ...| saturation| 75| test| test| 84| test| respiratory rate|VITAL SIGNS - Tem...| O| 49|[{document, 0, 10...|[{token, 0, 4, VI...|[{word_embeddings...|[{pos, 0, 4, NN, ...|[{dependency, 0, ...|[{category, 49, 8...|
 9|[{chunk, 3, 9, lo...| incisions| 34| test|problem| 42|treatment| lotions|No lotions , crea...|TrNAP| 3|[{document, 0, 44...|[{token, 0, 1, No...|[{word_embeddings...|[{pos, 0, 1, DD, ...|[{dependency, 0, ...|[{category, 3, 42...|
 239|[{chunk, 196, 239...|1-mm st depressio...| 145| test|problem| 176| problem|an inferior and r...|Initial electroca...| PIP| 196|[{document, 0, 24...|[{token, 0, 6, In...|[{word_embeddings...|[{pos, 0, 6, JJ, ...|[{dependency, 0, ...| []|
 20|[{chunk, 1, 20, a...| gallbladder sludge| 54| test|problem| 71| test|abdominal ultrasound|Abdominal ultraso...| TeRP| 1|[{document, 0, 95...|[{token, 0, 8, Ab...|[{word_embeddings...|[{pos, 0, 8, JJ, ...|[{dependency, 0, ...|[{category, 1, 71...|
 133|[{chunk, 99, 133,...|his abdominopelvi...| 139| test|problem| 173|treatment|ir placement of a...|At that time he w...| TrAP| 99|[{document, 0, 17...|[{token, 0, 1, At...|[{word_embeddings...|[{pos, 0, 1, II, ...|[{dependency, 0, ...|[{category, 99, 1...|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

When you check the `relations_pred` column, you can see that some of the relations between the chunk pairs were not detected this time. 

Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [0]:
pd_test_results = test_results.select('rel', 'relations_pred.result').toPandas()
pd_test_results.head()

Out[22]:

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the `result` column and fill null values as `O` label.

In [0]:
pd_test_results = pd_test_results.explode("result").fillna("O")
pd_test_results.result.value_counts()

Out[23]: O 354
TrAP 97
PIP 97
TeRP 96
TrNAP 2
TeCP 2
TrCP 1
TrIP 1
Name: result, dtype: int64

In [0]:
pd_test_results.rel.value_counts()

Out[24]: O 274
TeRP 116
TrAP 108
PIP 88
TrCP 20
TeCP 15
TrIP 11
TrNAP 9
TrWP 9
Name: rel, dtype: int64

In [0]:
from sklearn.metrics import classification_report
print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 O 0.66 0.85 0.75 274
 PIP 0.57 0.62 0.59 88
 TeCP 0.00 0.00 0.00 15
 TeRP 0.79 0.66 0.72 116
 TrAP 0.59 0.53 0.56 108
 TrCP 0.00 0.00 0.00 20
 TrIP 1.00 0.09 0.17 11
 TrNAP 0.00 0.00 0.00 9
 TrWP 0.00 0.00 0.00 9

 accuracy 0.65 650
 macro avg 0.40 0.31 0.31 650
weighted avg 0.61 0.65 0.62 650

# 6.Clinical Entity Resolvers

## Sentence Entity Resolver Models

**Entity Resolvers in SparkNLP**
- RxNorm
- ICD10-CM
- CPT
- SNOMED
- LOINC
- UMLS
- HPO
- HCPCS
- ICD-O
- MeSH
- NDC
- RX-CUI
- ATC
- CVX

Sparknlp has more than 50 pretrained **sentence entity resolver models**
<center><b>MODEL LIST</b>

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [sbertresolve_icd10cm_slim_billable_hcc_med](https://nlp.johnsnowlabs.com/2021/05/25/sbertresolve_icd10cm_slim_billable_hcc_med_en.html)  | 2| [sbertresolve_jsl_rxnorm_augmented_med](https://nlp.johnsnowlabs.com/2021/12/28/sbertresolve_jsl_rxnorm_augmented_med_en.html)  | 3| [sbertresolve_ner_model_finder](https://nlp.johnsnowlabs.com/2022/09/05/sbertresolve_ner_model_finder_en.html)  | 4| [sbertresolve_rxnorm_disposition](https://nlp.johnsnowlabs.com/2021/08/28/sbertresolve_rxnorm_disposition_en.html)  |
| 5| [sbertresolve_snomed_bodyStructure_med](https://nlp.johnsnowlabs.com/2021/06/15/sbertresolve_snomed_bodyStructure_med_en.html)  | 6| [sbertresolve_snomed_conditions](https://nlp.johnsnowlabs.com/2021/08/28/sbertresolve_snomed_conditions_en.html)  | 7| [sbiobert_jsl_rxnorm_cased](https://nlp.johnsnowlabs.com/2021/12/23/sbiobert_jsl_rxnorm_cased_en.html)  | 8| [sbiobertresolve_HPO](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_HPO_en.html)  |
| 9| [sbiobertresolve_atc](https://nlp.johnsnowlabs.com/2022/03/01/sbiobertresolve_atc_en_3_0.html)  | 10| [sbiobertresolve_clinical_abbreviation_acronym](https://nlp.johnsnowlabs.com/2022/01/03/sbiobertresolve_clinical_abbreviation_acronym_en.html)  | 11| [sbiobertresolve_clinical_snomed_procedures_measurements](https://nlp.johnsnowlabs.com/2021/11/15/sbiobertresolve_clinical_snomed_procedures_measurements_en.html)  | 12| [sbiobertresolve_cpt](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_cpt_en.html)  |
| 13| [sbiobertresolve_cpt_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_cpt_augmented_en.html)  | 14| [sbiobertresolve_cpt_procedures_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_cpt_procedures_augmented_en.html)  | 15| [sbiobertresolve_cpt_procedures_measurements_augmented](https://nlp.johnsnowlabs.com/2022/05/10/sbiobertresolve_cpt_procedures_measurements_augmented_en_3_0.html)  | 16| [sbiobertresolve_cvx](https://nlp.johnsnowlabs.com/2022/10/12/sbiobertresolve_cvx_en.html)  |
| 17| [sbiobertresolve_hcc_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_hcc_augmented_en.html)  | 18| [sbiobertresolve_hcpcs](https://nlp.johnsnowlabs.com/2021/09/29/sbiobertresolve_hcpcs_en.html)  | 19| [sbiobertresolve_icd10cm](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icd10cm_en.html)  | 20| [sbiobertresolve_icd10cm_augmented](https://nlp.johnsnowlabs.com/2021/10/31/sbiobertresolve_icd10cm_augmented_en.html)  |
| 21| [sbiobertresolve_icd10cm_augmented_billable_hcc](https://nlp.johnsnowlabs.com/2021/11/01/sbiobertresolve_icd10cm_augmented_billable_hcc_en.html)  | 22| [sbiobertresolve_icd10cm_generalised](https://nlp.johnsnowlabs.com/2021/09/29/sbiobertresolve_icd10cm_generalised_en.html)  | 23| [sbiobertresolve_icd10cm_slim_billable_hcc](https://nlp.johnsnowlabs.com/2021/05/25/sbiobertresolve_icd10cm_slim_billable_hcc_en.html)  | 24| [sbiobertresolve_icd10cm_slim_normalized](https://nlp.johnsnowlabs.com/2021/05/17/sbiobertresolve_icd10cm_slim_normalized_en.html)  |
| 25| [sbiobertresolve_icd10pcs](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icd10pcs_en.html)  | 26| [sbiobertresolve_icd10pcs_augmented](https://nlp.johnsnowlabs.com/2022/10/28/sbiobertresolve_icd10pcs_augmented_en.html)  | 27| [sbiobertresolve_icd9](https://nlp.johnsnowlabs.com/2022/09/30/sbiobertresolve_icd9_en.html)  | 28| [sbiobertresolve_icdo](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icdo_en.html)  |
| 29| [sbiobertresolve_icdo_augmented](https://nlp.johnsnowlabs.com/2021/06/22/sbiobertresolve_icdo_augmented_en.html)  | 30| [sbiobertresolve_icdo_base](https://nlp.johnsnowlabs.com/2021/07/02/sbiobertresolve_icdo_base_en.html)  | 31| [sbiobertresolve_jsl_rxnorm_augmented](https://nlp.johnsnowlabs.com/2021/12/27/sbiobertresolve_jsl_rxnorm_augmented_en.html)  | 32| [sbiobertresolve_loinc](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_loinc_en.html)  |
| 33| [sbiobertresolve_loinc_augmented](https://nlp.johnsnowlabs.com/2021/11/23/sbiobertresolve_loinc_augmented_en.html)  | 34| [sbiobertresolve_loinc_cased](https://nlp.johnsnowlabs.com/2021/12/24/sbiobertresolve_loinc_cased_en.html)  | 35| [sbiobertresolve_mesh](https://nlp.johnsnowlabs.com/2021/11/14/sbiobertresolve_mesh_en.html)  | 36| [sbiobertresolve_ndc](https://nlp.johnsnowlabs.com/2021/11/27/sbiobertresolve_ndc_en.html)  |
| 37| [sbiobertresolve_rxcui](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_rxcui_en.html)  | 38| [sbiobertresolve_rxnorm](https://nlp.johnsnowlabs.com/2021/10/10/sbiobertresolve_rxnorm_en.html)  | 39| [sbiobertresolve_rxnorm_action_treatment](https://nlp.johnsnowlabs.com/2022/04/25/sbiobertresolve_rxnorm_action_treatment_en_2_4.html)  | 40| [sbiobertresolve_rxnorm_augmented](https://nlp.johnsnowlabs.com/2022/01/03/sbiobertresolve_rxnorm_augmented_en.html)  |
| 41| [sbiobertresolve_rxnorm_augmented_cased](https://nlp.johnsnowlabs.com/2021/12/28/sbiobertresolve_rxnorm_augmented_cased_en.html)  | 42| [sbiobertresolve_rxnorm_augmented_re](https://nlp.johnsnowlabs.com/2022/02/09/sbiobertresolve_rxnorm_augmented_re_en.html)  | 43| [sbiobertresolve_rxnorm_disposition](https://nlp.johnsnowlabs.com/2021/08/12/sbiobertresolve_rxnorm_disposition_en.html)  | 44| [sbiobertresolve_rxnorm_ndc](https://nlp.johnsnowlabs.com/2021/10/05/sbiobertresolve_rxnorm_ndc_en.html)  |
| 45| [sbiobertresolve_snomed_auxConcepts](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_auxConcepts_en.html)  | 46| [sbiobertresolve_snomed_auxConcepts_int](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_auxConcepts_int_en.html)  | 47| [sbiobertresolve_snomed_bodyStructure](https://nlp.johnsnowlabs.com/2021/06/15/sbiobertresolve_snomed_bodyStructure_en.html)  | 48| [sbiobertresolve_snomed_drug](https://nlp.johnsnowlabs.com/2022/01/18/sbiobertresolve_snomed_drug_en.html)  |
| 49| [sbiobertresolve_snomed_findings](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_findings_en.html)  | 50| [sbiobertresolve_snomed_findings_aux_concepts](https://nlp.johnsnowlabs.com/2021/07/14/sbiobertresolve_snomed_findings_aux_concepts_en.html)  | 51| [sbiobertresolve_snomed_findings_int](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_findings_int_en.html)  | 52| [sbiobertresolve_snomed_procedures_measurements](https://nlp.johnsnowlabs.com/2021/11/11/sbiobertresolve_snomed_procedures_measurements_en.html)  |
| 53| [sbiobertresolve_umls_clinical_drugs](https://nlp.johnsnowlabs.com/2021/10/11/sbiobertresolve_umls_clinical_drugs_en.html)  | 54| [sbiobertresolve_umls_disease_syndrome](https://nlp.johnsnowlabs.com/2021/10/11/sbiobertresolve_umls_disease_syndrome_en.html)  | 55| [sbiobertresolve_umls_drug_substance](https://nlp.johnsnowlabs.com/2021/12/06/sbiobertresolve_umls_drug_substance_en.html)  | 56| [sbiobertresolve_umls_findings](https://nlp.johnsnowlabs.com/2021/10/03/sbiobertresolve_umls_findings_en.html)  |
| 57| [sbiobertresolve_umls_major_concepts](https://nlp.johnsnowlabs.com/2021/10/03/sbiobertresolve_umls_major_concepts_en.html)  | 58| [sbluebertresolve_loinc](https://nlp.johnsnowlabs.com/2021/04/29/sbluebertresolve_loinc_en.html)  | 59| [sbluebertresolve_loinc_uncased](https://nlp.johnsnowlabs.com/2022/01/18/sbluebertresolve_loinc_uncased_en.html)  | 60| [sbluebertresolve_rxnorm_augmented_uncased](https://nlp.johnsnowlabs.com/2021/12/28/sbluebertresolve_rxnorm_augmented_uncased_en.html)  |


<br>
<br>

<b>PRETRAINED PIPELINE LIST</b>

|index|model|
|-----:|:-----|
| 1| [cvx_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/10/12/cvx_resolver_pipeline_en.html)  |
| 2| [icd10cm_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/11/02/icd10cm_resolver_pipeline_en.html)  |
| 3| [icd9_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/30/icd9_resolver_pipeline_en.html)  |
| 4| [medication_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_pipeline_en.html)  |
| 5| [medication_resolver_transform_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_transform_pipeline_en.html)  |
| 6| [umls_clinical_findings_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_clinical_findings_resolver_pipeline_en_3_0.html)  |
| 7| [umls_disease_syndrome_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_disease_syndrome_resolver_pipeline_en_3_0.html)  |
| 8| [umls_drug_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_drug_resolver_pipeline_en_3_0.html)  |
| 9| [umls_drug_substance_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_drug_substance_resolver_pipeline_en_3_0.html)  |
| 10| [umls_major_concepts_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_major_concepts_resolver_pipeline_en_3_0.html)  |





</center>

You can find all resolver models and more on [Models Hub](https://nlp.johnsnowlabs.com/models?task=Entity+Resolution&edition=Spark+NLP+for+Healthcare) page.

Our pipelines provide more metadata about the matching process other than providing the code in the "result" field;

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- all_k_resolutions -> All codes descriptions
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId

We create a pipeline that from each of these problems, will try to assign a resolution on the content, the sentence embeddings and some pretrained models for resolver annotation.

The architecture of this new pipeline will be as follows:

- DocumentAssembler (text -> document)

- SentenceDetector (document -> sentence)

- Tokenizer (sentence -> token)

- WordEmbeddingsModel ([sentence, token] -> embeddings)

- MedicalNerModel ([sentence, token, embeddings] -> ner)

- NerConverter (["sentence, token, ner] -> ner_chunk

- Chunk2Doc (ner_chunk) -> ner_chunk_doc

- BertSentenceEmbeddings (ner_chunk_doc) -> sbert_embeddings

- SentenceEntityResolverModel ([ner_chunk, sbert_embeddings] -> resolution)

So from a text we end having a list of Named Entities (ner_chunk) and their resolutions.

Let's see how we build a resolver pipeline

Firstly, we will write some **generic functions** for getting the codes and relation pairs

In [0]:
# returns LP resolution results

import pandas as pd
pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', col_name="ner_chunk", hcc=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0][col_name], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes, 'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_code'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])

    df = df.drop(['all_k_aux_labels'], axis=1)
    
    return df



In [0]:
# returns billable informations in distinct columns

def extract_billable(bil):
  
    billable = []
    status = []
    code = []
 
    for b in bil:
        billable.append(b.split("||")[0])
        status.append(b.split("||")[1])
        code.append(b.split("||")[2])

    return (billable, status, code)

In [0]:
# returns relation pair dataframe

def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for rel in results[0][col]:
        rel_pairs.append(( 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['entity1','entity1_begin','entity1_end','chunk1',
                                              'entity2','entity2_end','entity2_end','chunk2', 
                                              'confidence'])
    
    # limit df columns to get entity and chunks with results only
    rel_df = rel_df.iloc[:,[0,3,4,7,8]]
    
    return rel_df

### RxNorm Resolver

- RxNorm is a second vocabulary for prescription drugs. RxNorm provides a set of codes for clinical drugs, which are the combination of active ingredients, dose form, and strength of a drug. For example, the RxNorm code for ciprofloxacin 500 mg 24-hour extended-release tablet (the generic name for Cipro XR 500 mg) is RX10359383, regardless of brand or packaging.

- The goal of RxNorm is to allow computer systems to communicate drug-related information efficiently and unambiguously. Produced by the National Library of Medicine (NLM), RxNorm is available for distribution in both Metathesaurus Relation (MR) and Rich Release Format (RRF) tables. Currently there are no RxNorm names available for drugs with more than four active ingredients, those that are sold over the counter (OTC) or those that are international, due to the lack of appropriate information available about such drugs. 


<img src="https://www.nlm.nih.gov/research/umls/rxnorm/RxNorm_Drug_Relationships.png" width="750" alt="RxNorm Overview">

**Pretrained Models**

- `sbiobertresolve_rxnorm`
- `demo_sbiobertresolve_rxnorm`
- `sbiobertresolve_rxnorm_dispo`
- `sbiobertresolve_rxnorm_disposition`
- `sbertresolve_rxnorm_disposition`
- `sbiobertresolve_rxnorm_ndc`
- `sbiobertresolve_rxnorm_augmented`
- `sbiobertresolve_rxnorm_augmented_re`

Now, we will create a pipeline with `sbiobertresolve_rxnorm_augmented` pretrained model and see the results.

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sentence_embeddings")\
      .setCaseSensitive(False)
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          sbert_embedder,
          rxnorm_resolver])

rxnorm_lp = LightPipeline(rxnorm_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

Checking the results with the help of our helper functions

In [0]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, col_name = 'ner_chunk', vocab='rxnorm_code')

CPU times: user 37.8 ms, sys: 29.3 ms, total: 67 ms
Wall time: 9.3 s
Out[6]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,2200518,"[2200518, 861024, 141916, 334738, 332848, 104494, 199955, 333262, 439563, 429178, 103910, 429408, 450523, 1744000, 401938, 402346, 1726496, 485246, 198758, 316350, 213092]","[metformin hydrochloride 100 MG/ML Extended Release Suspension, metformin hydrochloride 100 MG/ML, fenofibrate 100 MG Oral Capsule, fenofibrate 100 MG, ciprofibrate 100 MG, ciprofibrate 100 MG Oral Tablet, rutin 100 MG Oral Tablet, rutin 100 MG, fendiline 100 MG, fendiline 100 MG Oral Tablet, carbamazepine 100 MG Oral Tablet [Epimaz], perazine 100 MG Oral Tablet, perazine 100 MG, emtricitabine 100 MG, miglustat 100 MG Oral Capsule, miglustat 100 MG, azacitidine 100 MG, azacitidine 100 MG Injection, niacin 100 MG Oral Capsule, niacin 100 MG, dolasetron 100 MG Oral Tablet [Anzemet]]","[0.0658, 0.0658, 0.0684, 0.0684, 0.0697, 0.0697, 0.0765, 0.0765, 0.0775, 0.0775, 0.0780, 0.0804, 0.0804, 0.0818, 0.0811, 0.0811, 0.0812, 0.0812, 0.0811, 0.0811, 0.0836]"


### RxNorm with DrugNormalizer

We have `DrugNormalizer` annotator that transforms text to the format used in the RxNorm and SNOMED standards.

It has **`policy`** parameter to remove patterns from text. Valid policy values are:
- "all",
- "abbreviations",
- "dosages" <br/>

Default is "all". "abbreviation" policy used to expend common drugs abbreviations, "dosages" policy used to convert drugs dosages and values to the standard form (see examples bellow).

For more details about the usage of `DrugNormalizer` annotator, please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/23.Drug_Normalizer.ipynb)

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk_v0")

drug_normalizer = DrugNormalizer() \
    .setInputCols("ner_chunk_v0") \
    .setOutputCol("ner_chunk") \
    .setPolicy('all')

rxnorm_pipelineModel2 = PipelineModel(
    stages = [
        documentAssembler,
        drug_normalizer,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp2 = LightPipeline(rxnorm_pipelineModel2)

In [0]:
text = 'aspirin 10 meq/ 5 ml oral sol'

%time get_codes (rxnorm_lp2, text, col_name = 'ner_chunk', vocab='rxnorm_code')

CPU times: user 13.9 ms, sys: 6.8 ms, total: 20.7 ms
Wall time: 1.5 s
Out[8]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,aspirin 2 meq/ml oral solution,0,29,688214,"[688214, 342917, 688213, 755937, 756028, 979446, 755332, 545588, 1091135, 246229, 104692, 755920, 261342, 756064, 756078, 2381144, 1115998, 1092456, 248847, 1094188, 250944, 108870, 995243, 421803]","[aspirin 2.5 MG/ML Oral Solution, aspirin 2.2 MG/ML, aspirin 2.5 MG/ML, periciazine 2 MG/ML Oral Solution, fenspiride 2 MG/ML Oral Solution, metaproterenol sulfate 2 MG/ML Oral Solution [Alupent], dicyclomine hydrochloride 2 MG/ML Oral Solution [Merbentyl], racepinephrine 2 MG/ML Oral Solution [Orostat], methylphenidate hydrochloride 2 MG/ML Oral Solution [Methylin], thioridazine 2 MG/ML Oral Solution, temazepam 2 MG/ML Oral Solution [Euhypnos], butamirate 2 MG/ML Oral Solution, citalopram 2 MG/ML Oral Solution [Celexa], oxeladin 2 MG/ML Oral Solution, pipazethate 2 MG/ML Oral Solution, fenfluramine 2.2 MG/ML Oral Solution [Fintepla], ephedrine sulfate 2.2 MG/ML Oral Solution, diphenhydramine hydrochloride 2.5 MG/ML Oral Solution [Diphenhist], homatropine 2 MG/ML Oral Solution, diphenhydramine hydrochloride 2.5 MG/ML Oral Solution [Diphedryl], piracetam 2 MG/ML Oral Solution, theophylline 2 MG/ML Oral Solution, hydroxyzine hydrochloride 2 MG/ML Oral Solution [Atarax], codeine 2 MG/ML Oral Suspension]","[0.0347, 0.0566, 0.0748, 0.0947, 0.1040, 0.1009, 0.1075, 0.1081, 0.1097, 0.1155, 0.1137, 0.1149, 0.1123, 0.1154, 0.1178, 0.1175, 0.1196, 0.1212, 0.1211, 0.1242, 0.1196, 0.1236, 0.1214, 0.1175]"


### Drug Spell Checker

We have pretrained spell checker model for drugs. Let's see how it works.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

spell = NorvigSweetingModel.pretrained("spellcheck_drug_norvig", "en", "clinical/models")\
    .setInputCols("token")\
    .setOutputCol("corrected_token")\

spell_pipeline = Pipeline(stages = [documentAssembler,
                              tokenizer, 
                              spell])

spell_model = spell_pipeline.fit(spark.createDataFrame([['']]).toDF('text')) 

spell_lp = LightPipeline(spell_model)
corrected = spell_lp.annotate("You have to take Neutrcare and colfosinum and a bit of Fluorometholne & Ribotril")

spellcheck_drug_norvig download started this may take some time.
Approximate size to download 3.7 MB
[ | ][ / ][ — ][OK!]

Checking the results

In [0]:
print(" ".join(corrected['token']))
print(" ".join(corrected['corrected_token']))

You have to take Neutrcare and colfosinum and a bit of Fluorometholne & Ribotril
You have to take Neutracare and colforsinum and a bit of Fluorometholone & Rivotril

### ICD-10CM Resolver

**ICD10 BACKGROUND INFO**

**ICD-10-CM vs. ICD-10-PCS**

- With the transition to ICD-10, in the United States, ICD-9 codes are segmented into ICD-10-CM and ICD-10-PCS codes. **The "CM" in ICD-10-CM codes stands for clinical modification**; ICD-10-CM codes were developed by the Centers for Disease Control and Prevention in conjunction with the National Center for Health Statistics (NCHS), for outpatient medical coding and reporting in the United States, as published by the World Health Organization (WHO).

- **The "PCS" in ICD-10-PCS codes stands for the procedural classification system**. ICD-10-PCS is a completely separate medical coding system from ICD-10-CM, containing an additional 87,000 codes for use ONLY in United States inpatient, hospital settings. The procedure classification system (ICD-10-PCS) was developed by the Centers for Medicare and Medicaid Services (CMS) in conjunction with 3M Health Information Management (HIM).

- ICD-10-CM codes add increased specificity to their ICD-9 predecessors, growing to five times the number of codes as the present system; a total of 68,000 clinical modification diagnosis codes. ICD-10-CM codes provide the ability to track and reveal more information about the quality of healthcare, allowing healthcare providers to better understand medical complications, better design treatment and care, and better comprehend and determine the outcome of care.

- ICD-10-PCS is used only for inpatient, hospital settings in the United States, and is meant to replace volume 3 of ICD-9 for facility reporting of inpatient procedures. Due to the rapid and constant state of flux in medical procedures and technology, ICD-10-PCS was developed to accommodate the changing landscape. Common procedures, lab tests, and educational sessions that are not unique to the inpatient, hospital setting have been omitted from ICD-10-PCS.

- ICD-10 is confusing enough when you’re trying to digest the differences between ICD-9 and ICD-10, but there are also different types of ICD-10 codes that providers should be aware of.


**Primary difference between ICD-10-CM and ICD-10-PCS**

- When most people talk about ICD-10, they are referring to ICD-10CM. This is the code set for diagnosis coding and is used for all healthcare settings in the United States. ICD-10PCS, on the other hand, is used in hospital inpatient settings for inpatient procedure coding.

**ICD-10-CM breakdown**

- Approximately 68,000 codes
- 3–7 alphanumeric characters
- Facilitates timely processing of claims


**ICD-10-PCS breakdown**

- Will replace ICD-9-CM for hospital inpatient use only. 
- ICD-10-PCS will not replace CPT codes used by physicians. According to HealthCare Information Management, Inc. (HCIM), “Its only intention is to identify inpatient facility services in a way not directly related to physician work, but directed towards allocation of hospital services.”
- 7 alphanumeric characters

- ICD-10-PCS is very different from ICD-9-CM procedure coding due to its ability to be more specific and accurate. “This becomes increasingly important when assessing and tracking the quality of medical processes and outcomes, and compiling statistics that are valuable tools for research,” according to HCIM.

**Hierarchical Condition Category (HCC)**

- Hierarchical condition category (HCC) coding is a risk-adjustment model originally designed to estimate future health care costs for patients. The Centers for Medicare & Medicaid Services (CMS) HCC model was initiated in 2004 but is becoming increasingly prevalent as the environment shifts to value-based payment models.

- Hierarchical condition category relies on ICD-10 coding to assign risk scores to patients. Each HCC is mapped to an ICD-10 code. Along with demographic factors (such as age and gender), insurance companies use HCC coding to assign patients a risk adjustment factor (RAF) score. Using algorithms, insurances can use a patient’s RAF score to predict costs. For example, a patient with few serious health conditions could be expected to have average medical costs for a given time. However, a patient with multiple chronic conditions would be expected to have higher health care utilization and costs.

**Pretrained ICD-10 Models**

- `sbiobertresolve_icd10cm_augmented`
- `sbiobertresolve_icd10pcs`
- `sbiobertresolve_icd10cm_augmented_billable_hcc`
- `sbiobertresolve_icd10cm`
- `sbiobertresolve_icd10cm_slim_normalized`
- `sbiobertresolve_icd10cm_slim_billable_hcc`
- `sbertresolve_icd10cm_slim_billable_hcc_med`
- `sbiobertresolve_icd10cm_generalised`

Let's create an ICD10 pipeline with `sbiobertresolve_icd10cm_augmented_billable_hcc` model and see how it works

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector DL annotator, processes various sentences per line
sentenceDetectorDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# WordEmbeddingsModel pretrained "embeddings_clinical" includes a model of 1.7Gb that needs to be downloaded
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("word_embeddings")

# Named Entity Recognition for clinical concepts.
clinical_ner = MedicalNerModel().pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_chunk = NerConverterInternal()\
    .setInputCols("sentence","token","clinical_ner")\
    .setOutputCol("clinical_ner_chunk")\
    .setWhiteList(["PROBLEM"])

c2doc = Chunk2Doc()\
    .setInputCols("clinical_ner_chunk")\
    .setOutputCol("ner_chunk_doc") 

sbiobert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["ner_chunk_doc"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)
    
icd_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")
    

# Build up the pipeline
resolver_pipeline = Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        clinical_ner_chunk,
        c2doc,
        sbiobert_embedder,
        icd_resolver
  ])

icd_model = resolver_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_clinical download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
#sample clinical note
clinical_note = (
    'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years '
    'prior to presentation and subsequent type two diabetes mellitus, associated '
    'with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, '
    'presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. ')


data_ner = spark.createDataFrame([[clinical_note]]).toDF("text")

Transforming the pipeline with clinical note

In [0]:
icd10_result = icd_model.transform(data_ner)

Checking the results

In [0]:
%%time
icd10_result.select("icd10cm_code.metadata").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
 metadata|
+----------------------------------------------------------------------------------------------------+
[{chunk -> 0, all_k_results -> O2441:::O2443:::Z8632:::Z875:::O2431:::O2411:::O244:::O241:::O2481...|
+----------------------------------------------------------------------------------------------------+

CPU times: user 141 ms, sys: 62.3 ms, total: 203 ms
Wall time: 51.2 s

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

icd10_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.clinical_ner_chunk.result, 
                                                   icd10_result.clinical_ner_chunk.metadata, 
                                                   icd10_result.icd10cm_code.result, 
                                                   icd10_result.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(icd10_df['all_codes'], icd10_df['resolutions'], icd10_df['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

icd10_df['all_codes'] = codes  
icd10_df['resolutions'] = resolutions
icd10_df['hcc_list'] = hcc_all

**The values in `billable`, `hcc_code` and `hcc_status` columns are seperated by || and we will change them to a list.**

In [0]:
icd10_df["billable"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
icd10_df["hcc_status"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
icd10_df["hcc_score"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

icd10_df.drop("hcc_list", axis=1, inplace= True)

In [0]:
icd10_df.head(15)

Out[17]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]","[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E11, E139, E119, E113, E1144, Z863, Z8639, E1132, E1143, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, diabetes mellitus type 2, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]","[0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]","[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0]","[0, 18, 0, 19, 19, 0, 75/18, 0, 0, 18, 75/18, 0]"
2,0,an acute hepatitis,PROBLEM,B15,"[B15, K720, B179, B172, Z0389, B159, K752, K712, B199, B16, K701, B169]","[acute hepatitis a, acute hepatitis, acute infectious hepatitis, acute hepatitis e, acute infectious hepatitis suspected, acute type a viral hepatitis, acute focal hepatitis, toxic liver disease with acute hepatitis, fulminant hepatitis, acute hepatitis b, acute alcoholic hepatitis, acute fulminating viral hepatitis]","[0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,0,obesity,PROBLEM,E669,"[E669, E668, Z6841, Q130, E66, E6601, Z8639, E349, H3550, Z8349, Q5562]","[obesity, abdominal obesity, obese, central obesity, overweight and obesity, morbid obesity, h/o: obesity, severe obesity, centripetal obesity, fh: obesity, truncal obesity]","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 22, 0, 0, 22, 0, 0, 0, 0, 0]"
4,0,a body mass index,PROBLEM,Z6841,"[Z6841, E669, R229, Z681, R223, R221, Z68, R222, R220, R4189, M6258, P299, R1900, R898, M2170, R190, M2180]","[finding of body mass index, observation of body mass index, mass of body region, finding of body mass index (finding), mass of upper limb, head and neck mass, body mass index [bmi], mass of trunk, mass of head, preoccupation with body size, finding of size of upper limb, mass of cardiovascular structure, mass of abdominal cavity structure, increase in circumference, finding of proportion of upper limb, visible abdominal mass, observation of size of upper limb]","[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,0,polyuria,PROBLEM,R35,"[R35, R358, E232, R31, R350, R8299, N401, E723, O048, R300, R319, E7201, R809, E888, R808, N030, P960, R80, N026, B80]","[polyuria, polyuric state, polyuric state (disorder), hematuria, micturition frequency and polyuria, uricosuria, pollakiuria, saccharopinuria, oligouria, dysuria, hematuria syndrome, cystinuria, cyclic proteinuria, sialuria, paroxysmal proteinuria, renal hematuria (disorder), oliguria and anuria, proteinuria, persistent hematuria (disorder), oxyuriasis]","[0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]","[0, 0, 23, 0, 0, 0, 0, 23, 0, 0, 0, 23, 0, 0, 0, 141, 0, 0, 141, 0]"
6,0,polydipsia,PROBLEM,R631,"[R631, F6389, E232, F639, O40, G475, M7989, R632, R06

### Entity Resolver Visualizer

Let's apply `EntityResolverVisualizer`  by using `sparknlp_display` library to see the results of the pipeline in a nicer layout:

In [0]:
from sparknlp_display import EntityResolverVisualizer

icd10_lp = LightPipeline(icd_model)

light_result = icd10_lp.fullAnnotate(clinical_note)

visualiser = EntityResolverVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080'})

vis= visualiser.display(light_result[0], 'clinical_ner_chunk', 'icd10cm_code', return_html=True)

displayHTML(vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM O2441 gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM O2411 pre-existing type 2 diabetes mellitus , associated with an acute hepatitis PROBLEM B15 acute hepatitis a , and obesity PROBLEM E669 obesity with a body mass index PROBLEM Z6841 finding of body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria PROBLEM R35 polyuria , polydipsia PROBLEM R631 polydipsia , poor appetite PROBLEM R630 poor appetite , and vomiting PROBLEM R111 vomiting .

### CPT Resolver

The Current Procedural Terminology (CPT) code set is a medical code set maintained by the American Medical Association. The CPT code set describes medical, surgical, and diagnostic services and is designed to communicate uniform information about medical services and procedures among physicians, coders, patients, accreditation organizations, and payers for administrative, financial, and analytical purposes.

**Pretrained Models**

- `sbiobertresolve_cpt`
- `sbiobertresolve_cpt_procedures_augmented`
- `sbiobertresolve_cpt_augmented`
- `sbiobertresolve_cpt_procedures_measurements_augmented`

**We will create a pipeline to detect bodyparts and imaging tests entities and their relations. Then we will use these related chunks in CPT resolver model to get their CPT code.**

In [0]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetectorDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("word_embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

# Named Entity Recognition for radiology reports.
radiology_ner = MedicalNerModel.pretrained("jsl_rd_ner_wip_greedy_clinical", "en", "clinical/models") \
   .setInputCols(["sentence", "token", "word_embeddings"]) \
   .setOutputCol("ner")

radiology_ner_chunker = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

re_model = RelationExtractionModel()\
    .pretrained("re_bodypart_directions", "en", 'clinical/models')\
    .setInputCols(["word_embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["imagingtest-bodypart", "bodypart-imagingtest"])\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.5)
      

# Build up the pipeline
rd_relation_pipeline = Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        pos_tagger,
        dependency_parser,
        radiology_ner,
        radiology_ner_chunker,
        re_model
        ])


rd_rel_model = rd_relation_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
jsl_rd_ner_wip_greedy_clinical download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][OK!]
re_bodypart_directions download started this may take some time.
Approximate size to download 9.2 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
light_rel_model = LightPipeline(rd_rel_model)

In [0]:
text="Left shin pain. I have ordered x-ray of the left fibula and knee today. The patient will return to the clinic in 3 weeks. He is to call me in the interim for any problems."
light_result = light_rel_model.fullAnnotate(text)

get_relations_df(light_result)

Out[21]:

,entity1,chunk1,entity2,chunk2,confidence
0,ImagingTest,x-ray,BodyPart,fibula,1.0
1,ImagingTest,x-ray,BodyPart,knee,0.9458425


Now we can get CPT codes of these related chunks.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("clinical_ner_chunk")

sbert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["clinical_ner_chunk"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)
    
cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_augmented","en", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("cpt_code")\
    .setDistanceFunction("EUCLIDEAN")


cpt_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])


cpt_lp = LightPipeline(cpt_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_cpt_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
text = 'fibula x-ray'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 158 ms, sys: 592 ms, total: 750 ms
Wall time: 2.02 s
Out[23]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,fibula x-ray,0,11,73590,"[73590, 27676, 27455, 27712, 27457, 27707, 73550, 73090, 27635]","[Tibia and fibula X-ray, Incision of fibula, Incision of fibula, Incision of fibula, Incision of fibula, Incision of fibula, Femur X-ray (procedure), Forearm X-ray, Excision of lesion of fibula (procedure)]","[0.0895, 0.0970, 0.0970, 0.0970, 0.0970, 0.0970, 0.1360, 0.1364, 0.1474]"


In [0]:
text = 'knee x-ray'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 7.08 ms, sys: 4.38 ms, total: 11.5 ms
Wall time: 219 ms
Out[24]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,knee x-ray,0,9,73564,"[73564, 73562, 73580, 27486, 1010487, 27540, 27332, 27310, 27334, 27335, 27330, 73600, 73610, 1005006, 27447, 27446]","[Knee X-ray, Knee X-ray, Knee arthrogram, Knee joint operations, Radiologic examination, knee, Operative procedure on knee, Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Ankle X-ray, Ankle X-ray, Arthrotomy, knee, Knee replacement, Knee replacement]","[0.0000, 0.0000, 0.0907, 0.1096, 0.1171, 0.1309, 0.1472, 0.1472, 0.1472, 0.1472, 0.1472, 0.1483, 0.1483, 0.1538, 0.1527, 0.1527]"


In [0]:
text="TECHNIQUE IN DETAIL: After informed consent was obtained from the patient and his mother, the chest was scanned with portable ultrasound."
light_result = light_rel_model.fullAnnotate(text)

get_relations_df(light_result)

Out[25]:

,entity1,chunk1,entity2,chunk2,confidence
0,BodyPart,chest,ImagingTest,portable ultrasound,0.9999802


In [0]:
text = 'chest portable ultrasound'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 231 ms
Out[26]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,chest portable ultrasound,0,24,1010771,"[1010771, 76512, 76510, 3320F, 76977, 76999, 76377, 45391, 76881, 31620, 3319F, 76882]","[Diagnostic Ultrasound Procedures of the Chest, A scan ultrasound, A scan ultrasound, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan]","[0.1269, 0.1563, 0.1563, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582]"


### BertSentenceChunkEmbeddings

**BertSentenceChunkEmbeddings**

- This annotator let users to aggregate sentence embeddings and ner chunk embeddings to get more specific and accurate resolution codes. It works by averaging context and chunk embeddings to get contextual information. Input to this annotator is the context (sentence) and ner chunks, while the output is embedding for each chunk that can be fed to the resolver model. The `setChunkWeight` parameter can be used to control the influence of surrounding context.

- For more information and examples of `BertSentenceChunkEmbeddings` annotator, you can check here: 
[24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

Lets do the same process like we did with `BertSentenceEmbeddings` annotator and compare the results. We will create a new pipeline by using this annotator with SentenceEntityResolverModel.

In [0]:
#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence", "clinical_ner_chunk"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)\
    .setChunkWeight(0.5) #default : 0.5
    
icd_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")
  
resolver_pipeline_SCE = Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        clinical_ner_chunk,
        sentence_chunk_embeddings,
        icd_resolver
  ])


model_SCE = resolver_pipeline_SCE.fit(spark.createDataFrame([['']]).toDF("text"))

sbiobert_base_cased_mli download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[ | ][OK!]

In [0]:
model_SCE.stages

Out[28]: [DocumentAssembler_e305d53bcebc,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_d3061317450f,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_cd5ce67b529f,
 NerConverter_f47ea4398ef1,
 BERTSENTENCECHUNKEMBEDDINGS_a896bf42dac8,
 ENTITY_1868e4d7b1a3]

In [0]:
icd10_result_SCE = model_SCE.transform(data_ner)

Checking the SentenceChunkEmbeddings results

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

res_SCE_pd = icd10_result_SCE.select(F.explode(F.arrays_zip(icd10_result_SCE.clinical_ner_chunk.result, 
                                                   icd10_result_SCE.clinical_ner_chunk.metadata, 
                                                   icd10_result_SCE.icd10cm_code.result, 
                                                   icd10_result_SCE.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(res_SCE_pd['all_codes'], res_SCE_pd['resolutions'], res_SCE_pd['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

res_SCE_pd['all_codes'] = codes  
res_SCE_pd['resolutions'] = resolutions
res_SCE_pd['hcc_list'] = hcc_all

In [0]:
res_SCE_pd["billable"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
res_SCE_pd["hcc_status"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
res_SCE_pd["hcc_score"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

res_SCE_pd.drop("hcc_list", axis=1, inplace= True)

In [0]:
res_SCE_pd.head(15)

Out[32]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,gestational diabetes mellitus,PROBLEM,O244,"[O244, O2443, O2441, O2442, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, maternal gestational diabetes mellitus, gestational diabetes mellitus in childbirth, pre-existing type 2 diabetes mellitus, in the puerperium]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2413,"[O2413, E116, E1143, O2411, E1110, E110, O2412, E114, E1169, E111, O241, K318, E1133, O244, E112]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, hyperosmolar coma due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, in childbirth, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, angina due to type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, diabetic gastroparesis associated with type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, gestational diabetes mellitus, class a>2<, persistent microalbuminuria due to type 2 diabetes mellitus]","[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]","[0, 0, 75/18, 0, 17, 0, 0, 0, 18, 0, 0, 0, 18, 0, 0]"
2,0,an acute hepatitis,PROBLEM,B169,"[B169, B159, B162, B179, B160, E8021, K712, O266, B181, F1098, B150]","[acute fulminating type b viral hepatitis (disorder), acute fulminating type a viral hepatitis (disorder), hepatic coma due to acute hepatitis b, acute fulminating viral hepatitis (disorder), acute viral hepatitis b with hepatic coma, acute intermittent (hepatic) porphyria, toxic liver disease with acute hepatitis (disorder), acute fatty liver of pregnancy, chronic viral hepatitis b with hepatic coma, ketoacidosis due to acute alcoholic intoxication, viral hepatitis a with hepatic coma (disorder)]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]","[0, 0, 0, 0, 0, 23, 0, 0, 29, 55, 0]"
3,0,obesity,PROBLEM,O9921,"[O9921, E233, E668, E349, O2600, E662, E669, E236, R7302, E6601, O99215, R730, N911]","[severe obesity complicating pregnancy, rapid-onset childhood obesity, hypothalamic dysfunction, hypoventilation, autonomic dysregulation syndrome, gynaecoid obesity, severe obesity, maternal obesity syndrome, rapid-onset childhood obesity, hypothalamic dysfunction, hypoventilation, autonomic dysregulation syndrome (disorder), lifelong obesity, hypothalamic infantilism with obesity syndrome, impaired glucose tolerance in obese, morbid (severe) obesity due to excess calories, obesity complicating the puerperium, impaired glucose tolerance in obese (disorder), amenorrhea associated with obesity]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]","[0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]","[0, 23, 0, 0, 0, 22, 0, 23, 0, 22, 0, 0, 0]"
4,0,a body mass index,PROBLEM,E1300,"[E1300, O211, R730, E110, O9921, O2441, O992, R7302, P90, O998, N911, E0900, E090, E668, O99215, E662, E233, E111, R298]","[hyperosmolarity due to secondary diabetes mellitus, hyperemesis gravidarum with metabolic disturbance, impaired glucose tolerance in obese (disorder), type 2 diabetes mellitus with hyperosmolarity, severe obesity complicating pregnancy, gestational diabetes, hyperglycemic disorder in pregnancy, impaired glucose tolerance in obese, obesity by age of onset, pregnancy and impaired glucose tolerance, amenorrhoea associated with obesity, hyperosmolarity due to drug induced diabetes mellitus, drug or chemical induced diabetes mellitus w hyperosmolarity, obesity by age of onset (disorder), obesity complicating the puerperium, rapid-onset childhood obesity, hypoth

Lets compare the results that we got from these two methods.

In [0]:
sentence_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.sentence.metadata, icd10_result.sentence.result)).alias("cols")) \
                          .select(F.expr("cols['0']['sentence']").alias("sent_id"),
                                  F.expr("cols['1']").alias("sentence_all")).toPandas()

comparison_df = pd.merge(icd10_df.loc[:,'sent_id':'resolutions'],res_SCE_pd.loc[:,'sent_id':'resolutions'],
                         on=['sent_id',"ner_chunk", "entity"], how='inner')
comparison_df.columns=['sent_id','ner_chunk', 'entity', 'icd10_code', 'all_codes', 'resolutions', 'icd10_code_SCE', 'all_codes_SCE', 'resolutions_SCE']

comparison_df = pd.merge(sentence_df, comparison_df,on="sent_id").drop('sent_id', axis=1)
comparison_df.head(15)

Out[33]:

,sentence_all,ner_chunk,entity,icd10_code,all_codes,resolutions,icd10_code_SCE,all_codes_SCE,resolutions_SCE
0,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]",O244,"[O244, O2443, O2441, O2442, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, maternal gestational diabetes mellitus, gestational diabetes mellitus in childbirth, pre-existing type 2 diabetes mellitus, in the puerperium]"
1,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E11, E139, E119, E113, E1144, Z863, Z8639, E1132, E1143, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, diabetes mellitus type 2, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]",O2413,"[O2413, E116, E1143, O2411, E1110, E110, O2412, E114, E1169, E111, O241, K318, E1133, O244, E112]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, hyperosmolar coma due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, in childbirth, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, angina due to type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, diabetic gastroparesis associated with type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, gestational diabetes mellitus, class a>2<, persistent microalbuminuria due to type 2 diabetes mellitus]"
2,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",an acute hepatitis,PROBLEM,B15,"[B15, K720, B179, B172, Z0389, B159, K752, K712, B199, B16, K701, B169]","[acute hepatitis a, acute hepatitis, acute infectious hepatitis, acute hepatitis e, acute infectious hepatitis suspected, acute type a viral hepatitis, acute focal hepatitis, toxic liver disease with acute hepatitis, fulminant hepatitis, acute hepatitis b, acute alcoholic hepatitis, acute fulminating viral hepatitis]",B169,"[B169, B159, B162, B179, B

### Router - Using Resolver Models Together

- Normally, when we need to use more than one sentence entity resolver models in the same pipeline, we used to hit `BertSentenceEmbeddings` annotator more than once given the number of different resolver models in the same pipeline. Now we are introducing a solution with the help of `Router` annotator that could allow us to feed all the NER chunks to `BertSentenceEmbeddings` at once and then route the output of Sentence Embeddings to different resolver models needed.

- In this example, lets use `sbiobertresolve_rxnorm_augmented` RxNorm model and `sbiobertresolve_icd10cm_augmented_billable_hcc` ICD10CM-HCC models together. First we will get the `PROBLEM` entities form `ner_clinical` model and then get the `DRUG` entities from `ner_posology` model. Then we will merge them and use the `Chunk2Doc` annotator to create sentence chunks to populate Sentence Embeddings column. Then, we route the embeddings of `PROBLEM` entities to ICD10CM model and embeddings of `DRUG` entities to RxNorm model at the same time.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols("document")\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("word_embeddings")

# to get PROBLEM entitis
clinical_ner = MedicalNerModel().pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_chunk = NerConverterInternal()\
    .setInputCols("sentence","token","clinical_ner")\
    .setOutputCol("clinical_ner_chunk")\
    .setWhiteList(["PROBLEM"])

# to get DRUG entities 
posology_ner = MedicalNerModel().pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_chunk = NerConverterInternal()\
    .setInputCols("sentence","token","posology_ner")\
    .setOutputCol("posology_ner_chunk")\
    .setWhiteList(["DRUG"])

# merge the chunks into a single ner_chunk
chunk_merger = ChunkMergeApproach()\
    .setInputCols("clinical_ner_chunk","posology_ner_chunk")\
    .setOutputCol("final_ner_chunk")


# convert chunks to doc to get sentence embeddings of them
chunk2doc = Chunk2Doc().setInputCols("final_ner_chunk").setOutputCol("final_chunk_doc")


sbiobert_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["final_chunk_doc"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)

# filter PROBLEM entity embeddings
router_sentence_icd10 = Router() \
    .setInputCols("sbert_embeddings") \
    .setFilterFieldsElements(["PROBLEM"]) \
    .setOutputCol("problem_embeddings")

# filter DRUG entity embeddings
router_sentence_rxnorm = Router() \
    .setInputCols("sbert_embeddings") \
    .setFilterFieldsElements(["DRUG"]) \
    .setOutputCol("drug_embeddings")

# use problem_embeddings only
icd_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
    .setInputCols(["problem_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")


# use drug_embeddings only
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
    .setInputCols(["drug_embeddings"]) \
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")


routerPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    clinical_ner_chunk,
    posology_ner,
    posology_ner_chunk,
    chunk_merger,
    chunk2doc,
    sbiobert_embeddings,
    router_sentence_icd10,
    router_sentence_rxnorm,
    icd_resolver,
    rxnorm_resolver
])

router_model = routerPipeline.fit(spark.createDataFrame([['']]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][ / ][ — ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[ | ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][OK!]

In [0]:
#sample clinical note
clinical_note = """The patient is a 41-year-old Vietnamese female with a cough that started last week. 
She has had right-sided chest pain radiating to her back with fever starting yesterday. 
She has a history of pericarditis in May 2006 and developed cough with right-sided chest pain. 
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
"""

Let's transform and see the results

In [0]:
df = spark.createDataFrame([[clinical_note]]).toDF("text")

result = router_model.transform(df)

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

icd10_result = result.select(F.explode(F.arrays_zip(result.clinical_ner_chunk.result, 
                                                   result.clinical_ner_chunk.metadata, 
                                                   result.icd10cm_code.result, 
                                                   result.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(icd10_result['all_codes'], icd10_result['resolutions'], icd10_result['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

icd10_result['all_codes'] = codes  
icd10_result['resolutions'] = resolutions
icd10_result['hcc_list'] = hcc_all

In [0]:
icd10_result["billable"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
icd10_result["hcc_status"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
icd10_result["hcc_score"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

icd10_result.drop("hcc_list", axis=1, inplace= True)

In [0]:
icd10_result.head(15)

Out[39]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,a cough,PROBLEM,R05,"[R05, B948, R053, R0989, R093]","[cough, persistent cough, chronic cough, respiratory tract congestion and cough (disorder), productive cough (finding)]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,1,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1011, M79621, M7962, M79604, R078, R101, R0781]","[right sided chest pain (finding), retrosternal chest pain, acute chest pain, chronic pain of right upper limb (finding), localised chest pain, ischaemic chest pain, chronic pain of right upper limb, right upper quadrant pain, pain in right upper arm, pain of right upper arm, pain in right leg, pleuropericardial chest pain, right upper quadrant pain (finding), costal margin chest pain]","[1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 88, 0, 0, 0, 0, 0, 0, 0, 0]"
2,1,fever,PROBLEM,P819,"[P819, R508, R509, B338, A938, F681, A689, P818, A230]","[fever, intermittent fever, fever symptoms, ossa fever, piry fever, artificial fever, recurrent fever, swinging fever, undulant fever]","[1, 0, 1, 1, 1, 0, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,2,pericarditis,PROBLEM,I319,"[I319, I301, I308, I318, B3323, A188, I425, I30, I309, I310, I241, T465X, I313, S2780]","[pericarditis, infectious pericarditis, adhesive pericarditis, chronic pericarditis, viral pericarditis, constrictive pericarditis, obliterative pericarditis, acute pericarditis, acute pericarditis (disorder), obliterative pericarditis (disorder), myopericarditis, drug-induced pericarditis, chronic pericarditis (disorder), injury of pericardium]","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 85, 0, 0, 0, 87, 59, 0, 0]"
4,2,cough,PROBLEM,R05,"[R05, B948, A37, R053, R292, R093]","[cough, persistent cough, whooping cough, chronic cough, cough reflex impaired, productive cough (finding)]","[1, 1, 0, 1, 1, 1]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
5,2,right-sided chest pain,PROBLEM,R0789,"[R0789, R072, R079, M796, R52, I209, M7960, R1011, M79621, M7962, M79604, R078, R101, R0781]","[right sided chest pain (finding), retrosternal chest pain, acute chest pain, chronic pain of right upper limb (finding), localised chest pain, ischaemic chest pain, chronic pain of right upper limb, right upper quadrant pain, pain in right upper arm, pain of right upper arm, pain in right leg, pleuropericardial chest pain, right upper quadrant pain (finding), costal margin chest pain]","[1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 88, 0, 0, 0, 0, 0, 0, 0, 0]"


Now let's get RxNorm codes from our results.

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

rxnorm_result = result.select(F.explode(F.arrays_zip(result.posology_ner_chunk.result, 
                                                   result.posology_ner_chunk.metadata, 
                                                   result.rxnorm_code.result, 
                                                   result.rxnorm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(rxnorm_result['all_codes'], rxnorm_result['resolutions'], rxnorm_result['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

rxnorm_result['all_codes'] = codes  
rxnorm_result['resolutions'] = resolutions
rxnorm_result['hcc_list'] = hcc_all

In [0]:
rxnorm_result.head()

Out[41]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,hcc_list
0,4,Coumadin,DRUG,202421,"[202421, 374998, 2898, 1855075, 128793, 1598, 152085, 218274, 21732, 375868, 1435932, 227590, 202392, 374560, 87866, 1603522, 5011, 54908]","[Coumadin, coumarin Oral Tablet, coumarin, coumaran, Vicodin, dicumarol, Mycifradin, Medcodin, cresatin, cresatin Oral Solution, Reumacetin, Geocillin, Garamycin, ardeparin Injectable Solution, ardeparin, glabridin, gramicidin, policresulen]","[Brand Name, Clinical Drug Form, Ingredient, Ingredient, Brand Name, Ingredient, Brand Name, Brand Name, Ingredient, Clinical Drug Form, Brand Name, Brand Name, Brand Name, Clinical Drug Form, Ingredient, Ingredient, Ingredient, Ingredient]"
1,6,Amiodarone,DRUG,703,"[703, 1663223, 1151983, 1151982, 203114, 377132, 1151981, 727379, 370568, 620, 1944369, 151347, 722, 370575, 1000082, 1000084, 5936, 17767, 2184119, 1153467, 1364410, 151908]","[amiodarone, amiodarone Injection, amiodarone Pill, amiodarone Oral Product, amiodarone hydrochloride, amiodarone Injectable Solution, amiodarone Injectable Product, amiodarone Prefilled Syringe, amiodarone Oral Tablet, amantadine, amantadine Extended Release Oral Capsule, Amsidine, amoxapine, amoxapine Oral Tablet, alcaftadine, alcaftadine Ophthalmic Solution, iodipamide, amlodipine, amlodipine Oral Suspension, adenosine Injectable Product, ioxapine, Iopidine]","[Ingredient, Clinical Drug Form, Clinical Dose Group, Clinical Dose Group, Precise Ingredient, Clinical Drug Form, Clinical Dose Group, Clinical Drug Form, Clinical Drug Form, Ingredient, Clinical Drug Form, Brand Name, Ingredient, Clinical Drug Form, Ingredient, Clinical Drug Form, Ingredient, Ingredient, Clinical Drug Form, Clinical Dose Group, Ingredient, Brand Name]"


### Sentence Entity Resolvers with EntityChunkEmbedding

**Entity Chunk Embeddings** uses BERT Sentence embeddings to compute a weighted average vector represention of related entity chunks. This model is a subclass of `BertSentenceEmbeddings` and shares all parameters with it. It can load any pretrained `BertSentenceEmbeddings` model. 

The input of the model consists of chunks of recognized named entities. One or more entities are selected as target entities and for each of them a list of related entities is specified (if empty, all other entities are assumed to be related).

 The model looks for chunks of the target entities and then tries to pair each target entity (e.g. DRUG) with other related entities (e.g. DOSAGE, STRENGTH, FORM, etc). The criterion for pairing a target entity with another related entity is that they appear in the same sentence and the **maximal syntactic distance** is below a predefined threshold. 

The relationship between target and related entities is one-to-many, meaning that if there are multiple instances of the same target entity (e.g. DRUG) within a sentence, the model will map a related entity (e.g. DOSAGE) to at most one of the instances of the target entity.

For example, if there is a sentence; "The patient was given 125 mg of paracetamol and metformin", the model will pair "125 mg" to "paracetamol", but not to "metformin". The output of the model is an average embeddings of the chunks of each of the target entities and their related entities. It is possible to specify a particular weight for each entity type. An entity can be defined both as target a entity and as a related entity for some other target entity.

Now, let's see how `EntityChunkEmbeddings` works.

In [0]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

posology_ner_model = MedicalNerModel.pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("posology_ner")

posology_ner_converter = NerConverterInternal()\
    .setInputCols("sentence", "token", "posology_ner")\
    .setOutputCol("posology_ner_chunk")

pos_tager = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos_tag")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tag", "token"])\
    .setOutputCol("dependencies")

entity_chunk_embeddings = EntityChunkEmbeddings.pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["posology_ner_chunk", "dependencies"])\
    .setOutputCol("drug_chunk_embeddings")\
    .setMaxSyntacticDistance(3)

entity_chunk_embeddings.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})
entity_chunk_embeddings.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})

rxnorm_re = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented_re", "en","clinical/models")\
    .setInputCols(["drug_chunk_embeddings"])\
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")

rxnorm_pipeline_re = Pipeline(
    stages = [
        documenter,
        sentence_detector,
        tokenizer,
        embeddings,
        posology_ner_model,
        posology_ner_converter,
        pos_tager,
        dependency_parser,
        entity_chunk_embeddings,
        rxnorm_re
        ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology_large download started this may take some time.
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
sbiobert_base_cased_mli download started this may take some time.
[ | ][ / ][ — ][OK!]
sbiobertresolve_rxnorm_augmented_re download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
sample_df = spark.createDataFrame([(1,"The patient was given metformin 500 mg tablet, 2.5 mg of coumadin and then ibuprofen."),
                                   (2,"The patient was given metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet")]).toDF("id","text")

sample_df.show(truncate=False)

+---+----------------------------------------------------------------------------------------+
id |text |
+---+----------------------------------------------------------------------------------------+
1 |The patient was given metformin 500 mg tablet, 2.5 mg of coumadin and then ibuprofen. |
2 |The patient was given metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet|
+---+----------------------------------------------------------------------------------------+

Fit/transform the model with sample data

In [0]:
results = rxnorm_pipeline_re.fit(sample_df).transform(sample_df)

Chunks extracted by NER model

In [0]:
results.select("id", F.explode(F.arrays_zip(results.posology_ner_chunk.result, results.posology_ner_chunk.metadata)).alias("cols")) \
       .select("id", F.expr("cols['0']").alias("chunk"),
                        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+---+----------+---------+
id |chunk |ner_label|
+---+----------+---------+
1 |metformin |DRUG |
1 |500 mg |STRENGTH |
1 |tablet |FORM |
1 |2.5 mg |STRENGTH |
1 |coumadin |DRUG |
1 |ibuprofen |DRUG |
2 |metformin |DRUG |
2 |400 mg |STRENGTH |
2 |coumadin |DRUG |
2 |5 mg |STRENGTH |
2 |coumadin |DRUG |
2 |amlodipine|DRUG |
2 |10 MG |STRENGTH |
2 |tablet |FORM |
+---+----------+---------+

**Merged chunks by internal relation extraction model feature**

- We specified the relations as following by `.setTargetEntities` parameter in the EntityChunkEmbeddings annotator :    
`.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})`

- EntityChunkEmbeddings calculates those new chunks embeddings according to the weights specified in`.setEntityWeights` parameter.

`.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})`

In [0]:
results.select('id','drug_chunk_embeddings.result').show(truncate = False)

+---+--------------------------------------------------------------------+
id |result |
+---+--------------------------------------------------------------------+
1 |[metformin 500 mg tablet, 2.5 mg coumadin, ibuprofen] |
2 |[metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet]|
+---+--------------------------------------------------------------------+

**RxNorm results**

In [0]:
results.select('id', F.explode(F.arrays_zip(results.drug_chunk_embeddings.result, 
                                               results.rxnorm_code.result, 
                                               results.rxnorm_code.metadata).alias("col")))\
       .select('id', F.expr("col['0']").alias("chunk"),
                     F.expr("col['1']").alias("rxnorm_code_"),
                     F.expr("col['2']['all_k_resolutions']").alias("Concept_Name")).show(truncate = 50)

+---+-----------------------+------------+--------------------------------------------------+
 id| chunk|rxnorm_code_| Concept_Name|
+---+-----------------------+------------+--------------------------------------------------+
 1|metformin 500 mg tablet| 860974|metformin hydrochloride 500 MG:::metformin 500 ...|
 1| 2.5 mg coumadin| 855313|warfarin sodium 2.5 MG [Coumadin]:::warfarin so...|
 1| ibuprofen| 1747293|ibuprofen Injection:::ibuprofen Pill:::ibuprofe...|
 2| metformin 400 mg| 332809|metformin 400 MG:::metformin 250 MG Oral Tablet...|
 2| coumadin 5 mg| 855333|warfarin sodium 5 MG [Coumadin]:::warfarin sodi...|
 2| coumadin| 202421|Coumadin:::warfarin sodium 2 MG/ML Injectable S...|
 2|amlodipine 10 MG tablet| 308135|amlodipine 10 MG Oral Tablet:::amlodipine 10 MG...|
+---+-----------------------+------------+--------------------------------------------------+

### Healthcare Codes Mapping by Using Pretrained Pipelines

In Spark NLP, there are pretrained pipelines that can map these healthcare codes each other. Here is a list of these pipelines:

|index|model|
|-----:|:-----|
| 1| [icd10_icd9_mapping](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapping_en.html)   |
| 2| [icdo_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icdo_snomed_mapping_en_3_0.html)   |
| 3| [icd10cm_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_snomed_mapping_en_3_0.html)   |
| 4| [rxnorm_ndc_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_ndc_mapping_en_3_0.html)   |
| 5| [rxnorm_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_umls_mapping_en_3_0.html)   |
| 6| [snomed_icd10cm_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icd10cm_mapping_en_3_0.html)   |
| 7| [snomed_icdo_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icdo_mapping_en_3_0.html)   |
| 8| [snomed_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_umls_mapping_en_3_0.html)   |
| 9| [icd10cm_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_umls_mapping_en_3_0.html)   |
| 10| [mesh_umls_mapping](https://nlp.johnsnowlabs.com/2021/07/01/mesh_umls_mapping_en.html)   |
| 11| [rxnorm_mesh_mapping](https://nlp.johnsnowlabs.com/2021/07/01/rxnorm_mesh_mapping_en.html)   |


Lets show an example of ICD codes mapping to Snomed Codes to show how these pipelines work.

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
icd10_snomed_pipeline = PretrainedPipeline("icd10cm_snomed_mapping", "en", "clinical/models")

icd10cm_snomed_mapping download started this may take some time.
Approx size to download 1 MB
[ | ][ / ][OK!]

In [0]:
icd10_snomed_pipeline.annotate('M8950 E119 H16269')

Out[11]: {'document': ['M8950 E119 H16269'],
 'icd10cm_code': ['M8950', 'E119', 'H16269'],
 'snomed_code': ['716868003', '170771004', '51264003']}

|**ICD10CM Code** | **ICD10CM Details** | **SNOMED Code** | **SNOMED Details** |
| ---------- | -----------:| ---------- | -----------:|
| M8950 |  Osteolysis, unspecified site | 716868003 | Multicentric osteolysis nodulosis arthropathy spectrum |
| E119 | Type 2 diabetes mellitus | 170771004 | Diabetic - follow-up default |
| H16269 | Vernal keratoconjunctivitis, with limbar and corneal involvement, unspecified eye | 51264003 | Limbal AND/OR corneal involvement in vernal conjunctivitis |

Also, if you want to see more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.1.Healthcare_Code_Mapping.ipynb).

# 7.Chunk Mapping

### Pretrained ChunkMapper Models

**<center>MAPPER MODELS**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [abbreviation_category_mapper](https://nlp.johnsnowlabs.com/2022/11/16/abbreviation_category_mapper_en.html)  | 2| [abbreviation_mapper](https://nlp.johnsnowlabs.com/2022/05/11/abbreviation_mapper_en_3_0.html)  | 3| [abbreviation_mapper_augmented](https://nlp.johnsnowlabs.com/2022/10/30/abbreviation_mapper_augmented_en.html)  |
| 4| [cvx_code_mapper](https://nlp.johnsnowlabs.com/2022/10/12/cvx_code_mapper_en.html)  | 5| [cvx_name_mapper](https://nlp.johnsnowlabs.com/2022/10/12/cvx_name_mapper_en.html)  | 6| [drug_action_treatment_mapper](https://nlp.johnsnowlabs.com/2022/03/31/drug_action_treatment_mapper_en_3_0.html)  |
| 7| [drug_ade_mapper](https://nlp.johnsnowlabs.com/2022/08/23/drug_ade_mapper_en.html)  | 8| [drug_brandname_ndc_mapper](https://nlp.johnsnowlabs.com/2022/05/11/drug_brandname_ndc_mapper_en_3_0.html)  | 9| [drug_category_mapper](https://nlp.johnsnowlabs.com/2022/12/18/drug_category_mapper_en.html)  |
| 10| [icd10_icd9_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapper_en.html)  | 11| [icd10cm_mapper](https://nlp.johnsnowlabs.com/2022/10/29/icd10cm_mapper_en.html)  | 12| [icd10cm_snomed_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icd10cm_snomed_mapper_en_3_0.html)  |
| 13| [icd10cm_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icd10cm_umls_mapper_en_3_0.html)  | 14| [icd9_icd10_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd9_icd10_mapper_en.html)  | 15| [icd9_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd9_mapper_en.html)  |
| 16| [icdo_snomed_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icdo_snomed_mapper_en_3_0.html)  | 17| [kegg_disease_mapper](https://nlp.johnsnowlabs.com/2022/11/18/kegg_disease_mapper_en.html)  | 18| [kegg_drug_mapper](https://nlp.johnsnowlabs.com/2022/11/21/kegg_drug_mapper_en.html)  |
| 19| [mesh_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/mesh_umls_mapper_en_3_0.html)  | 20| [normalized_section_header_mapper](https://nlp.johnsnowlabs.com/2022/06/26/normalized_section_header_mapper_en_3_0.html)  | 21| [rxnorm_action_treatment_mapper](https://nlp.johnsnowlabs.com/2022/05/08/rxnorm_action_treatment_mapper_en_3_0.html)  |
| 22| [rxnorm_mapper](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_mapper_en_3_0.html)  | 23| [rxnorm_ndc_mapper](https://nlp.johnsnowlabs.com/2022/05/20/rxnorm_ndc_mapper_en_3_0.html)  | 24| [rxnorm_normalized_mapper](https://nlp.johnsnowlabs.com/2022/09/29/rxnorm_normalized_mapper_en.html)  |
| 25| [rxnorm_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/rxnorm_umls_mapper_en_3_0.html)  | 26| [snomed_icd10cm_mapper](https://nlp.johnsnowlabs.com/2022/06/26/snomed_icd10cm_mapper_en_3_0.html)  | 27| [snomed_icdo_mapper](https://nlp.johnsnowlabs.com/2022/06/26/snomed_icdo_mapper_en_3_0.html)  |
| 28| [snomed_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/27/snomed_umls_mapper_en_3_0.html)  | 29| [umls_clinical_drugs_mapper](https://nlp.johnsnowlabs.com/2022/07/06/umls_clinical_drugs_mapper_en_3_0.html)  | 30| [umls_clinical_findings_mapper](https://nlp.johnsnowlabs.com/2022/07/08/umls_clinical_findings_mapper_en_3_0.html)  |
| 31| [umls_disease_syndrome_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_disease_syndrome_mapper_en_3_0.html)  | 32| [umls_drug_substance_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_drug_substance_mapper_en_3_0.html)  | 33| [umls_major_concepts_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_major_concepts_mapper_en_3_0.html)  |

**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=Chunk+Mapping&edition=Spark+NLP+for+Healthcare)**

<br>

**<center>PRETRAINED MAPPER PIPELINES**

|index|model|
|-----:|:-----|
| 1| [icd10_icd9_mapping](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapping_en.html)  |
| 2| [icd10cm_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_snomed_mapping_en_3_0.html)  |
| 3| [icd10cm_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/icd10cm_umls_mapping_en.html)  |
| 4| [icdo_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icdo_snomed_mapping_en_3_0.html)  |
| 5| [mesh_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/mesh_umls_mapping_en.html)  |
| 6| [rxnorm_mesh_mapping](https://nlp.johnsnowlabs.com/2021/05/04/rxnorm_mesh_mapping_en.html)  |
| 7| [rxnorm_ndc_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_ndc_mapping_en_3_0.html)  |
| 8| [rxnorm_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/rxnorm_umls_mapping_en.html)  |
| 9| [snomed_icd10cm_mapping](https://nlp.johnsnowlabs.com/2021/05/02/snomed_icd10cm_mapping_en.html)  |
| 10| [snomed_icdo_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icdo_mapping_en_3_0.html)  |
| 11| [snomed_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/snomed_umls_mapping_en.html)  |



You can check [Healthcare Code Mapping Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/11.1.Healthcare_Code_Mapping.ipynb) for the examples of pretrained mapper pipelines.

We will take a look at 2 pretrained `ChunkMapperModel()`; <br/>
**`drug_action_treatment_mapper`** model maps drugs with their corresponding action and treatment through `ChunkMapperModel()` annotator. 
**`normalized_section_header_mapper`** model normalizes the section headers in clinical notes. It returns two levels of normalization called level_1 and level_2. 

In both model, we can choose which option(action/treatment or level_1/level_2) we want to use by setting `setRel()` parameter of `ChunkMapperModel()`

For more detail about chunk mapping, please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/26.Chunk_Mapping.ipynb)

Let's create a piepline with `drug_action_treatment_mapper` and see how it works.

We will create a pipeline consisting `bert_token_classifier_drug_development_trials` ner model to extract ner chunk as well as `ChunkMapperModel()`.

In [0]:
#ChunkMapper Pipeline
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

btfc_ner =  MedicalBertForTokenClassifier.pretrained("bert_token_classifier_drug_development_trials", "en", "clinical/models")\
      .setInputCols("token","sentence")\
      .setOutputCol("btfc_ner")

btfc_ner_converter = NerConverterInternal()\
      .setInputCols("sentence", "token", "btfc_ner")\
      .setOutputCol("btfc_ner_chunk")

#drug_action_treatment_mapper with "action" mappings
chunkerMapper= ChunkMapperModel().pretrained("drug_action_treatment_mapper", "en", "clinical/models")\
    .setInputCols(["btfc_ner_chunk"])\
    .setOutputCol("action_mappings")\
    .setLowerCase(True) \
    .setRels(["action"]) # or treatment


mapperPipeline = Pipeline().setStages([
    document_assembler,
    sentence_detector,
    tokenizer,
    btfc_ner, 
    btfc_ner_converter, 
    chunkerMapper])


test_data = spark.createDataFrame([["The patient was female and patient of Dr. X. and she was given Dermovate, Aspagin"]]).toDF("text")

res = mapperPipeline.fit(test_data).transform(test_data)

bert_token_classifier_drug_development_trials download started this may take some time.
[ | ][OK!]
drug_action_treatment_mapper download started this may take some time.
[ | ][OK!]

Chunks detected by ner model

In [0]:
res.select(F.explode('btfc_ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
chunks |
+---------+
Dermovate|
Aspagin |
+---------+

Checking the mapping results for **action**

In [0]:
res.selectExpr("action_mappings.metadata").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
metadata |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[{chunk -> 0, __trained__ -> Dermovate, relation -> action, __distance_function__ -> levenshtein, confidence -> 0.9997595, ops -> 0.0, all_relations -> corticosteroids::: dermatological preparations:::very strong, entity -> Dermovate, distance -> 0.0, sentence -> 0, __relation_name__ -> action}, {chunk -> 1, __trained__ -> Aspagin, relation -> action, __distance_function__ -> levenshtein, confidence -> 0.99668664, ops -> 0.0, all_relations -> anti-inflammatory:::antipyretic, entity -> Aspagin, distance -> 0.0, sentence -> 0, __relation_name__ -> action}]|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

As you see above under the metadata column, if exist, we can see all the relations for each chunk.

In [0]:
res.select(F.explode(F.arrays_zip(res.btfc_ner_chunk.result, res.action_mappings.result, res.action_mappings.metadata)).alias("col"))\
   .select(F.expr("col['0']").alias("ner_chunk"),
           F.expr("col['1']").alias("pharmaceutical_action"),
           F.expr("col['2']['all_relations']").alias("other_pharmaceutical_actions")).show(truncate=False)

+---------+---------------------+------------------------------------------------------------+
ner_chunk|pharmaceutical_action|other_pharmaceutical_actions |
+---------+---------------------+------------------------------------------------------------+
Dermovate|anti-inflammatory |corticosteroids::: dermatological preparations:::very strong|
Aspagin |analgesic |anti-inflammatory:::antipyretic |
+---------+---------------------+------------------------------------------------------------+

### Creating a Mapper Model

There is `ChunkMapperApproach()` to create your own mapper model. <br/>

This receives an `ner_chunk` and a Json with a mapping of ner entities and relations, and returns the `ner_chunk` augmented with the relations from the Json ontology. <br/> We give the path of json file to the `setDictionary()` parameter.

Let's create an example Json with single key, then create a drug mapper model. This model will match the given drug name (only "metformin" for our example) with correpsonding action and treatment.  

The format of json file should be like following:

````
{
  "mappings": [
    {
      "key": "metformin",
      "relations": [
        {
          "key": "action",
          "values" : ["hypoglycemic", "Drugs Used In Diabets"]
        },
        {
          "key": "treatment",
          "values" : ["diabetes", "t2dm"]
        }
      ]
    }
  ]
}

````

By using `setRel()` parameter, we tell the model which type of mapping we want. In our case, if we want from our model to return **action** mapping, we set the parameter as `setRel("action")`,  we set as `setRel("treatment")` for **treatment**

Let's create a pipeline and see it in action.

In [0]:
data_set = {
  "mappings": [
    {
      "key": "metformin",
      "relations": [
        {
          "key": "action",
          "values" : ["hypoglycemic", "Drugs Used In Diabetes"]
        },
        {
          "key": "treatment",
          "values" : ["diabetes", "t2dm"]
        }
      ]
    }
  ]
}

import json
with open('/dbfs/sample_drug.json', 'w') as f:
    json.dump(data_set, f)

In [0]:
document_assembler = DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

posology_ner_model = MedicalNerModel.pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("posology_ner")

posology_ner_converter = NerConverterInternal()\
    .setInputCols("sentence", "token", "posology_ner")\
    .setOutputCol("posology_ner_chunk")\
    .setWhiteList(["DRUG"])

chunkerMapper = ChunkMapperApproach()\
      .setInputCols(["posology_ner_chunk"])\
      .setOutputCol("mappings")\
      .setDictionary("/sample_drug.json")\
      .setLowerCase(True) \
      .setRel("action") #or treatment

mapper_pipeline = Pipeline().setStages([
        document_assembler,
        sentence_detector,
        tokenizer, 
        word_embeddings,
        posology_ner_model, 
        posology_ner_converter, 
        chunkerMapper])


test_data = spark.createDataFrame([["The patient was given 1 unit of metformin daily."]]).toDF("text")

mapper_model = mapper_pipeline.fit(test_data)
res= mapper_model.transform(test_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_posology_large download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

Let's check our model's results

In [0]:
#ner result
res.select(F.explode('posology_ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
chunks |
+---------+
metformin|
+---------+

In [0]:
res.select(F.explode(F.arrays_zip(res.posology_ner_chunk.result, res.mappings.result, res.mappings.metadata)).alias("col"))\
   .select(F.expr("col['0']").alias("ner_chunk"),
           F.expr("col['1']").alias("mapping_result"),
           F.expr("col['2']['all_relations']").alias("all_relations")).show(truncate=False)

+---------+--------------+----------------------+
ner_chunk|mapping_result|all_relations |
+---------+--------------+----------------------+
metformin|hypoglycemic |Drugs Used In Diabetes|
+---------+--------------+----------------------+

As you see, we successfully created our custom mapper model.

# 8.Pretrained Clinical Pipelines

In order to save you from creating a pipeline from scratch, Spark NLP also has a pre-trained pipelines that are already fitted using certain annotators and transformers according to various use cases.

Here is the list of clinical pre-trained pipelines: 

**1.   explain_clinical_doc_carp** :

> A pipeline with `ner_clinical`, `assertion_dl`, `re_clinical` and `ner_posology`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

**2.   explain_clinical_doc_era** :

> A pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.

**3.   explain_clinical_doc_ade** :

> A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_biobert`, `assertiondl_biobert`, `classifierdl_ade_conversational_biobert` and `re_ade_biobert`. It will classify the document, extract `ADE` and `DRUG` entities, assign assertion status to `ADE` entities, and relate them with `DRUG` entities, then assign ADE status to a text (`True` means ADE, `False` means not related to ADE).

**letter codes in the naming conventions:**

> c : ner_clinical

> e : ner_clinical_events

> r : relation extraction

> p : ner_posology

> a : assertion

> ade : adverse drug events

**Relation Extraction types:**

`re_clinical` >> TrIP (improved), TrWP (worsened), TrCP (caused problem), TrAP (administered), TrNAP (avoided), TeRP (revealed problem), TeCP (investigate problem), PIP (problems related)

`re_temporal_events_clinical` >> `AFTER`, `BEFORE`, `OVERLAP`

**4. explain_clinical_doc_medication:**

> A pipeline for detecting posology entities with the `ner_posology_large` NER model, assigning their assertion status with `assertion_jsl` model, and extracting relations between posology-related terminology with `posology_re` relation extraction model.


**5. explain_clinical_doc_radiology**

> A pipeline for detecting radiology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between the diagnosis, test, and findings with `re_test_problem_finding` relation extraction model.

**6. Clinical Deidentification** :

>This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities.

**7. NER Pipelines:**

> Pipelines for all the available pretrained NER models.

**8. BERT Based NER Pipelines**

> Pipelines for all the available Bert token classification models.

**9. ner_profiling_clinical and ner_profiling_biobert:**

> Pipelines for exploring all the available pretrained NER models at once.

**10. ner_model_finder**

> A pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

**11. Resolver Pipelines**

> Pipelines for converting clinical entities to their UMLS CUI codes and medication entities to their ADE, Action, Treatment, UMLS, RxNorm, ICD9, SNOMED and NDC codes.

**12. Oncology Pipelines**

> Pipelines includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from oncology texts.


**Also, you can find clinical CODE MAPPING pretrained pipelines in this notebook: [Healthcare Code Mapping Notebook](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/11.1.Healthcare_Code_Mapping.ipynb)**

<table>
<tr><th>Pretraned NER Pipelines </th><th>
<tr><td>

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [bert_token_classifier_drug_development_trials_pipeline](https://nlp.johnsnowlabs.com/2022/03/23/bert_token_classifier_drug_development_trials_pipeline_en_3_0.html)  | 2| [bert_token_classifier_ner_ade_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_ade_pipeline_en_3_0.html)  | 3| [bert_token_classifier_ner_anatomy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_anatomy_pipeline_en_3_0.html)  |
| 4| [bert_token_classifier_ner_bacteria_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_bacteria_pipeline_en_3_0.html)  | 5| [bert_token_classifier_ner_bionlp_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_bionlp_pipeline_en_3_0.html)  | 6| [bert_token_classifier_ner_cellular_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_cellular_pipeline_en_3_0.html)  |
| 7| [bert_token_classifier_ner_chemicals_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_chemicals_pipeline_en_3_0.html)  | 8| [bert_token_classifier_ner_chemprot_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_chemprot_pipeline_en_3_0.html)  | 9| [bert_token_classifier_ner_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_clinical_pipeline_en_3_0.html)  |
| 10| [bert_token_classifier_ner_deid_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_deid_pipeline_en_3_0.html)  | 11| [bert_token_classifier_ner_drugs_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_drugs_pipeline_en_3_0.html)  | 12| [bert_token_classifier_ner_jsl_pipeline](https://nlp.johnsnowlabs.com/2022/03/23/bert_token_classifier_ner_jsl_pipeline_en_3_0.html)  |
| 13| [bert_token_classifier_ner_jsl_slim_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_jsl_slim_pipeline_en_3_0.html)  | 14| [german_deid_pipeline_spark24](https://nlp.johnsnowlabs.com/2022/03/03/german_deid_pipeline_spark24_de_2_4.html)  | 15| [jsl_ner_wip_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_clinical_pipeline_en_3_0.html)  |
| 16| [jsl_ner_wip_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_greedy_biobert_pipeline_en_3_0.html)  | 17| [jsl_ner_wip_greedy_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_greedy_clinical_pipeline_en_3_0.html)  | 18| [jsl_ner_wip_modifier_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_modifier_clinical_pipeline_en_3_0.html)  |
| 19| [jsl_rd_ner_wip_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_rd_ner_wip_greedy_biobert_pipeline_en_3_0.html)  | 20| [jsl_rd_ner_wip_greedy_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_rd_ner_wip_greedy_clinical_pipeline_en_3_0.html)  | 21| [ner_abbreviation_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_abbreviation_clinical_pipeline_en_3_0.html)  |
| 22| [ner_ade_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_biobert_pipeline_en_3_0.html)  | 23| [ner_ade_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_clinical_pipeline_en_3_0.html)  | 24| [ner_ade_clinicalbert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_clinicalbert_pipeline_en_3_0.html)  |
| 25| [ner_ade_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_ade_healthcare_pipeline_en_3_0.html)  | 26| [ner_anatomy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_biobert_pipeline_en_3_0.html)  | 27| [ner_anatomy_coarse_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_coarse_biobert_pipeline_en_3_0.html)  |
| 28| [ner_anatomy_coarse_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_coarse_pipeline_en_3_0.html)  | 29| [ner_anatomy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_pipeline_en_3_0.html)  | 30| [ner_bacterial_species_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bacterial_species_pipeline_en_3_0.html)  |
| 31| [ner_biomarker_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_biomarker_pipeline_en_3_0.html)  | 32| [ner_bionlp_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bionlp_biobert_pipeline_en_3_0.html)  | 33| [ner_bionlp_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bionlp_pipeline_en_3_0.html)  |
| 34| [ner_cancer_genetics_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cancer_genetics_pipeline_en_3_0.html)  | 35| [ner_cellular_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cellular_biobert_pipeline_en_3_0.html)  | 36| [ner_cellular_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cellular_pipeline_en_3_0.html)  |
| 37| [ner_chemicals_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemicals_pipeline_en_3_0.html)  | 38| [ner_chemprot_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemprot_biobert_pipeline_en_3_0.html)  | 39| [ner_chemprot_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemprot_clinical_pipeline_en_3_0.html)  |
| 40| [ner_chexpert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chexpert_pipeline_en_3_0.html)  | 41| [ner_clinical_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_biobert_pipeline_en_3_0.html)  | 42| [ner_clinical_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_large_pipeline_en_3_0.html)  |
| 43| [ner_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_pipeline_en_3_0.html)  | 44| [ner_diseases_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_biobert_pipeline_en_3_0.html)  | 45| [ner_diseases_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_large_pipeline_en_3_0.html)  |
| 46| [ner_diseases_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_pipeline_en_3_0.html)  | 47| [ner_drugprot_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugprot_clinical_pipeline_en_3_0.html)  | 48| [ner_drugs_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_greedy_pipeline_en_3_0.html)  |
| 49| [ner_drugs_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_large_pipeline_en_3_0.html)  | 50| [ner_drugs_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_pipeline_en_3_0.html)  | 51| [ner_events_admission_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_admission_clinical_pipeline_en_3_0.html)  |
| 52| [ner_events_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_biobert_pipeline_en_3_0.html)  | 53| [ner_events_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_clinical_pipeline_en_3_0.html)  | 54| [ner_events_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_events_healthcare_pipeline_en_3_0.html)  |
| 55| [ner_genetic_variants_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_genetic_variants_pipeline_en_3_0.html)  | 56| [ner_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_healthcare_pipeline_en_3_0.html)  | 57| [ner_human_phenotype_gene_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_gene_biobert_pipeline_en_3_0.html)  |
| 58| [ner_human_phenotype_gene_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_gene_clinical_pipeline_en_3_0.html)  | 59| [ner_human_phenotype_go_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_go_biobert_pipeline_en_3_0.html)  | 60| [ner_human_phenotype_go_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_go_clinical_pipeline_en_3_0.html)  |
| 61| [ner_jsl_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_biobert_pipeline_en_3_0.html)  | 62| [ner_jsl_enriched_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_enriched_biobert_pipeline_en_3_0.html)  | 63| [ner_jsl_enriched_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_enriched_pipeline_en_3_0.html)  |
| 64| [ner_jsl_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_greedy_biobert_pipeline_en_3_0.html)  | 65| [ner_jsl_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_greedy_pipeline_en_3_0.html)  | 66| [ner_jsl_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_pipeline_en_3_0.html)  |
| 67| [ner_jsl_slim_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_slim_pipeline_en_3_0.html)  | 68| [ner_measurements_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_measurements_clinical_pipeline_en_3_0.html)  | 69| [ner_medmentions_coarse_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_medmentions_coarse_pipeline_en_3_0.html)  |
| 70| [ner_nihss_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_nihss_pipeline_en_3_0.html)  | 71| [ner_posology_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_biobert_pipeline_en_3_0.html)  | 72| [ner_posology_experimental_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_experimental_pipeline_en_3_0.html)  |
| 73| [ner_posology_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_greedy_pipeline_en_3_0.html)  | 74| [ner_posology_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_posology_healthcare_pipeline_en_3_0.html)  | 75| [ner_posology_large_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_large_biobert_pipeline_en_3_0.html)  |
| 76| [ner_posology_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_large_pipeline_en_3_0.html)  | 77| [ner_posology_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_pipeline_en_3_0.html)  | 78| [ner_posology_small_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_small_pipeline_en_3_0.html)  |
| 79| [ner_radiology_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_radiology_pipeline_en_3_0.html)  | 80| [ner_radiology_wip_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_radiology_wip_clinical_pipeline_en_3_0.html)  | 81| [ner_risk_factors_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_risk_factors_biobert_pipeline_en_3_0.html)  |
| 82| [ner_risk_factors_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_risk_factors_pipeline_en_3_0.html)  | 83| []()| 84| []()|

 </table>

<table>
<tr><th>Pretraned Pipelines for Different Use Cases</th><th>
<td><tr>

|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [ner_deid_augmented_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_augmented_pipeline_en_3_0.html)  | 2| [ner_deid_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_biobert_pipeline_en_3_0.html)  |
| 3| [ner_deid_enriched_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_enriched_biobert_pipeline_en_3_0.html)  | 4| [ner_deid_enriched_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_enriched_pipeline_en_3_0.html)  |
| 5| [ner_deid_generic_augmented_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_generic_augmented_pipeline_en_3_0.html)  | 6| [ner_deid_generic_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_generic_pipeline_de_3_0.html)  |
| 7| [ner_deid_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_large_pipeline_en_3_0.html)  | 8| [ner_deid_sd_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_sd_large_pipeline_en_3_0.html)  |
| 9| [ner_deid_sd_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_sd_pipeline_en_3_0.html)  | 10| [ner_deid_subentity_augmented_i2b2_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_subentity_augmented_i2b2_pipeline_en_3_0.html)  |
| 11| [ner_deid_subentity_augmented_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_subentity_augmented_pipeline_en_3_0.html)  | 12| [ner_deid_subentity_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deid_subentity_pipeline_de_3_0.html)  |
| 13| [ner_deidentify_dl_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_deidentify_dl_pipeline_en_3_0.html)  | 14| [re_bodypart_directions_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_directions_pipeline_en_3_0.html)  |
| 15| [re_bodypart_proceduretest_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_proceduretest_pipeline_en_3_0.html)  | 16| [re_date_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_date_clinical_pipeline_en_3_0.html)  |
| 17| [re_human_phenotype_gene_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_human_phenotype_gene_clinical_pipeline_en_3_0.html)  | 18| [re_temporal_events_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_clinical_pipeline_en_3_0.html)  |
| 19| [re_temporal_events_enriched_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_enriched_clinical_pipeline_en_3_0.html)  | 20| [re_test_problem_finding_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_problem_finding_pipeline_en_3_0.html)  |
| 21| [re_test_result_date_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_result_date_pipeline_en_3_0.html)  | 22| [explain_clinical_doc_radiology](https://nlp.johnsnowlabs.com/2022/03/31/explain_clinical_doc_radiology_en_3_0.html)|
| 23| [explain_clinical_doc_ade](https://nlp.johnsnowlabs.com/2021/07/15/explain_clinical_doc_ade_en.html)  | 24| [explain_clinical_doc_carp](https://nlp.johnsnowlabs.com/2020/08/19/explain_clinical_doc_carp_en.html)  |
| 25| [explain_clinical_doc_cra](https://nlp.johnsnowlabs.com/2020/08/19/explain_clinical_doc_cra_en.html)  | 26| [explain_clinical_doc_era](https://nlp.johnsnowlabs.com/2020/09/30/explain_clinical_doc_era_en.html)  |
| 27| [explain_clinical_doc_medication](https://nlp.johnsnowlabs.com/2022/04/01/explain_clinical_doc_medication_en_3_0.html)  | 28|   |


</td></tr>

</table>

You can visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.Pretrained_Clinical_Pipelines.ipynb) in order to see the usage of all these pipelines. 

Now, firstly let's pick the `explain_clinical_doc_carp` and see how it works.

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
pipeline = PretrainedPipeline('explain_clinical_doc_carp', 'en', 'clinical/models')

explain_clinical_doc_carp download started this may take some time.
Approx size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
# Load pretrained pipeline from local disk:

# >> pipeline_local = PretrainedPipeline.from_disk('/root/cache_pretrained/explain_clinical_doc_carp_en_2.5.5_2.4_1597841630062')

We will annotate our pretrained pipeline with sample data and see the result

In [0]:
text ="""28-year-old female with a history of gestational diabetes mellitus, used to take metformin 1000 mg two times a day, presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
She was seen by the endocrinology service and discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals.
"""

annotations = pipeline.annotate(text)

annotations.keys()


Out[22]: dict_keys(['sentences', 'clinical_ner_tags', 'document', 'clinical_ner_chunks', 'assertion', 'clinical_relations', 'posology_ner_tags', 'tokens', 'posology_ner_chunks', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
import pandas as pd

rows = list(zip(annotations['tokens'], annotations['clinical_ner_tags'], annotations['posology_ner_tags'], annotations['pos_tags'], annotations['dependencies']))

df = pd.DataFrame(rows, columns = ['tokens','clinical_ner_tags','posology_ner_tags','POS_tags','dependencies'])

df.head(20)

Out[23]:

,tokens,clinical_ner_tags,posology_ner_tags,POS_tags,dependencies
0,28-year-old,O,O,JJ,female
1,female,O,O,NN,ROOT
2,with,O,O,II,history
3,a,O,O,DD,history
4,history,O,O,NN,female
5,of,O,O,II,history
6,gestational,B-PROBLEM,O,JJ,of
7,diabetes,I-PROBLEM,O,NN,mellitus
8,mellitus,I-PROBLEM,O,NN,gestational
9,",",O,O,NN,used


**`oncology pretrained ` Model List**

This pipeline includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from oncology texts.


|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [oncology_biomarker_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_biomarker_pipeline_en.html)  | 2| [oncology_general_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_general_pipeline_en.html)  |
| 3| [oncology_therapy_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_therapy_pipeline_en.html)  | 4| [oncology_diagnosis_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_diagnosis_pipeline_en.html)

In [0]:
oncology_pipeline = PretrainedPipeline("oncology_biomarker_pipeline", "en", "clinical/models")

oncology_biomarker_pipeline download started this may take some time.
Approx size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
oncology_pipeline.model.stages

Out[65]: [DocumentAssembler_dab6eeac879e,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_c1e2dce98caf,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_ecf280ca65e5,
 NerConverter_929f666beebc,
 MedicalNerModel_aeadb24f76a3,
 NerConverter_4f9b8da8c4c3,
 MedicalNerModel_eb9da4b9039b,
 NerConverter_40028785be7b,
 MedicalNerModel_299a97740594,
 NerConverter_a8bf552d0249,
 MERGE_acda18b976a6,
 MERGE_12af4df6fa27,
 ASSERTION_DL_8d77f383c928,
 ASSERTION_DL_163867728788,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_d0af74510daa,
 RelationExtractionModel_68ebe11369b6,
 RelationExtractionModel_513eb6317779]

In [0]:
text = """Immunohistochemistry was negative for thyroid transcription factor-1 and napsin A. The test was positive for ER and PR, and negative for HER2."""

result = oncology_pipeline.fullAnnotate(text)[0]

result.keys()

Out[73]: dict_keys(['re_oncology_granular_wip', 'assertion_oncology_test_binary_wip', 're_oncology_wip', 'ner_oncology_biomarker_wip_chunk', 'ner_biomarker_chunk', 'ner_oncology_test_wip', 'document', 're_oncology_biomarker_result_wip', 'merged_chunk', 'ner_oncology_biomarker_wip', 'ner_biomarker', 'ner_oncology_test_wip_chunk', 'token', 'embeddings', 'pos_tags', 'assertion_oncology_wip', 'assertion_chunk', 'ner_oncology_wip', 'dependencies', 'ner_oncology_wip_chunk', 'sentence'])

**NER Results**

In [0]:
chunks=[]
entities=[]
begins=[]
ends=[]
confidence=[]
for n in result['merged_chunk']:
    
    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence']) 
        
df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities, 'confidence':confidence})

df

Out[74]:

,chunks,begin,end,entities,confidence
0,Immunohistochemistry,0,19,Pathology_Test,0.9967
1,negative,25,32,Biomarker_Result,0.8323
2,thyroid transcription factor-1,38,67,Biomarker,0.296675
3,napsin A,73,80,Biomarker,0.64309996
4,positive,96,103,Biomarker_Result,0.8017
5,ER,109,110,Biomarker,0.948
6,PR,116,117,Biomarker,0.8711
7,negative,124,131,Biomarker_Result,0.8385
8,HER2,137,140,Oncogene,0.9359


**Assertion Status Results**

In [0]:
chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(result['merged_chunk'],result['assertion_oncology_test_binary_wip']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(m.metadata['confidence'])
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

Out[68]:

,chunks,entities,assertion,confidence
0,Immunohistochemistry,Pathology_Test,Medical_History,0.9926
1,negative,Biomarker_Result,Medical_History,0.9951
2,thyroid transcription factor-1,Biomarker,Medical_History,0.9951
3,napsin A,Biomarker,Medical_History,0.9926
4,positive,Biomarker_Result,Medical_History,0.9931
5,ER,Biomarker,Medical_History,0.9938


**Relation Extraction Results**

In [0]:
def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  rel_df.confidence = rel_df.confidence.astype(float)
  
  return rel_df

In [0]:
result = oncology_pipeline.fullAnnotate(text)

rel_df = get_relations_df(result, 're_oncology_wip')

rel_df[rel_df.relation!= "O"]

Out[76]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
1,is_related_to,Biomarker_Result,25,32,negative,Biomarker,38,67,thyroid transcription factor-1,0.999746
2,is_related_to,Biomarker_Result,25,32,negative,Biomarker,73,80,napsin A,0.999423
3,is_related_to,Biomarker_Result,96,103,positive,Biomarker,109,110,ER,0.998581
4,is_related_to,Biomarker_Result,96,103,positive,Biomarker,116,117,PR,0.915055
8,is_related_to,Biomarker_Result,124,131,negative,Oncogene,137,140,HER2,0.997258


In [0]:
result[0]['re_oncology_wip']

Out[77]: [Annotation(category, 0, 32, O, {'chunk2': 'negative', 'confidence': '0.9989562', 'entity2_end': '32', 'chunk1': 'Immunohistochemistry', 'entity2_begin': '25', 'entity1': 'Pathology_Test', 'entity1_begin': '0', 'direction': 'both', 'entity1_end': '19', 'entity2': 'Biomarker_Result'}),
 Annotation(category, 25, 67, is_related_to, {'chunk2': 'thyroid transcription factor-1', 'confidence': '0.9997464', 'entity2_end': '67', 'chunk1': 'negative', 'entity2_begin': '38', 'entity1': 'Biomarker_Result', 'entity1_begin': '25', 'direction': 'both', 'entity1_end': '32', 'entity2': 'Biomarker'}),
 Annotation(category, 25, 80, is_related_to, {'chunk2': 'napsin A', 'confidence': '0.99942315', 'entity2_end': '80', 'chunk1': 'negative', 'entity2_begin': '73', 'entity1': 'Biomarker_Result', 'entity1_begin': '25', 'direction': 'both', 'entity1_end': '32', 'entity2': 'Biomarker'}),
 Annotation(category, 96, 110, is_related_to, {'chunk2': 'ER', 'confidence': '0.9985808', 'entity2_end': '110', 'chunk1': 'positive', 'entity2_begin': '109', 'entity1': 'Biomarker_Result', 'entity1_begin': '96', 'direction': 'both', 'entity1_end': '103', 'entity2': 'Biomarker'}),
 Annotation(category, 96, 117, is_related_to, {'chunk2': 'PR', 'confidence': '0.91505456', 'entity2_end': '117', 'chunk1': 'positive', 'entity2_begin': '116', 'entity1': 'Biomarker_Result', 'entity1_begin': '96', 'direction': 'both', 'entity1_end': '103', 'entity2': 'Biomarker'}),
 Annotation(category, 96, 140, O, {'chunk2': 'HER2', 'confidence': '0.9692235', 'entity2_end': '140', 'chunk1': 'positive', 'entity2_begin': '137', 'entity1': 'Biomarker_Result', 'entity1_begin': '96', 'direction': 'both', 'entity1_end': '103', 'entity2': 'Oncogene'}),
 Annotation(category, 109, 131, O, {'chunk2': 'negative', 'confidence': '0.99970347', 'entity2_end': '131', 'chunk1': 'ER', 'entity2_begin': '124', 'entity1': 'Biomarker', 'entity1_begin': '109', 'direction': 'both', 'entity1_end': '110', 'entity2': 'Biomarker_Result'}),
 Annotation(category, 116, 131, O, {'chunk2': 'negative', 'confidence': '0.99997675', 'entity2_end': '131', 'chunk1': 'PR', 'entity2_begin': '124', 'entity1': 'Biomarker', 'entity1_begin': '116', 'direction': 'both', 'entity1_end': '117', 'entity2': 'Biomarker_Result'}),
 Annotation(category, 124, 140, is_related_to, {'chunk2': 'HER2', 'confidence': '0.9972582', 'entity2_end': '140', 'chunk1': 'negative', 'entity2_begin': '137', 'entity1': 'Biomarker_Result', 'entity1_begin': '124', 'direction': 'both', 'entity1_end': '131', 'entity2': 'Oncogene'})]